<a href="https://colab.research.google.com/github/BetaUliansyah/automasi-portal-djpk/blob/main/Portal_TKDD_DJPK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automasi TKDD v3

In [ ]:
!pip install python-dotenv
import os
from dotenv import load_dotenv

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError
import datetime
from pytz import timezone
import urllib3
import json
from random import randint
import time
import pandas as pd

session = None
def google_drive():
    from google.colab import drive
    drive.mount('/content/drive')
    # pajak = pd.read_csv(path+filename, error_bad_lines=False)
    
    filename = "data-tkdd-"+ datetime.now(timezone('Asia/Jakarta')).strftime("%Y-%m-%d--%H-%M") + ".csv"
    path = F"/content/drive/My Drive/Colab Notebooks/Portal APBD/"

def set_global_session():
    from requests.adapters import HTTPAdapter
    from requests.packages.urllib3.util.retry import Retry
    
    retry_strategy = Retry(
        total=300,
        backoff_factor=0.5,
        status_forcelist=[429, 500, 502, 503, 504],
        method_whitelist=["HEAD", "GET", "OPTIONS"]
    )
    # Credits: https://findwork.dev/blog/advanced-usage-python-requests-timeouts-retries-hooks/

    MAX_RETRIES = 20
    global session
    if not session:
        adapter = HTTPAdapter(max_retries=retry_strategy)
        session = requests.Session()
        # adapter = requests.adapters.HTTPAdapter(max_retries=MAX_RETRIES) # Credits: https://stackoverflow.com/questions/33895739/python-requests-module-error-cant-load-any-url-remote-end-closed-connection
        session.mount('https://', adapter)
        session.mount('http://', adapter)
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        session.verify = False

def load_env():
    load_dotenv()
    global WACHAT_APIKEY
    WACHAT_APIKEY = os.getenv('WACHAT_APIKEY')
    return True

def get_token():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/tkdd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            token = bsoup.find("input", {"name":"_token"})['value']
            #print("Token: " + token)
            return token

def filter_tkdd(token, tahun, provinsi, pemda):
    t0 = time.time()
    try:
        with session.post('http://www.djpk.kemenkeu.go.id/portal/filter',
            data={'_token': token, 'data': 'tkdd ', 'tahun': tahun, 'provinsi': provinsi, 'pemda': pemda}) as response:
            if response.status_code==200:
                # return (json.loads(response.text))
                return response.text
    #except requests.exceptions.ConnectionError as e:
    #    pass
    except Exception as e:
           print('It failed :(', e.__class__.__name__)
    else:
        print('It eventually worked', response.status_code)
    finally:
        t1 = time.time()
        print('Took', t1 - t0, 'seconds')
    # Credits: https://www.peterbe.com/plog/best-practice-with-retries-with-requests

def get_kodeakun_tkdd(): # return 3 list values
    print("Getting Kode Akun TKDD...")
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/tkdd') as response:
        if response.status_code==200:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            token = bsoup.find("input", {"name":"_token"})['value']
            
            kodepostur_dict = {}
            kodesubpostur_dict = {}
            kodeakun_dict = {}
            koderow_dict = {}
            
            for tahun in range(2018, 2022): # populate all tahun
                print("Populating kode akun from tahun: " + str(tahun))
                result = filter_tkdd(token, tahun, "--", "--")
                json_result = json.loads(result)
                                
                # populate all kode akun
                if len(json_result) == 0:
                    continue
                for kode_postur in json_result['postur'].keys():
                    kodepostur_dict[kode_postur] = json_result['postur'][kode_postur]['text']
                    koderow_dict[kode_postur] = json_result['postur'][kode_postur]['text']
                    for kode_subpostur in json_result['postur'][kode_postur]['child'].keys():
                        kodesubpostur_dict[str(kode_postur)+str(kode_subpostur)] = json_result['postur'][kode_postur]['child'][kode_subpostur]['text']
                        koderow_dict[str(kode_postur)+str(kode_subpostur)] = json_result['postur'][kode_postur]['child'][kode_subpostur]['text']
                        for kodeakun_leaf in json_result['postur'][kode_postur]['child'][kode_subpostur]['child']:
                            kodeakun_dict[str(kode_postur)+str(kode_subpostur)+"_"+kodeakun_leaf['kodeakun']] = kodeakun_leaf['text']
                            koderow_dict[str(kode_postur)+str(kode_subpostur)+"_"+kodeakun_leaf['kodeakun']] = kodeakun_leaf['text']
            kodesubpostur_dict = dict(sorted(kodesubpostur_dict.items()))
            kodeakun_dict = dict(sorted(kodeakun_dict.items()))
            koderow_dict = dict(sorted(koderow_dict.items()))
            print("Populating kode akun completed")
            return kodepostur_dict, kodesubpostur_dict, kodeakun_dict, koderow_dict, token

def generate_header(koderows):
    print("Generating header...")
    # generate header
    header_data = ['no', 'tahun', 'kdpemda', 'nmpemda', 'kdprov', 'nmprov', 'wilayah', 'disclaimer',  'special_row' ]
    gabung_tail = list()
    for i in list(koderows.keys()):
        a = i + "_a"
        r = i + "_r"
        p = i + "_p"
        gabung_tail = gabung_tail + [a, r, p]
    header_data = header_data + gabung_tail
    return header_data
            
def parse_tkdd(json_result, kodeposturs, kodesubposturs, kodeakuns, koderows):
    print("Parsing...")
    data_dict = {}
    data_dict['wilayah'] = str(json_result['wilayah'])
    data_dict['disclaimer'] = str(json_result['disclaimer'])
    data_dict['special_row'] = json_result['special_row']

    for kodepostur in kodeposturs.keys():
        if kodepostur in json_result['postur'].keys():
            data_dict[kodepostur + "_a"] = str(json_result['postur'][str(kodepostur)]['total']['anggaran'])
            data_dict[kodepostur + "_r"] = str(json_result['postur'][kodepostur]['total']['realisasi'])
            data_dict[kodepostur + "_p"] = str(json_result['postur'][kodepostur]['total']['persentase']).replace(",",".").replace(" ","")
        for kodesubpostur in kodesubposturs:
            if kodepostur in json_result['postur'].keys() and kodesubpostur[-1:] in json_result['postur'][kodepostur]['child'].keys():
                data_dict[kodesubpostur + "_a"] = str(json_result['postur'][kodepostur]['child'][str(kodesubpostur[-1:])]['total']['anggaran'])
                data_dict[kodesubpostur + "_r"] = str(json_result['postur'][kodepostur]['child'][str(kodesubpostur[-1:])]['total']['realisasi'])
                data_dict[kodesubpostur + "_p"] = str(json_result['postur'][kodepostur]['child'][str(kodesubpostur[-1:])]['total']['persentase']).replace(",",".").replace(" ","")
            i = 0
            for kodeakun in kodeakuns:
                if kodepostur == kodeakun[:1] and kodesubpostur[-1:] == kodeakun[1:2] and kodepostur in json_result['postur'].keys() and kodesubpostur[-1:] in json_result['postur'][kodepostur]['child'].keys():
                    if i < len(json_result['postur'][kodepostur]['child'][str(kodesubpostur[-1:])]['child']) and kodeakun[-6:] in json_result['postur'][kodepostur]['child'][str(kodesubpostur[-1:])]['child'][i]['kodeakun']:
                        data_dict[kodeakun + "_a"] = json_result['postur'][kodepostur]['child'][str(kodesubpostur[-1:])]['child'][i]['value']['anggaran']
                        data_dict[kodeakun + "_r"] = str(json_result['postur'][kodepostur]['child'][str(kodesubpostur[-1:])]['child'][i]['value']['realisasi'])
                        data_dict[kodeakun + "_p"] = str(json_result['postur'][kodepostur]['child'][str(kodesubpostur[-1:])]['child'][i]['value']['persentase']).replace(",",".").replace(" ","")
                    
                    i = i + 1            
    return data_dict

def get_all_provinsi(): # return dict with kdprov as key, nmprov as value
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/tkdd') as response:
        if response.status_code==200:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            allprovinsi = bsoup.find("select", {"id":"sel_prov"}) 
            provinsi = {}
            for option_provinsi in allprovinsi.find_all("option"): # populate all provinsi
               provinsi[option_provinsi['value']] = option_provinsi.text
            return provinsi

def get_pemdas(provinsi): # return dict with kdpemda as key, nmpemda as value
    with session.get('http://www.djpk.kemenkeu.go.id/portal/pemda/' + provinsi) as response:
        if response.status_code==200 and len(response.text) != 0:
            pemdas = json.loads(response.text)
            return pemdas

def wachat_send_message(tujuan, pesan, sender, apikey='F0C584900AB90E1040862FC0B43F561E'):
    HEADERS = {"Accept": "application/json", "APIKey": apikey }
    PAYLOADS = {'destination': tujuan, 'sender': sender, 'message': pesan}

    with session.post(
        'https://api.wachat-api.com/wachat_api/1.0/message',
        headers=HEADERS,
        json=PAYLOADS,
        # Skip SSL Verification
        # verify=False
    ) as response:
        if response.status_code==200:
            #print("Status code: " + str(r.status_code))
            #print("Response: " + r.text)
            return True

# loop

In [ ]:
if __name__ == "__main__":
    set_global_session()
    load_env()
    start_time = datetime.datetime.now(timezone('Asia/Jakarta'))
    print("Start script at: " + datetime.datetime.now(timezone('Asia/Jakarta')).strftime("%-d-%m-%Y") + " at " + str(start_time))
    
    kodeposturs, kodesubposturs, kodeakuns, koderows, token = get_kodeakun_tkdd()

    # generate header
    header_data = generate_header(koderows)
    # generate prov
    allprovinsi = get_all_provinsi()
    
    datadf = pd.DataFrame(columns = header_data)
    print("Are you executing this?")
    for tahun in range(2018, 2022):
        print("Executing tahun: ", tahun)
        for kdprov, nmprov in allprovinsi.items():
            print("Executing provinsi loop: ", kdprov, nmprov)
            for kdpemda, nmpemda in get_pemdas(kdprov).items():    
                print("Executing pemda loop: ", tahun, kdpemda, nmpemda)
                result_dict = {}
                pemda_result = filter_tkdd(token, tahun, kdprov, kdpemda)
                if pemda_result is None:
                    continue
                json_result = json.loads(pemda_result)
                result_dict = parse_tkdd(json_result, kodeposturs, kodesubposturs, kodeakuns, koderows)
                result_dict['tahun'] = tahun
                result_dict['kdpemda'] = kdpemda
                result_dict['nmpemda'] = nmpemda
                result_dict['kdprov'] = kdprov
                result_dict['nmprov'] = nmprov
                print("Get data tahun", tahun, "prov", nmprov, "pemda", nmpemda)
                print(result_dict)
                datadf = datadf.append(result_dict, ignore_index = True)
                filename = "data-tkdd-"+ datetime.datetime.now(timezone('Asia/Jakarta')).strftime("%Y-%m-%d--%H-%M") + ".csv"
                datadf.to_csv(filename)
        pesan = 'Get data TKDD tahun ' + str(tahun) + ' sudah selesai'
        wachat_send_message('628567074554', pesan, '6282189096866', apikey='F0C584900AB90E1040862FC0B43F561E')
    print("datadf:")
    print(datadf)
    filename = "data-tkdd-"+ datetime.datetime.now(timezone('Asia/Jakarta')).strftime("%Y-%m-%d--%H-%M") + ".csv"
    datadf.to_csv(filename)

    duration = datetime.datetime.now(timezone('Asia/Jakarta')) - start_time
    print("Duration: " + str(duration))            

Streaming output truncated to the last 5000 lines.
Took 0.5958027839660645 seconds
Parsing...
Get data tahun 2020 prov Prov. Lampung pemda Kab. Lampung Tengah
{'wilayah': 'se-Kab. Lampung Tengah', 'disclaimer': 'Realisasi Berdasarkan SP2D', 'special_row': {'text': 'Transfer ke daerah dan Dana Desa', 'total': {'anggaran': 1988953156884, 'realisasi': 1974454329630, 'persentase': '99,27 %'}}, '1_a': '1678776179884', '1_r': '1664277352630', '1_p': '99.14%', '11_a': '310176977000', '11_r': '310176977000', '11_p': '100.00%', '11_611111_a': 13565100737, '11_611111_r': '13565100737', '11_611111_p': '100.00%', '11_611112_a': 769146346, '11_611112_r': '769146346', '11_611112_p': '100.00%', '12_a': '1259528176000', '12_r': '1248686762000', '12_p': '99.14%', '12_621111_a': 1259528176000, '12_621111_r': '1248686762000', '12_621111_p': '99.14%', '13_a': '62777496000', '13_r': '59050683193', '13_p': '94.06%', '13_631111_a': 45184074000, '13_631111_r': '42698499221', '13_631111_p': '94.50%', '14_a': '

In [ ]:
filename = "final-tkdd-"+ datetime.datetime.now(timezone('Asia/Jakarta')).strftime("%Y-%m-%d--%H-%M") + ".csv"
datadf.to_csv(filename)

# single

In [ ]:
if __name__ == "__main__":
    set_global_session()
    start_time = datetime.datetime.now(timezone('Asia/Jakarta'))
    print("Start script at: " + datetime.datetime.now(timezone('Asia/Jakarta')).strftime("%-d-%m-%Y") + " at " + str(start_time))
    
    kodeposturs, kodesubposturs, kodeakuns, koderows, token = get_kodeakun_tkdd()

    # generate header
    header_data = generate_header(koderows)

    # query single
    tahun = '2019'
    kdprov = '--'
    kdpemda = '--'
    result_dict = {}
    pemda_result = filter_tkdd(token, tahun, kdprov, kdpemda)
    
    if pemda_result is None:
        print("Kosong!")
    else:
        json_result = json.loads(pemda_result)
        print(json_result)
        result_dict = parse_tkdd(json_result, kodeposturs, kodesubposturs, kodeakuns, koderows) 
        filename = "data-tkdd-"+ datetime.datetime.now(timezone('Asia/Jakarta')).strftime("%Y-%m-%d--%H-%M") + ".csv"
        datadf = pd.DataFrame(result_dict, columns=header_data)
        datadf.to_csv(filename)
    duration = datetime.datetime.now(timezone('Asia/Jakarta')) - start_time
    print("Duration: " + str(duration))            

Start script at: 17-10-2021 at 2021-10-17 21:01:24.737303+07:00
Getting Kode Akun TKDD...
Populating kode akun from tahun: 2018
Took 1.1131391525268555 seconds
Populating kode akun from tahun: 2019
Took 0.5986185073852539 seconds
Populating kode akun from tahun: 2020
Took 0.7371256351470947 seconds
Populating kode akun from tahun: 2021
Took 0.6241238117218018 seconds
Populating kode akun completed
Generating header...
Took 0.5921094417572021 seconds
{'postur': {'1': {'text': 'Transfer ke Daerah', 'child': {'1': {'text': 'DBH', 'child': [{'kodeakun': '611111', 'text': 'DBH PPh Pasal 21', 'value': {'anggaran': 25125437153933, 'realisasi': 20714393426624, 'persentase': '82,44 %'}}, {'kodeakun': '611112', 'text': 'DBH PPh Pasal 25/29 OP', 'value': {'anggaran': 1892004681401, 'realisasi': 1358694128906, 'persentase': '71,81 %'}}, {'kodeakun': '611211', 'text': 'DBH PBB Bagian Daerah untuk Provinsi', 'value': {'anggaran': 3854596935692, 'realisasi': 2979116026823, 'persentase': '77,29 %'}}, 

## test mumet

In [ ]:
for kode_postur in json_result['postur'].keys():
    print("Kode Postur:")
    print(kode_postur)
    print(json_result['postur'][kode_postur]['text'])
    for kode_subpostur in json_result['postur'][kode_postur]['child'].keys():
        print("\t", "Kode Subpostur:")
        print("\t", kode_subpostur)
        print("\t", json_result['postur'][kode_postur]['child'][kode_subpostur]['text'])
        i = 0
        for kodeakun_leaf in json_result['postur'][kode_postur]['child'][kode_subpostur]['child']:
            print(kodeakun_leaf)
            print(kodeakun_leaf['kodeakun'])
            print(kodeakun_leaf['text'])
            print(kodeakun_leaf['value']['anggaran'])
            print(kodeakun_leaf['value']['realisasi'])
            print(kodeakun_leaf['value']['persentase'])
            print(json_result['postur'][kode_postur]['child'][kode_subpostur]['child'])
            print(json_result['postur'][kode_postur]['child'][kode_subpostur]['child'][i])
            print(json_result['postur'][kode_postur]['child'][kode_subpostur]['child'][i]['value']['anggaran'])
            
            i = i + 1

Kode Postur:
1
Transfer ke Daerah
	 Kode Subpostur:
	 1
	 DBH
{'kodeakun': '611111', 'text': 'DBH PPh Pasal 21', 'value': {'anggaran': 25125437153933, 'realisasi': 20714393426624, 'persentase': '82,44 %'}}
611111
DBH PPh Pasal 21
25125437153933
20714393426624
82,44 %
[{'kodeakun': '611111', 'text': 'DBH PPh Pasal 21', 'value': {'anggaran': 25125437153933, 'realisasi': 20714393426624, 'persentase': '82,44 %'}}, {'kodeakun': '611112', 'text': 'DBH PPh Pasal 25/29 OP', 'value': {'anggaran': 1892004681401, 'realisasi': 1358694128906, 'persentase': '71,81 %'}}, {'kodeakun': '611211', 'text': 'DBH PBB Bagian Daerah untuk Provinsi', 'value': {'anggaran': 3854596935692, 'realisasi': 2979116026823, 'persentase': '77,29 %'}}, {'kodeakun': '611212', 'text': 'DBH PBB Bagian Daerah untuk Kabupaten/Kota', 'value': {'anggaran': 15492819558079, 'realisasi': 11771315922666, 'persentase': '75,98 %'}}, {'kodeakun': '611213', 'text': 'DBH PBB Biaya Pemungutan untuk Provinsi', 'value': {'anggaran': 1077960

# Portal TKDD awal

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import urllib3
import json

session = None
# Snippet Save to Google Drive
from datetime import datetime
from pytz import timezone
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

def set_global_session():
    global session
    if not session:
        session = requests.Session()
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        session.verify = False

def get_alldata():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            token = bsoup.find("input", {"name":"_token"})['value']
            options_provinsi = bsoup.find("select", {"id":"sel_prov"})
            allprovinsi = {}
            for option_provinsi in options_provinsi.find_all("option"):
                allprovinsi[option_provinsi['value']]=option_provinsi.text 
            return [token, allprovinsi]

def get_token():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            token = bsoup.find("input", {"name":"_token"})['value']
            return token

def get_provinsi():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            options_provinsi = bsoup.find("select", {"id":"sel_prov"})
            allprovinsi = []
            for option_provinsi in options_provinsi.find_all("option"):
                allprovinsi[option_provinsi['value']]=option_provinsi['value'].text
            return allprovinsi

def get_pemdas(provinsi):
    with session.get('http://www.djpk.kemenkeu.go.id/portal/pemda/' + provinsi) as response:
        if response.status_code==200 and len(response.text) != 0:
            pemdas = json.loads(response.text)
            return pemdas

def get_headers(token, tahun=2018):
    with session.post('http://www.djpk.kemenkeu.go.id/portal/filter', 
                data={'_token': token, 'data': 'tkdd', 'tahun': tahun, 'provinsi': '--', 'pemda': '--'}) as response:# query nasional saja as response
        if response.status_code==200 and len(response.text) != 0:
            print(response.text)
            # olah gaes
            # {"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Kabupaten\/kota","value":{"anggaran":398764072000,"realisasi":390261862177,"persentase":"97,87 %"}},{"kodeakun":"611215","text":"Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten\/Kota","value":{"anggaran":1736910100000,"realisasi":1650781100705,"persentase":"95,04 %"}},{"kodeakun":"612111","text":"Transfer DBH Minyak Bumi","value":{"anggaran":8581982674000,"realisasi":12376493484658,"persentase":"144,21 %"}},{"kodeakun":"612112","text":"Transfer DBH Minyak Bumi 0.5%","value":{"anggaran":282732216000,"realisasi":407714843494,"persentase":"144,21 %"}},{"kodeakun":"612113","text":"Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus","value":{"anggaran":370605160000,"realisasi":481008154249,"persentase":"129,79 %"}},{"kodeakun":"612211","text":"Transfer DBH Gas Bumi","value":{"anggaran":5966265026000,"realisasi":8745074752035,"persentase":"146,58 %"}},{"kodeakun":"612212","text":"Transfer DBH Gas Bumi 0.5%","value":{"anggaran":96882460000,"realisasi":124683590580,"persentase":"128,70 %"}},{"kodeakun":"612213","text":"Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus","value":{"anggaran":270583164000,"realisasi":340653879134,"persentase":"125,90 %"}},{"kodeakun":"612311","text":"Transfer DBH Pertambangan Umum - Iuran Tetap","value":{"anggaran":615858746000,"realisasi":574574040835,"persentase":"93,30 %"}},{"kodeakun":"612312","text":"Transfer DBH Pertambangan Umum - Royalti","value":{"anggaran":13670958915000,"realisasi":19233330257405,"persentase":"140,69 %"}},{"kodeakun":"612411","text":"Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah","value":{"anggaran":545092142000,"realisasi":814945712335,"persentase":"149,51 %"}},{"kodeakun":"612412","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Tetap","value":{"anggaran":14680315000,"realisasi":17093550748,"persentase":"116,44 %"}},{"kodeakun":"612413","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)","value":{"anggaran":703068000,"realisasi":897327311,"persentase":"127,63 %"}},{"kodeakun":"612511","text":"Transfer DBH Kehutanan - IIUPH\/IHPH","value":{"anggaran":164730247000,"realisasi":165979915021,"persentase":"100,76 %"}},{"kodeakun":"612512","text":"Transfer DBH Kehutanan - PSDH","value":{"anggaran":736821430000,"realisasi":706696713825,"persentase":"95,91 %"}},{"kodeakun":"612513","text":"Transfer DBH Kehutanan - Dana Reboisasi","value":{"anggaran":743479609000,"realisasi":741420178157,"persentase":"99,72 %"}},{"kodeakun":"612611","text":"Transfer DBH Perikanan","value":{"anggaran":480000648000,"realisasi":357622699445,"persentase":"74,50 %"}},{"kodeakun":"613111","text":"Transfer DBH Cukai Hasil Tembakau","value":{"anggaran":2964600000000,"realisasi":2922191165245,"persentase":"98,57 %"}}],"total":{"anggaran":89225342014000,"realisasi":93705559169270,"persentase":"105,02 %"}},"2":{"text":"DAU","child":[{"kodeakun":"621111","text":"Dana Alokasi Umum","value":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}}],"total":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}},"3":{"text":"DAK FISIK","child":[{"kodeakun":"631111","text":"Dana Alokasi Khusus","value":{"anggaran":31350835954000,"realisasi":29459738989382,"persentase":"93,97 %"}},{"kodeakun":"631211","text":"Dana Alokasi Khusus Infrastruktur Publik Daerah","value":{"anggaran":0,"realisasi":null,"persentase":"-"}},{"kodeakun":"631212","text":"Dana Alokasi Khusus Penugasan","value":{"anggaran":24463658880000,"realisasi":22492927163424,"persentase":"91,94 %"}},{"kodeakun":"631311","text":"Dana Alokasi Khusus Affirmasi","value":{"anggaran":6621768074000,"realisasi":6259405435103,"persentase":"94,53 %"}}],"total":{"anggaran":62436262908000,"realisasi":58212071587909,"persentase":"93,23 %"}},"6":{"text":"OTSUS","child":[{"kodeakun":"641111","text":"Dana Otonomi Khusus Provinsi Aceh","value":{"anggaran":8029791593000,"realisasi":8029791593000,"persentase":"100,00 %"}},{"kodeakun":"641211","text":"Dana Otonomi Khusus Provinsi Papua","value":{"anggaran":5620854115000,"realisasi":5620854115000,"persentase":"100,00 %"}},{"kodeakun":"641212","text":"Dana Tambahan Infrastruktur Provinsi Papua","value":{"anggaran":2400000000000,"realisasi":2400000000000,"persentase":"100,00 %"}},{"kodeakun":"641311","text":"Dana Otonomi Khusus Provinsi Papua Barat","value":{"anggaran":2408937478000,"realisasi":2408937478000,"persentase":"100,00 %"}},{"kodeakun":"641312","text":"Dana Tambahan Infrastruktur Provinsi Papua Barat","value":{"anggaran":1600000000000,"realisasi":1600000000000,"persentase":"100,00 %"}}],"total":{"anggaran":20059583186000,"realisasi":20059583186000,"persentase":"100,00 %"}},"7":{"text":"KEISTIMEWAAN DIY","child":[{"kodeakun":"642111","text":"Dana Keistimewaan Daerah Istimewa Yogyakarta","value":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}}],"total":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}},"5":{"text":"DID","child":[{"kodeakun":"643111","text":"Dana Insentif Daerah","value":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}}],"total":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}},"4":{"text":"DAK NON FISIK","child":[{"kodeakun":"654111","text":"Dana Tunjangan Profesi Guru PNSD","value":{"anggaran":56861747531798,"realisasi":55016720212332,"persentase":"96,76 %"}},{"kodeakun":"654112","text":"Dana Tunjangan Khusus Guru PNSD di Daerah Khusus","value":{"anggaran":1922303294676,"realisasi":1727102404861,"persentase":"89,85 %"}},{"kodeakun":"654211","text":"Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil","value":{"anggaran":879468864366,"realisasi":588007130574,"persentase":"66,86 %"}},{"kodeakun":"654311","text":"Dana Bantuan Operasional Sekolah","value":{"anggaran":46311243600000,"realisasi":44367351931006,"persentase":"95,80 %"}},{"kodeakun":"654711","text":"Dana Bantuan Operasional Kesehatan","value":{"anggaran":8551232700000,"realisasi":7454358890915,"persentase":"87,17 %"}},{"kodeakun":"654712","text":"Dana Bantuan Operasional Keluarga Berencana","value":{"anggaran":1808787300000,"realisasi":1753419417979,"persentase":"96,94 %"}},{"kodeakun":"654811","text":"Dana Peningkatan Kapasitas Koperasi","value":{"anggaran":100000000000,"realisasi":92221616897,"persentase":"92,22 %"}},{"kodeakun":"654814","text":"Dana Pelayanan Administrasi Kependudukan","value":{"anggaran":825000000000,"realisasi":745193742984,"persentase":"90,33 %"}},{"kodeakun":"654911","text":"Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini","value":{"anggaran":3741708600000,"realisasi":3555904601891,"persentase":"95,03 %"}}],"total":{"anggaran":121001491890840,"realisasi":115300279949439,"persentase":"95,29 %"}}},"total":{"anggaran":703712259647840,"realisasi":697657198541618,"persentase":"99,14 %"}},"2":{"text":"Dana Desa","child":{"1":{"text":"DANA DESA","child":[{"kodeakun":"662111","text":"Dana Desa","value":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}],"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"wilayah":"Nasional","judul":"POSTUR TKDD TA 2018","link_excel":"http:\/\/www.djpk.kemenkeu.go.id\/portal\/data\/csv\/2018\/--\/--\/tkdd","disclaimer":"Data bulan desember","header":["Alokasi","Realisasi"],"special_row":{"text":"Transfer ke daerah dan Dana Desa","total":{"anggaran":763712259647840,"realisasi":757533338915843,"persentase":"99,19 %"}}}

            kodeakun = 0

            return kodeakun

def get_data_tkdd(token, provinsi, pemda, tahun=2018):
    try:
        with session.post('http://www.djpk.kemenkeu.go.id/portal/filter', 
                    data={'_token': token, 'data': 'tkdd', 'tahun': tahun, 'provinsi': provinsi, 'pemda': pemda}) as response:
            if response.status_code==200 and len(response.text) != 0:
                return response.text
    except requests.exceptions.ConnectionError as e:
        return False
        pass
    except Exception as e:
        logger.error(e)
        randomtime = random.randint(1,5)
        logger.warn('ERROR - Retrying again website %s, retrying in %d secs' % (url, randomtime))
        time.sleep(randomtime)
        pass
        return False

def save_to_gdrive():
    path = F"/content/drive/My Drive/Colab Notebooks/Data DJSN/"
    filename = "data-tkdd-"+ datetime.now(timezone('Asia/Jakarta')).strftime("%Y-%m-%d--%H-%M") + ".csv"
    return True

if __name__ == "__main__":
    set_global_session()
    data = []
    allprovinsi = []
    # tahun yg ada datanya hanya 2018-2019
    token, allprovinsi = get_alldata()
    # loop tahun
    for tahun in range(2018, 2021):
        if tahun == '2000':
            continue
        # loop provinsi
        for provinsi in allprovinsi:
            pemdas = get_pemdas(provinsi)
        
            # loop pemda per provinsi
            for pemda in pemdas.keys():
                datapemda = json.loads(get_data_tkdd(token, provinsi, pemda))
                print(datapemda)
                if len(datapemda) == 0:
                     continue
                wilayah = datapemda['wilayah']
                provinsi = allprovinsi[provinsi]
                disclaimer = datapemda['disclaimer']

                tkdd_total_a = datapemda['special_row']['total']['anggaran']
                tkdd_total_r = datapemda['special_row']['total']['realisasi']
                tkdd_total_p = datapemda['special_row']['total']['persentase']
                
                tkd_total_a = datapemda['postur']['1']['total']['anggaran']
                tkd_total_r = datapemda['postur']['1']['total']['realisasi']
                tkd_total_p = datapemda['postur']['1']['total']['persentase']
                
                dbh_total_a = datapemda['postur']['1']['total']['anggaran']
                dbh_total_r = datapemda['postur']['1']['total']['realisasi']
                dbh_total_p = datapemda['postur']['1']['total']['persentase']

                dbh_pph21_total_kodeakun = datapemda['postur']['1']['child']['0']['kodeakun']
                dbh_pph21_total_a = datapemda['postur']['1']['child']['0']['value']['anggaran']
                dbh_pph21_total_r = datapemda['postur']['1']['child']['0']['value']['realisasi']
                dbh_pph21_total_p = datapemda['postur']['1']['child']['0']['value']['persentase']

                
                
                print(wilayah, provinsi, disclaimer, 
                      tkdd_total_a, tkdd_total_r, tkdd_total_p, 
                      tkd_total_a, tkd_total_r, tkd_total_p, 
                      dbh_total_a, dbh_total_r, dbh_total_p, 
                      dbh_pph21_total_kodeakun, dbh_pph21_total_a, dbh_pph21_total_r, dbh_pph21_total_p)
                # ['header'][0] -> 'Alokasi'
                #data.append(i, wilayah)

                #i = i+1
                

    # save via DataFrame
    #df = pd.DataFrame(data)
    #df.to_csv( path+filename , index=False, encoding="utf-8")                

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{'postur': {'1': {'text': 'Transfer ke Daerah', 'child': {'1': {'text': 'DBH', 'child': [{'kodeakun': '611111', 'text': 'Transfer DBH PPh Pasal 21', 'value': {'anggaran': 32986586000000, 'realisasi': 27458081283272, 'persentase': '83,24 %'}}, {'kodeakun': '611112', 'text': 'Transfer DBH PPh Pasal 25/29 OP', 'value': {'anggaran': 4441882000000, 'realisasi': 2533606329484, 'persentase': '57,04 %'}}, {'kodeakun': '611211', 'text': 'Transfer DBH PBB untuk Propinsi', 'value': {'anggaran': 2857407829000, 'realisasi': 2737579489820, 'persentase': '95,81 %'}}, {'kodeakun': '611212', 'text': 'Transfer DBH PBB untuk Kabupaten/Kota', 'value': {'anggaran': 11211563981000, 'realisasi': 10840669654158, 'persentase': '96,69 %'}}, {'kodeakun': '611213', 'text': 'Transfer DBH Biaya/Upah Pungut PBB untuk Propinsi', 'value': {'anggaran': 86252212000, 'realisasi': 84199185177, '

KeyError: ignored

# Metani

In [ ]:
import json
datapemda = json.loads('{"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Kabupaten\/kota","value":{"anggaran":398764072000,"realisasi":390261862177,"persentase":"97,87 %"}},{"kodeakun":"611215","text":"Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten\/Kota","value":{"anggaran":1736910100000,"realisasi":1650781100705,"persentase":"95,04 %"}},{"kodeakun":"612111","text":"Transfer DBH Minyak Bumi","value":{"anggaran":8581982674000,"realisasi":12376493484658,"persentase":"144,21 %"}},{"kodeakun":"612112","text":"Transfer DBH Minyak Bumi 0.5%","value":{"anggaran":282732216000,"realisasi":407714843494,"persentase":"144,21 %"}},{"kodeakun":"612113","text":"Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus","value":{"anggaran":370605160000,"realisasi":481008154249,"persentase":"129,79 %"}},{"kodeakun":"612211","text":"Transfer DBH Gas Bumi","value":{"anggaran":5966265026000,"realisasi":8745074752035,"persentase":"146,58 %"}},{"kodeakun":"612212","text":"Transfer DBH Gas Bumi 0.5%","value":{"anggaran":96882460000,"realisasi":124683590580,"persentase":"128,70 %"}},{"kodeakun":"612213","text":"Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus","value":{"anggaran":270583164000,"realisasi":340653879134,"persentase":"125,90 %"}},{"kodeakun":"612311","text":"Transfer DBH Pertambangan Umum - Iuran Tetap","value":{"anggaran":615858746000,"realisasi":574574040835,"persentase":"93,30 %"}},{"kodeakun":"612312","text":"Transfer DBH Pertambangan Umum - Royalti","value":{"anggaran":13670958915000,"realisasi":19233330257405,"persentase":"140,69 %"}},{"kodeakun":"612411","text":"Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah","value":{"anggaran":545092142000,"realisasi":814945712335,"persentase":"149,51 %"}},{"kodeakun":"612412","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Tetap","value":{"anggaran":14680315000,"realisasi":17093550748,"persentase":"116,44 %"}},{"kodeakun":"612413","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)","value":{"anggaran":703068000,"realisasi":897327311,"persentase":"127,63 %"}},{"kodeakun":"612511","text":"Transfer DBH Kehutanan - IIUPH\/IHPH","value":{"anggaran":164730247000,"realisasi":165979915021,"persentase":"100,76 %"}},{"kodeakun":"612512","text":"Transfer DBH Kehutanan - PSDH","value":{"anggaran":736821430000,"realisasi":706696713825,"persentase":"95,91 %"}},{"kodeakun":"612513","text":"Transfer DBH Kehutanan - Dana Reboisasi","value":{"anggaran":743479609000,"realisasi":741420178157,"persentase":"99,72 %"}},{"kodeakun":"612611","text":"Transfer DBH Perikanan","value":{"anggaran":480000648000,"realisasi":357622699445,"persentase":"74,50 %"}},{"kodeakun":"613111","text":"Transfer DBH Cukai Hasil Tembakau","value":{"anggaran":2964600000000,"realisasi":2922191165245,"persentase":"98,57 %"}}],"total":{"anggaran":89225342014000,"realisasi":93705559169270,"persentase":"105,02 %"}},"2":{"text":"DAU","child":[{"kodeakun":"621111","text":"Dana Alokasi Umum","value":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}}],"total":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}},"3":{"text":"DAK FISIK","child":[{"kodeakun":"631111","text":"Dana Alokasi Khusus","value":{"anggaran":31350835954000,"realisasi":29459738989382,"persentase":"93,97 %"}},{"kodeakun":"631211","text":"Dana Alokasi Khusus Infrastruktur Publik Daerah","value":{"anggaran":0,"realisasi":null,"persentase":"-"}},{"kodeakun":"631212","text":"Dana Alokasi Khusus Penugasan","value":{"anggaran":24463658880000,"realisasi":22492927163424,"persentase":"91,94 %"}},{"kodeakun":"631311","text":"Dana Alokasi Khusus Affirmasi","value":{"anggaran":6621768074000,"realisasi":6259405435103,"persentase":"94,53 %"}}],"total":{"anggaran":62436262908000,"realisasi":58212071587909,"persentase":"93,23 %"}},"6":{"text":"OTSUS","child":[{"kodeakun":"641111","text":"Dana Otonomi Khusus Provinsi Aceh","value":{"anggaran":8029791593000,"realisasi":8029791593000,"persentase":"100,00 %"}},{"kodeakun":"641211","text":"Dana Otonomi Khusus Provinsi Papua","value":{"anggaran":5620854115000,"realisasi":5620854115000,"persentase":"100,00 %"}},{"kodeakun":"641212","text":"Dana Tambahan Infrastruktur Provinsi Papua","value":{"anggaran":2400000000000,"realisasi":2400000000000,"persentase":"100,00 %"}},{"kodeakun":"641311","text":"Dana Otonomi Khusus Provinsi Papua Barat","value":{"anggaran":2408937478000,"realisasi":2408937478000,"persentase":"100,00 %"}},{"kodeakun":"641312","text":"Dana Tambahan Infrastruktur Provinsi Papua Barat","value":{"anggaran":1600000000000,"realisasi":1600000000000,"persentase":"100,00 %"}}],"total":{"anggaran":20059583186000,"realisasi":20059583186000,"persentase":"100,00 %"}},"7":{"text":"KEISTIMEWAAN DIY","child":[{"kodeakun":"642111","text":"Dana Keistimewaan Daerah Istimewa Yogyakarta","value":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}}],"total":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}},"5":{"text":"DID","child":[{"kodeakun":"643111","text":"Dana Insentif Daerah","value":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}}],"total":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}},"4":{"text":"DAK NON FISIK","child":[{"kodeakun":"654111","text":"Dana Tunjangan Profesi Guru PNSD","value":{"anggaran":56861747531798,"realisasi":55016720212332,"persentase":"96,76 %"}},{"kodeakun":"654112","text":"Dana Tunjangan Khusus Guru PNSD di Daerah Khusus","value":{"anggaran":1922303294676,"realisasi":1727102404861,"persentase":"89,85 %"}},{"kodeakun":"654211","text":"Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil","value":{"anggaran":879468864366,"realisasi":588007130574,"persentase":"66,86 %"}},{"kodeakun":"654311","text":"Dana Bantuan Operasional Sekolah","value":{"anggaran":46311243600000,"realisasi":44367351931006,"persentase":"95,80 %"}},{"kodeakun":"654711","text":"Dana Bantuan Operasional Kesehatan","value":{"anggaran":8551232700000,"realisasi":7454358890915,"persentase":"87,17 %"}},{"kodeakun":"654712","text":"Dana Bantuan Operasional Keluarga Berencana","value":{"anggaran":1808787300000,"realisasi":1753419417979,"persentase":"96,94 %"}},{"kodeakun":"654811","text":"Dana Peningkatan Kapasitas Koperasi","value":{"anggaran":100000000000,"realisasi":92221616897,"persentase":"92,22 %"}},{"kodeakun":"654814","text":"Dana Pelayanan Administrasi Kependudukan","value":{"anggaran":825000000000,"realisasi":745193742984,"persentase":"90,33 %"}},{"kodeakun":"654911","text":"Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini","value":{"anggaran":3741708600000,"realisasi":3555904601891,"persentase":"95,03 %"}}],"total":{"anggaran":121001491890840,"realisasi":115300279949439,"persentase":"95,29 %"}}},"total":{"anggaran":703712259647840,"realisasi":697657198541618,"persentase":"99,14 %"}},"2":{"text":"Dana Desa","child":{"1":{"text":"DANA DESA","child":[{"kodeakun":"662111","text":"Dana Desa","value":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}],"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"wilayah":"Nasional","judul":"POSTUR TKDD TA 2018","link_excel":"http:\/\/www.djpk.kemenkeu.go.id\/portal\/data\/csv\/2018\/--\/--\/tkdd","disclaimer":"Data bulan desember","header":["Alokasi","Realisasi"],"special_row":{"text":"Transfer ke daerah dan Dana Desa","total":{"anggaran":763712259647840,"realisasi":757533338915843,"persentase":"99,19 %"}}}')

# 1 Total TKDD
tkdd_total_a = datapemda['special_row']['total']['anggaran']
tkdd_total_r = datapemda['special_row']['total']['realisasi']
tkdd_total_a_header = "Total TKDD Anggaran"
tkdd_total_r_header = "Total TKDD Realisasi"

# 2 Total TKD
tkd_total_a = datapemda['postur']['1']['total']['anggaran']
tkd_total_r = datapemda['postur']['1']['total']['realisasi']

# 3 Total DBH
dbh_total_a = datapemda['postur']['1']['child']['1']['total']['anggaran']
dbh_total_r = datapemda['postur']['1']['child']['1']['total']['realisasi']
dbh_total_p = datapemda['postur']['1']['child']['1']['total']['persentase']

# 4
dbh_pph21                 = datapemda['postur']['1']['child']['1']['child'][0]
dbh_pph21_total_a         = dbh_pph21['value']['anggaran']
dbh_pph21_total_r         = dbh_pph21['value']['realisasi']
dbh_pph21_total_p         = dbh_pph21['value']['persentase']

# 5
dbh_pph2529opdn                 = datapemda['postur']['1']['child']['1']['child'][1]
dbh_pph2529opdn_total_a         = dbh_pph2529opdn['value']['anggaran']
dbh_pph2529opdn_total_r         = dbh_pph2529opdn['value']['realisasi']
dbh_pph2529opdn_total_p         = dbh_pph2529opdn['value']['persentase']

# 6
dbh_pbbprovinsi                 = datapemda['postur']['1']['child']['1']['child'][2]
dbh_pbbprovinsi_total_a         = dbh_pbbprovinsi['value']['anggaran']
dbh_pbbprovinsi_total_r         = dbh_pbbprovinsi['value']['realisasi']
dbh_pbbprovinsi_total_p         = dbh_pbbprovinsi['value']['persentase']

# 7
dbh_pbbkabkot                 = datapemda['postur']['1']['child']['1']['child'][3]
dbh_pbbkabkot_total_a         = dbh_pbbkabkot['value']['anggaran']
dbh_pbbkabkot_total_r         = dbh_pbbkabkot['value']['realisasi']
dbh_pbbkabkot_total_p         = dbh_pbbkabkot['value']['persentase']

# 8
dbh_pbbupahpungutprov                 = datapemda['postur']['1']['child']['1']['child'][4]
dbh_pbbupahpungutprov_total_a         = dbh_pbbupahpungutprov['value']['anggaran']
dbh_pbbupahpungutprov_total_r         = dbh_pbbupahpungutprov['value']['realisasi']
dbh_pbbupahpungutprov_total_p         = dbh_pbbupahpungutprov['value']['persentase']

# 9
dbh_pbbupahpungutkabkot                 = datapemda['postur']['1']['child']['1']['child'][5]
dbh_pbbupahpungutkabkot_total_a         = dbh_pbbupahpungutkabkot['value']['anggaran']
dbh_pbbupahpungutkabkot_total_r         = dbh_pbbupahpungutkabkot['value']['realisasi']
dbh_pbbupahpungutkabkot_total_p         = dbh_pbbupahpungutkabkot['value']['persentase']

# 10
dbh_pbbbagpusatrata                 = datapemda['postur']['1']['child']['1']['child'][6]
dbh_pbbbagpusatrata_total_a         = dbh_pbbbagpusatrata['value']['anggaran']
dbh_pbbbagpusatrata_total_r         = dbh_pbbbagpusatrata['value']['realisasi']
dbh_pbbbagpusatrata_total_p         = dbh_pbbbagpusatrata['value']['persentase']

# 11
dbh_minyak                 = datapemda['postur']['1']['child']['1']['child'][7]
dbh_minyak_total_a         = dbh_minyak['value']['anggaran']
dbh_minyak_total_r         = dbh_minyak['value']['realisasi']
dbh_minyak_total_p         = dbh_minyak['value']['persentase']

# 12
dbh_minyak_setengah_persen                 = datapemda['postur']['1']['child']['1']['child'][8]
dbh_minyak_setengah_persen_total_a         = dbh_minyak_setengah_persen['value']['anggaran']
dbh_minyak_setengah_persen_total_r         = dbh_minyak_setengah_persen['value']['realisasi']
dbh_minyak_setengah_persen_total_p         = dbh_minyak_setengah_persen['value']['persentase']

# 13
dbh_minyak_otsus                 = datapemda['postur']['1']['child']['1']['child'][9]
dbh_minyak_otsus_total_a         = dbh_minyak_otsus['value']['anggaran']
dbh_minyak_otsus_total_r         = dbh_minyak_otsus['value']['realisasi']
dbh_minyak_otsus_total_p         = dbh_minyak_otsus['value']['persentase']

# 14
dbh_gasbumi                 = datapemda['postur']['1']['child']['1']['child'][10]
dbh_gasbumi_total_a         = dbh_gasbumi['value']['anggaran']
dbh_gasbumi_total_r         = dbh_gasbumi['value']['realisasi']
dbh_gasbumi_total_p         = dbh_gasbumi['value']['persentase']

# 15
dbh_gasbumi_setengah_persen                 = datapemda['postur']['1']['child']['1']['child'][11]
dbh_gasbumi_setengah_persen_total_a         = dbh_gasbumi_setengah_persen['value']['anggaran']
dbh_gasbumi_setengah_persen_total_r         = dbh_gasbumi_setengah_persen['value']['realisasi']
dbh_gasbumi_setengah_persen_total_p         = dbh_gasbumi_setengah_persen['value']['persentase']

# 16
dbh_gasbumi_otsus                 = datapemda['postur']['1']['child']['1']['child'][12]
dbh_gasbumi_otsus_total_a         = dbh_gasbumi_otsus['value']['anggaran']
dbh_gasbumi_otsus_total_r         = dbh_gasbumi_otsus['value']['realisasi']
dbh_gasbumi_otsus_total_p         = dbh_gasbumi_otsus['value']['persentase']

# 17 
dbh_pertum_iurantetap                 = datapemda['postur']['1']['child']['1']['child'][13]
dbh_pertum_iurantetap_total_a         = dbh_pertum_iurantetap['value']['anggaran']
dbh_pertum_iurantetap_total_r         = dbh_pertum_iurantetap['value']['realisasi']
dbh_pertum_iurantetap_total_p         = dbh_pertum_iurantetap['value']['persentase']

# 18
dbh_pertum_royalti                 = datapemda['postur']['1']['child']['1']['child'][14]
dbh_pertum_royalti_total_a         = dbh_pertum_royalti['value']['anggaran']
dbh_pertum_royalti_total_r         = dbh_pertum_royalti['value']['realisasi']
dbh_pertum_royalti_total_p         = dbh_pertum_royalti['value']['persentase']

# 19
dbh_pabum_setoranbagpemerintah                 = datapemda['postur']['1']['child']['1']['child'][15]
dbh_pabum_setoranbagpemerintah_total_a         = dbh_pabum_setoranbagpemerintah['value']['anggaran']
dbh_pabum_setoranbagpemerintah_total_r         = dbh_pabum_setoranbagpemerintah['value']['realisasi']
dbh_pabum_setoranbagpemerintah_total_p         = dbh_pabum_setoranbagpemerintah['value']['persentase']

# 20
dbh_pabum_iurantetap                 = datapemda['postur']['1']['child']['1']['child'][16]
dbh_pabum_iurantetap_total_a         = dbh_pabum_iurantetap['value']['anggaran']
dbh_pabum_iurantetap_total_r         = dbh_pabum_iurantetap['value']['realisasi']
dbh_pabum_iurantetap_total_p         = dbh_pabum_iurantetap['value']['persentase']

# 21
dbh_pabum_royalti                 = datapemda['postur']['1']['child']['1']['child'][17]
dbh_pabum_royalti_total_a         = dbh_pabum_royalti['value']['anggaran']
dbh_pabum_royalti_total_r         = dbh_pabum_royalti['value']['realisasi']
dbh_pabum_royalti_total_p         = dbh_pabum_royalti['value']['persentase']

# 22
dbh_kehutanan_iiuph                 = datapemda['postur']['1']['child']['1']['child'][18]
dbh_kehutanan_iiuph_total_a         = dbh_kehutanan_iiuph['value']['anggaran']
dbh_kehutanan_iiuph_total_r         = dbh_kehutanan_iiuph['value']['realisasi']
dbh_kehutanan_iiuph_total_p         = dbh_kehutanan_iiuph['value']['persentase']

# 23
dbh_kehutanan_psdh                 = datapemda['postur']['1']['child']['1']['child'][19]
dbh_kehutanan_psdh_total_a         = dbh_kehutanan_psdh['value']['anggaran']
dbh_kehutanan_psdh_total_r         = dbh_kehutanan_psdh['value']['realisasi']
dbh_kehutanan_psdh_total_p         = dbh_kehutanan_psdh['value']['persentase']

# 24
dbh_kehutanan_danareboisasi                 = datapemda['postur']['1']['child']['1']['child'][20]
dbh_kehutanan_danareboisasi_total_a         = dbh_kehutanan_danareboisasi['value']['anggaran']
dbh_kehutanan_danareboisasi_total_r         = dbh_kehutanan_danareboisasi['value']['realisasi']
dbh_kehutanan_danareboisasi_total_p         = dbh_kehutanan_danareboisasi['value']['persentase']

# 25
dbh_perikanan                 = datapemda['postur']['1']['child']['1']['child'][21]
dbh_perikanan_total_a         = dbh_perikanan['value']['anggaran']
dbh_perikanan_total_r         = dbh_perikanan['value']['realisasi']
dbh_perikanan_total_p         = dbh_perikanan['value']['persentase']

# 26
dbh_cht                 = datapemda['postur']['1']['child']['1']['child'][22]
dbh_cht_total_a         = dbh_cht['value']['anggaran']
dbh_cht_total_r         = dbh_cht['value']['realisasi']
dbh_cht_total_p         = dbh_cht['value']['persentase']

# 27 Total DAU
dau_total_a = datapemda['postur']['1']['child']['2']['total']['anggaran']
dau_total_r = datapemda['postur']['1']['child']['2']['total']['realisasi']
dau_total_p = datapemda['postur']['1']['child']['2']['total']['persentase']

# 28 
dau                 = datapemda['postur']['1']['child']['2']['child'][0]
dau_total_a         = dau['value']['anggaran']
dau_total_r         = dau['value']['realisasi']
dau_total_p         = dau['value']['persentase']

# 29 Total DAK Fisik
dak_fisik_total_a = datapemda['postur']['1']['child']['3']['total']['anggaran']
dak_fisik_total_r = datapemda['postur']['1']['child']['3']['total']['realisasi']
dak_fisik_total_p = datapemda['postur']['1']['child']['3']['total']['persentase']

# 30
dak_fisik                 = datapemda['postur']['1']['child']['3']['child'][0]
dak_fisik_total_a         = dak_fisik['value']['anggaran']
dak_fisik_total_r         = dak_fisik['value']['realisasi']
dak_fisik_total_p         = dak_fisik['value']['persentase']

# 31
dak_fisik_infrastruktur                 = datapemda['postur']['1']['child']['3']['child'][1]
dak_fisik_infrastruktur_total_a         = dak_fisik_infrastruktur['value']['anggaran']
dak_fisik_infrastruktur_total_r         = dak_fisik_infrastruktur['value']['realisasi']
dak_fisik_infrastruktur_total_p         = dak_fisik_infrastruktur['value']['persentase']

# 32
dak_fisik_penugasan                 = datapemda['postur']['1']['child']['3']['child'][2]
dak_fisik_penugasan_total_a         = dak_fisik_penugasan['value']['anggaran']
dak_fisik_penugasan_total_r         = dak_fisik_penugasan['value']['realisasi']
dak_fisik_penugasan_total_p         = dak_fisik_penugasan['value']['persentase']

# 33
dak_fisik_afirmasi                 = datapemda['postur']['1']['child']['3']['child'][3]
dak_fisik_afirmasi_total_a         = dak_fisik_afirmasi['value']['anggaran']
dak_fisik_afirmasi_total_r         = dak_fisik_afirmasi['value']['realisasi']
dak_fisik_afirmasi_total_p         = dak_fisik_afirmasi['value']['persentase']

# 34
dak_nonfisik_total_a = datapemda['postur']['1']['child']['4']['total']['anggaran']
dak_nonfisik_total_r = datapemda['postur']['1']['child']['4']['total']['realisasi']
dak_nonfisik_total_p = datapemda['postur']['1']['child']['4']['total']['persentase']

# 35
dak_nonfisik_tpg                 = datapemda['postur']['1']['child']['4']['child'][0]
dak_nonfisik_tpg_total_a         = dak_nonfisik_tpg['value']['anggaran']
dak_nonfisik_tpg_total_r         = dak_nonfisik_tpg['value']['realisasi']
dak_nonfisik_tpg_total_p         = dak_nonfisik_tpg['value']['persentase']

# 36
dak_nonfisik_tkg_dasus                 = datapemda['postur']['1']['child']['4']['child'][1]
dak_nonfisik_tkg_dasus_total_a         = dak_nonfisik_tkg_dasus['value']['anggaran']
dak_nonfisik_tkg_dasus_total_r         = dak_nonfisik_tkg_dasus['value']['realisasi']
dak_nonfisik_tkg_dasus_total_p         = dak_nonfisik_tkg_dasus['value']['persentase']

# 37
dak_nonfisik_tamsil                 = datapemda['postur']['1']['child']['4']['child'][2]
dak_nonfisik_tamsil_total_a         = dak_nonfisik_tamsil['value']['anggaran']
dak_nonfisik_tamsil_total_r         = dak_nonfisik_tamsil['value']['realisasi']
dak_nonfisik_tamsil_total_p         = dak_nonfisik_tamsil['value']['persentase']

# 38
dak_nonfisik_bos                 = datapemda['postur']['1']['child']['4']['child'][3]
dak_nonfisik_bos_total_a         = dak_nonfisik_bos['value']['anggaran']
dak_nonfisik_bos_total_r         = dak_nonfisik_bos['value']['realisasi']
dak_nonfisik_bos_total_p         = dak_nonfisik_bos['value']['persentase']

# 39
dak_nonfisik_bos_kesehatan                 = datapemda['postur']['1']['child']['4']['child'][4]
dak_nonfisik_bos_kesehatan_total_a         = dak_nonfisik_bos_kesehatan['value']['anggaran']
dak_nonfisik_bos_kesehatan_total_r         = dak_nonfisik_bos_kesehatan['value']['realisasi']
dak_nonfisik_bos_kesehatan_total_p         = dak_nonfisik_bos_kesehatan['value']['persentase']

# 40
dak_nonfisik_bos_kb                 = datapemda['postur']['1']['child']['4']['child'][5]
dak_nonfisik_bos_kb_total_a         = dak_nonfisik_bos_kb['value']['anggaran']
dak_nonfisik_bos_kb_total_r         = dak_nonfisik_bos_kb['value']['realisasi']
dak_nonfisik_bos_kb_total_p         = dak_nonfisik_bos_kb['value']['persentase']

# 41
dak_nonfisik_koperasi                 = datapemda['postur']['1']['child']['4']['child'][6]
dak_nonfisik_koperasi_total_a         = dak_nonfisik_koperasi['value']['anggaran']
dak_nonfisik_koperasi_total_r         = dak_nonfisik_koperasi['value']['realisasi']
dak_nonfisik_koperasi_total_p         = dak_nonfisik_koperasi['value']['persentase']

# 42
dak_nonfisik_adminduk                 = datapemda['postur']['1']['child']['4']['child'][7]
dak_nonfisik_adminduk_total_a         = dak_nonfisik_adminduk['value']['anggaran']
dak_nonfisik_adminduk_total_r         = dak_nonfisik_adminduk['value']['realisasi']
dak_nonfisik_adminduk_total_p         = dak_nonfisik_adminduk['value']['persentase']

# 43
dak_nonfisik_bos_paud                 = datapemda['postur']['1']['child']['4']['child'][8]
dak_nonfisik_bos_paud_total_a         = dak_nonfisik_bos_paud['value']['anggaran']
dak_nonfisik_bos_paud_total_r         = dak_nonfisik_bos_paud['value']['realisasi']
dak_nonfisik_bos_paud_total_p         = dak_nonfisik_bos_paud['value']['persentase']

# 44 Total DID
did_total_a = datapemda['postur']['1']['child']['5']['total']['anggaran']
did_total_r = datapemda['postur']['1']['child']['5']['total']['realisasi']
did_total_p = datapemda['postur']['1']['child']['5']['total']['persentase']

# 45
did_value                 = datapemda['postur']['1']['child']['5']['child'][0]
did_value_total_a         = did_value['value']['anggaran']
did_value_total_r         = did_value['value']['realisasi']
did_value_total_p         = did_value['value']['persentase']

# 46 Total Otsus
otsus_total_a = datapemda['postur']['1']['child']['6']['total']['anggaran']
otsus_total_r = datapemda['postur']['1']['child']['6']['total']['realisasi']
otsus_total_p = datapemda['postur']['1']['child']['6']['total']['persentase']

# 47
otsus_aceh                 = datapemda['postur']['1']['child']['6']['child'][0]
otsus_aceh_total_a         = otsus_aceh['value']['anggaran']
otsus_aceh_total_r         = otsus_aceh['value']['realisasi']
otsus_aceh_total_p         = otsus_aceh['value']['persentase']

# 48
otsus_papua                 = datapemda['postur']['1']['child']['6']['child'][1]
otsus_papua_total_a         = otsus_papua['value']['anggaran']
otsus_papua_total_r         = otsus_papua['value']['realisasi']
otsus_papua_total_p         = otsus_papua['value']['persentase']

# 49
infrastruktur_papua                 = datapemda['postur']['1']['child']['6']['child'][2]
infrastruktur_papua_total_a         = infrastruktur_papua['value']['anggaran']
infrastruktur_papua_total_r         = infrastruktur_papua['value']['realisasi']
infrastruktur_papua_total_p         = infrastruktur_papua['value']['persentase']

# 50
otsus_papuabarat                 = datapemda['postur']['1']['child']['6']['child'][3]
otsus_papuabarat_total_a         = otsus_papuabarat['value']['anggaran']
otsus_papuabarat_total_r         = otsus_papuabarat['value']['realisasi']
otsus_papuabarat_total_p         = otsus_papuabarat['value']['persentase']

# 51
infrastruktur_papuabarat                 = datapemda['postur']['1']['child']['6']['child'][4]
infrastruktur_papuabarat_total_a         = infrastruktur_papuabarat['value']['anggaran']
infrastruktur_papuabarat_total_r         = infrastruktur_papuabarat['value']['realisasi']
infrastruktur_papuabarat_total_p         = infrastruktur_papuabarat['value']['persentase']

# 52 Total Dais
dais_total_a = datapemda['postur']['1']['child']['7']['total']['anggaran']
dais_total_r = datapemda['postur']['1']['child']['7']['total']['realisasi']
dais_total_p = datapemda['postur']['1']['child']['7']['total']['persentase']

# 53
dais_value                 = datapemda['postur']['1']['child']['7']['child'][0]
dais_value_total_a         = dais_value['value']['anggaran']
dais_value_total_r         = dais_value['value']['realisasi']
dais_value_total_p         = dais_value['value']['persentase']

# 54 Dana Desa
dd_total_a = datapemda['postur']['1']['total']['anggaran']
dd_total_r = datapemda['postur']['1']['total']['realisasi']
dd_total_p = datapemda['postur']['1']['total']['persentase']

# 55
dd_value                 = datapemda['postur']['2']['child']['1']['child'][0]
dd_value_total_a         = dd_value['value']['anggaran']
dd_value_total_r         = dd_value['value']['realisasi']
dd_value_total_p         = dd_value['value']['persentase']

# header
dbh_pph21_header                = dbh_pph21['kodeakun'] + "-" + dbh_pph21['text']
dbh_pph2529opdn_header          = dbh_pph2529opdn['kodeakun'] + "-" + dbh_pph2529opdn['text']
dbh_pbbprovinsi_header          = dbh_pbbprovinsi['kodeakun'] + "-" + dbh_pbbprovinsi['text']
dbh_pbbkabkot_header            = dbh_pbbkabkot['kodeakun'] + "-" + dbh_pbbkabkot['text']
dbh_pbbupahpungutprov_header    = dbh_pbbupahpungutprov['kodeakun'] + "-" + dbh_pbbupahpungutprov['text']
dbh_pbbupahpungutkabkot_header  = dbh_pbbupahpungutkabkot['kodeakun'] + "-" + dbh_pbbupahpungutkabkot['text']
dbh_pbbbagpusatrata_header      = dbh_pbbbagpusatrata['kodeakun'] + "-" + dbh_pbbbagpusatrata['text']
dbh_minyak_header               = dbh_minyak['kodeakun'] + "-" + dbh_minyak['text']
dbh_minyak_setengah_persen_header = dbh_minyak_setengah_persen['kodeakun'] + "-" + dbh_minyak_setengah_persen['text']
dbh_minyak_otsus_header = dbh_minyak_otsus['kodeakun'] + "-" + dbh_minyak_otsus['text']
dbh_gasbumi_header = dbh_gasbumi['kodeakun'] + "-" + dbh_gasbumi['text']
dbh_gasbumi_setengah_persen_header = dbh_gasbumi_setengah_persen['kodeakun'] + "-" + dbh_gasbumi_setengah_persen['text']
dbh_gasbumi_otsus_header = dbh_gasbumi_otsus['kodeakun'] + "-" + dbh_gasbumi_otsus['text']
dbh_pertum_iurantetap_header = dbh_pertum_iurantetap['kodeakun'] + "-" + dbh_pertum_iurantetap['text']
dbh_pertum_royalti_header = dbh_pertum_royalti['kodeakun'] + "-" + dbh_pertum_royalti['text']
dbh_pabum_setoranbagpemerintah_header = dbh_pabum_setoranbagpemerintah['kodeakun'] + "-" + dbh_pabum_setoranbagpemerintah['text']
dbh_pabum_iurantetap_header = dbh_pabum_iurantetap['kodeakun'] + "-" + dbh_pabum_iurantetap['text']
dbh_pabum_royalti_header = dbh_pabum_royalti['kodeakun'] + "-" + dbh_pabum_royalti['text']
dbh_kehutanan_iiuph_header = dbh_kehutanan_iiuph['kodeakun'] + "-" + dbh_kehutanan_iiuph['text']
dbh_kehutanan_psdh_header = dbh_kehutanan_psdh['kodeakun'] + "-" + dbh_kehutanan_psdh['text']
dbh_kehutanan_danareboisasi_header = dbh_kehutanan_danareboisasi['kodeakun'] + "-" + dbh_kehutanan_danareboisasi['text']
dbh_perikanan_header = dbh_perikanan['kodeakun'] + "-" + dbh_perikanan['text']
dbh_cht_header = dbh_cht['kodeakun'] + "-" + dbh_cht['text']
dau_header = dau['kodeakun'] + "-" + dau['text']
dak_fisik_header = dak_fisik['kodeakun'] + "-" + dak_fisik['text']
dak_fisik_infrastruktur_header = dak_fisik_infrastruktur['kodeakun'] + "-" + dak_fisik_infrastruktur['text']
dak_fisik_penugasan_header = dak_fisik_penugasan['kodeakun'] + "-" + dak_fisik_penugasan['text']
dak_fisik_afirmasi_header = dak_fisik_afirmasi['kodeakun'] + "-" + dak_fisik_afirmasi['text']
dak_nonfisik_tpg_header = dak_nonfisik_tpg['kodeakun'] + "-" + dak_nonfisik_tpg['text']
dak_nonfisik_tkg_dasus_header = dak_nonfisik_tkg_dasus['kodeakun'] + "-" + dak_nonfisik_tkg_dasus['text']
dak_nonfisik_tamsil_header = dak_nonfisik_tamsil['kodeakun'] + "-" + dak_nonfisik_tamsil['text']
dak_nonfisik_bos_header = dak_nonfisik_bos['kodeakun'] + "-" + dak_nonfisik_bos['text']
dak_nonfisik_bos_kesehatan_header = dak_nonfisik_bos_kesehatan['kodeakun'] + "-" + dak_nonfisik_bos_kesehatan['text']
dak_nonfisik_bos_kb_header = dak_nonfisik_bos_kb['kodeakun'] + "-" + dak_nonfisik_bos_kb['text']
dak_nonfisik_koperasi_header = dak_nonfisik_koperasi['kodeakun'] + "-" + dak_nonfisik_koperasi['text']
dak_nonfisik_adminduk_header = dak_nonfisik_adminduk['kodeakun'] + "-" + dak_nonfisik_adminduk['text']
dak_nonfisik_bos_paud_header = dak_nonfisik_bos_paud['kodeakun'] + "-" + dak_nonfisik_bos_paud['text']
did_value_header = did_value['kodeakun'] + "-" + did_value['text']
otsus_aceh_header = otsus_aceh['kodeakun'] + "-" + otsus_aceh['text']
otsus_papua_header = otsus_papua['kodeakun'] + "-" + otsus_papua['text']
infrastruktur_papua_header = infrastruktur_papua['kodeakun'] + "-" + infrastruktur_papua['text']
otsus_papuabarat_header = otsus_papuabarat['kodeakun'] + "-" + otsus_papuabarat['text']
infrastruktur_papuabarat_header = infrastruktur_papuabarat['kodeakun'] + "-" + infrastruktur_papuabarat['text']
dais_value_header = dais_value['kodeakun'] + "-" + dais_value['text']
dd_value_header = dd_value['kodeakun'] + "-" + dd_value['text']

headers = ["no", "tkdd_total", "tkd_total", "dbh_total", 
           # 4
           dbh_pph21_header + " Anggaran", dbh_pph21_header + " Realisasi", dbh_pph21_header + " Persentase",
           #5 
           dbh_pph2529opdn_header  + " Anggaran", dbh_pph2529opdn_header + " Realisasi", dbh_pph2529opdn_header + " Persentase",
           #6 
           dbh_pbbprovinsi_header
           #7
           dbh_pbbkabkot_header 
           #8
           dbh_pbbupahpungutprov_header
           #9
           dbh_pbbupahpungutkabkot_header
           # 10
           dbh_pbbbagpusatrata_header 
           # 11
           dbh_minyak_header 
           # 12
            dbh_minyak_setengah_persen_header  
            #13
           dbh_minyak_otsus_header 
           # 14
            dbh_gasbumi_header 
           # 15
            dbh_gasbumi_setengah_persen_header 
           # 16
            dbh_gasbumi_otsus_header 
           # 17
            dbh_pertum_iurantetap_header 
           # 18
            dbh_pertum_royalti_header 
           # 19
            dbh_pabum_setoranbagpemerintah_header 
           # 20
            dbh_pabum_iurantetap_header 
           # 21
            dbh_pabum_royalti_header 
           # 22
            dbh_kehutanan_iiuph_header 
           # 23
            dbh_kehutanan_psdh_header 
           # 24
            dbh_kehutanan_danareboisasi_header 
           # 25
            dbh_perikanan_header 
           # 26
            dbh_cht_header 
           #27
           "dau_total",
           # 28
            dau_header 
           # 29
           "dak_fisik_total",
           # 30
            dak_fisik_header 
           # 31
            dak_fisik_infrastruktur_header 
           #33
            dak_fisik_penugasan_header 
           #33
            dak_fisik_afirmasi_header 
           #34
           "dak_non_fisik_total",
           #35
            dak_nonfisik_tpg_header 
           #36
            dak_nonfisik_tkg_dasus_header 
           #37
            dak_nonfisik_tamsil_header 
           #38
            dak_nonfisik_bos_header 
           #39
            dak_nonfisik_bos_kesehatan_header 
           #40
            dak_nonfisik_bos_kb_header 
           #41
            dak_nonfisik_koperasi_header 
           #42
            dak_nonfisik_adminduk_header 
           #43
            dak_nonfisik_bos_paud_header 
           #44
           "did_total",
           # 45 
            did_value_header 
           # 46
           "otsus_total",
           # 47
            otsus_aceh_header 
           # 48
            otsus_papua_header 
           # 49
            infrastruktur_papua_header 
           #50 
            otsus_papuabarat_header 
           #51
            infrastruktur_papuabarat_header 
           #52
           "dais_total",
           # 53
            dais_value_header 
           # 54
           "dd_total",
           # 55
            dd_value_header
]





print(no,
      #tahun, wilayah, provinsi, disclaimer, 
      tkdd_total_a, 
      tkdd_total_r, 
      tkdd_total_p, 
      tkd_total_a, 
      tkd_total_r, 
      tkd_total_p, 
      dbh_total_a, 
      dbh_total_r, 
      dbh_total_p, 
      dbh_pph21_total_kodeakun, 
      dbh_pph21_total_a, 
      dbh_pph21_total_r, 
      dbh_pph21_total_p,
      dbh_pph2529opdn, 
      dbh_pph2529opdn_total_kodeakun, 
      dbh_pph2529opdn_total_a, 
      dbh_pph2529opdn_total_r, 
      dbh_pph2529opdn_total_p, 
dbh_pbbprovinsi, 
dbh_pbbprovinsi_total_kodeakun, 
dbh_pbbprovinsi_total_a, 
dbh_pbbprovinsi_total_r,
dbh_pbbprovinsi_total_p, 
dbh_pbbkabkot, 
dbh_pbbkabkot_total_a, 
dbh_pbbkabkot_total_r, 
dbh_pbbkabkot_total_p, 
dbh_pbbupahpungutprov, 
dbh_pbbupahpungutprov_total_kodeakun, dbh_pbbupahpungutprov_total_a, dbh_pbbupahpungutprov_total_r, dbh_pbbupahpungutprov_total_p, 
dbh_pbbupahpungutkabkot, dbh_pbbupahpungutkabkot_total_kodeakun, dbh_pbbupahpungutkabkot_total_a, dbh_pbbupahpungutkabkot_total_r, dbh_pbbupahpungutkabkot_total_p, 
dbh_pbbbagpusatrata, dbh_pbbbagpusatrata_total_kodeakun, dbh_pbbbagpusatrata_total_a, dbh_pbbbagpusatrata_total_r, dbh_pbbbagpusatrata_total_p, 
dbh_minyak, dbh_minyak_total_kodeakun, dbh_minyak_total_a, dbh_minyak_total_r, dbh_minyak_total_p, 
dbh_minyak_setengah_persen, dbh_minyak_setengah_persen_total_kodeakun, dbh_minyak_setengah_persen_total_a, dbh_minyak_setengah_persen_total_r, dbh_minyak_setengah_persen_total_p, 
dbh_minyak_otsus, dbh_minyak_otsus_total_kodeakun, dbh_minyak_otsus_total_a, dbh_minyak_otsus_total_r, dbh_minyak_otsus_total_p, 
dbh_gasbumi, dbh_gasbumi_total_kodeakun, dbh_gasbumi_total_a, dbh_gasbumi_total_r, dbh_gasbumi_total_p, 
dbh_gasbumi_setengah_persen, dbh_gasbumi_setengah_persen_total_kodeakun, dbh_gasbumi_setengah_persen_total_a, dbh_gasbumi_setengah_persen_total_r, dbh_gasbumi_setengah_persen_total_p, 
dbh_gasbumi_otsus, dbh_gasbumi_otsus_total_kodeakun, dbh_gasbumi_otsus_total_a, dbh_gasbumi_otsus_total_r, dbh_gasbumi_otsus_total_p, 
dbh_pertum_iurantetap, dbh_pertum_iurantetap_total_kodeakun, dbh_pertum_iurantetap_total_a, dbh_pertum_iurantetap_total_r, dbh_pertum_iurantetap_total_p, 
dbh_pertum_royalti, dbh_pertum_royalti_total_kodeakun, dbh_pertum_royalti_total_a, dbh_pertum_royalti_total_r, dbh_pertum_royalti_total_p, 
dbh_pabum_setoranbagpemerintah, dbh_pabum_setoranbagpemerintah_total_kodeakun, dbh_pabum_setoranbagpemerintah_total_a, dbh_pabum_setoranbagpemerintah_total_r, dbh_pabum_setoranbagpemerintah_total_p,
 dbh_pabum_iurantetap, dbh_pabum_iurantetap_total_kodeakun, dbh_pabum_iurantetap_total_a, dbh_pabum_iurantetap_total_r, dbh_pabum_iurantetap_total_p,
 dbh_pabum_royalti, dbh_pabum_royalti_total_kodeakun, dbh_pabum_royalti_total_a, dbh_pabum_royalti_total_r, dbh_pabum_royalti_total_p,
 dbh_kehutanan_iiuph, dbh_kehutanan_iiuph_total_kodeakun, dbh_kehutanan_iiuph_total_a, dbh_kehutanan_iiuph_total_r, dbh_kehutanan_iiuph_total_p,
 dbh_kehutanan_psdh, dbh_kehutanan_psdh_total_kodeakun, dbh_kehutanan_psdh_total_a, dbh_kehutanan_psdh_total_r, dbh_kehutanan_psdh_total_p,
 dbh_kehutanan_danareboisasi, dbh_kehutanan_danareboisasi_total_kodeakun, dbh_kehutanan_danareboisasi_total_a, dbh_kehutanan_danareboisasi_total_r, dbh_kehutanan_danareboisasi_total_p,
 dbh_perikanan, dbh_perikanan_total_kodeakun, dbh_perikanan_total_a, dbh_perikanan_total_r, dbh_perikanan_total_p,
 dbh_cht, dbh_cht_total_kodeakun, dbh_cht_total_a, dbh_cht_total_r, dbh_cht_total_p,
 dau_total_a, dau_total_r, dau_total_p,
 dau, dau_total_kodeakun, dau_total_a, dau_total_r, dau_total_p,
 dak_fisik_total_a, dak_fisik_total_r, dak_fisik_total_p,
 dak_fisik, dak_fisik_total_kodeakun, dak_fisik_total_a, dak_fisik_total_r, dak_fisik_total_p,
 dak_fisik_infrastruktur, dak_fisik_infrastruktur_total_kodeakun, dak_fisik_infrastruktur_total_a, dak_fisik_infrastruktur_total_r, dak_fisik_infrastruktur_total_p,
 dak_fisik_penugasan, dak_fisik_penugasan_total_kodeakun, dak_fisik_penugasan_total_a, dak_fisik_penugasan_total_r, dak_fisik_penugasan_total_p,
 dak_fisik_afirmasi, dak_fisik_afirmasi_total_kodeakun, dak_fisik_afirmasi_total_a, dak_fisik_afirmasi_total_r, dak_fisik_afirmasi_total_p,
 dak_nonfisik_total_a, dak_nonfisik_total_r, dak_nonfisik_total_p,
 dak_nonfisik_tpg, dak_nonfisik_tpg_total_kodeakun, dak_nonfisik_tpg_total_a, dak_nonfisik_tpg_total_r, dak_nonfisik_tpg_total_p,
 dak_nonfisik_tkg_dasus, dak_nonfisik_tkg_dasus_total_kodeakun, dak_nonfisik_tkg_dasus_total_a, dak_nonfisik_tkg_dasus_total_r, dak_nonfisik_tkg_dasus_total_p,
 dak_nonfisik_tamsil, dak_nonfisik_tamsil_total_kodeakun, dak_nonfisik_tamsil_total_a, dak_nonfisik_tamsil_total_r, dak_nonfisik_tamsil_total_p,
 dak_nonfisik_bos, dak_nonfisik_bos_total_kodeakun, dak_nonfisik_bos_total_a, dak_nonfisik_bos_total_r, dak_nonfisik_bos_total_p,
 dak_nonfisik_bos_kesehatan, dak_nonfisik_bos_kesehatan_total_kodeakun, dak_nonfisik_bos_kesehatan_total_a, dak_nonfisik_bos_kesehatan_total_r, dak_nonfisik_bos_kesehatan_total_p,
 dak_nonfisik_bos_kb, dak_nonfisik_bos_kb_total_kodeakun, dak_nonfisik_bos_kb_total_a, dak_nonfisik_bos_kb_total_r, dak_nonfisik_bos_kb_total_p,
 dak_nonfisik_koperasi, dak_nonfisik_koperasi_total_kodeakun, dak_nonfisik_koperasi_total_a, dak_nonfisik_koperasi_total_r, dak_nonfisik_koperasi_total_p,
 dak_nonfisik_adminduk, dak_nonfisik_adminduk_total_kodeakun, dak_nonfisik_adminduk_total_a, dak_nonfisik_adminduk_total_r, dak_nonfisik_adminduk_total_p,
 dak_nonfisik_bos_paud, dak_nonfisik_bos_paud_total_kodeakun, dak_nonfisik_bos_paud_total_a, dak_nonfisik_bos_paud_total_r, dak_nonfisik_bos_paud_total_p,
 did_total_a, did_total_r, did_total_p,
 did_value, did_value_total_kodeakun, did_value_total_a, did_value_total_r, did_value_total_p,
 otsus_total_a, otsus_total_r, otsus_total_p,
 otsus_aceh, otsus_aceh_total_kodeakun, otsus_aceh_total_a, otsus_aceh_total_r, otsus_aceh_total_p,
 otsus_papua, otsus_papua_total_kodeakun, otsus_papua_total_a, otsus_papua_total_r, otsus_papua_total_p,
 infrastruktur_papua, infrastruktur_papua_total_kodeakun, infrastruktur_papua_total_a, infrastruktur_papua_total_r, infrastruktur_papua_total_p,
 otsus_papuabarat, otsus_papuabarat_total_kodeakun, otsus_papuabarat_total_a, otsus_papuabarat_total_r, otsus_papuabarat_total_p,
 infrastruktur_papuabarat, infrastruktur_papuabarat_total_kodeakun, infrastruktur_papuabarat_total_a, infrastruktur_papuabarat_total_r, infrastruktur_papuabarat_total_p,
 dais_total_a, dais_total_r, dais_total_p,
 dais_value, dais_value_total_kodeakun, dais_value_total_a, dais_value_total_r, dais_value_total_p,
 #DanaDesa, dd_total_a, dd_total_r, dd_total_p,
 dd_value, dd_value_total_kodeakun, dd_value_total_a, dd_value_total_r, dd_value_total_p
      )

763712259647840 757533338915843 99,19 % 703712259647840 697657198541618 99,14 % 89225342014000 93705559169270 105,02 % 611111 32986586000000 27458081283272 83,24 % {'kodeakun': '611112', 'text': 'Transfer DBH PPh Pasal 25/29 OP', 'value': {'anggaran': 4441882000000, 'realisasi': 2533606329484, 'persentase': '57,04 %'}} 611112 4441882000000 2533606329484 57,04 % {'kodeakun': '611211', 'text': 'Transfer DBH PBB untuk Propinsi', 'value': {'anggaran': 2857407829000, 'realisasi': 2737579489820, 'persentase': '95,81 %'}} 611211 2857407829000 2737579489820 95,81 % {'kodeakun': '611212', 'text': 'Transfer DBH PBB untuk Kabupaten/Kota', 'value': {'anggaran': 11211563981000, 'realisasi': 10840669654158, 'persentase': '96,69 %'}} 611212 11211563981000 10840669654158 96,69 % {'kodeakun': '611213', 'text': 'Transfer DBH Biaya/Upah Pungut PBB untuk Propinsi', 'value': {'anggaran': 86252212000, 'realisasi': 84199185177, 'persentase': '97,62 %'}} 611213 86252212000 84199185177 97,62 % {'kodeakun': '61

# Metani Tanpa Persentase

In [ ]:
import json
datapemda = json.loads('{"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Kabupaten\/kota","value":{"anggaran":398764072000,"realisasi":390261862177,"persentase":"97,87 %"}},{"kodeakun":"611215","text":"Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten\/Kota","value":{"anggaran":1736910100000,"realisasi":1650781100705,"persentase":"95,04 %"}},{"kodeakun":"612111","text":"Transfer DBH Minyak Bumi","value":{"anggaran":8581982674000,"realisasi":12376493484658,"persentase":"144,21 %"}},{"kodeakun":"612112","text":"Transfer DBH Minyak Bumi 0.5%","value":{"anggaran":282732216000,"realisasi":407714843494,"persentase":"144,21 %"}},{"kodeakun":"612113","text":"Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus","value":{"anggaran":370605160000,"realisasi":481008154249,"persentase":"129,79 %"}},{"kodeakun":"612211","text":"Transfer DBH Gas Bumi","value":{"anggaran":5966265026000,"realisasi":8745074752035,"persentase":"146,58 %"}},{"kodeakun":"612212","text":"Transfer DBH Gas Bumi 0.5%","value":{"anggaran":96882460000,"realisasi":124683590580,"persentase":"128,70 %"}},{"kodeakun":"612213","text":"Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus","value":{"anggaran":270583164000,"realisasi":340653879134,"persentase":"125,90 %"}},{"kodeakun":"612311","text":"Transfer DBH Pertambangan Umum - Iuran Tetap","value":{"anggaran":615858746000,"realisasi":574574040835,"persentase":"93,30 %"}},{"kodeakun":"612312","text":"Transfer DBH Pertambangan Umum - Royalti","value":{"anggaran":13670958915000,"realisasi":19233330257405,"persentase":"140,69 %"}},{"kodeakun":"612411","text":"Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah","value":{"anggaran":545092142000,"realisasi":814945712335,"persentase":"149,51 %"}},{"kodeakun":"612412","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Tetap","value":{"anggaran":14680315000,"realisasi":17093550748,"persentase":"116,44 %"}},{"kodeakun":"612413","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)","value":{"anggaran":703068000,"realisasi":897327311,"persentase":"127,63 %"}},{"kodeakun":"612511","text":"Transfer DBH Kehutanan - IIUPH\/IHPH","value":{"anggaran":164730247000,"realisasi":165979915021,"persentase":"100,76 %"}},{"kodeakun":"612512","text":"Transfer DBH Kehutanan - PSDH","value":{"anggaran":736821430000,"realisasi":706696713825,"persentase":"95,91 %"}},{"kodeakun":"612513","text":"Transfer DBH Kehutanan - Dana Reboisasi","value":{"anggaran":743479609000,"realisasi":741420178157,"persentase":"99,72 %"}},{"kodeakun":"612611","text":"Transfer DBH Perikanan","value":{"anggaran":480000648000,"realisasi":357622699445,"persentase":"74,50 %"}},{"kodeakun":"613111","text":"Transfer DBH Cukai Hasil Tembakau","value":{"anggaran":2964600000000,"realisasi":2922191165245,"persentase":"98,57 %"}}],"total":{"anggaran":89225342014000,"realisasi":93705559169270,"persentase":"105,02 %"}},"2":{"text":"DAU","child":[{"kodeakun":"621111","text":"Dana Alokasi Umum","value":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}}],"total":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}},"3":{"text":"DAK FISIK","child":[{"kodeakun":"631111","text":"Dana Alokasi Khusus","value":{"anggaran":31350835954000,"realisasi":29459738989382,"persentase":"93,97 %"}},{"kodeakun":"631211","text":"Dana Alokasi Khusus Infrastruktur Publik Daerah","value":{"anggaran":0,"realisasi":null,"persentase":"-"}},{"kodeakun":"631212","text":"Dana Alokasi Khusus Penugasan","value":{"anggaran":24463658880000,"realisasi":22492927163424,"persentase":"91,94 %"}},{"kodeakun":"631311","text":"Dana Alokasi Khusus Affirmasi","value":{"anggaran":6621768074000,"realisasi":6259405435103,"persentase":"94,53 %"}}],"total":{"anggaran":62436262908000,"realisasi":58212071587909,"persentase":"93,23 %"}},"6":{"text":"OTSUS","child":[{"kodeakun":"641111","text":"Dana Otonomi Khusus Provinsi Aceh","value":{"anggaran":8029791593000,"realisasi":8029791593000,"persentase":"100,00 %"}},{"kodeakun":"641211","text":"Dana Otonomi Khusus Provinsi Papua","value":{"anggaran":5620854115000,"realisasi":5620854115000,"persentase":"100,00 %"}},{"kodeakun":"641212","text":"Dana Tambahan Infrastruktur Provinsi Papua","value":{"anggaran":2400000000000,"realisasi":2400000000000,"persentase":"100,00 %"}},{"kodeakun":"641311","text":"Dana Otonomi Khusus Provinsi Papua Barat","value":{"anggaran":2408937478000,"realisasi":2408937478000,"persentase":"100,00 %"}},{"kodeakun":"641312","text":"Dana Tambahan Infrastruktur Provinsi Papua Barat","value":{"anggaran":1600000000000,"realisasi":1600000000000,"persentase":"100,00 %"}}],"total":{"anggaran":20059583186000,"realisasi":20059583186000,"persentase":"100,00 %"}},"7":{"text":"KEISTIMEWAAN DIY","child":[{"kodeakun":"642111","text":"Dana Keistimewaan Daerah Istimewa Yogyakarta","value":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}}],"total":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}},"5":{"text":"DID","child":[{"kodeakun":"643111","text":"Dana Insentif Daerah","value":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}}],"total":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}},"4":{"text":"DAK NON FISIK","child":[{"kodeakun":"654111","text":"Dana Tunjangan Profesi Guru PNSD","value":{"anggaran":56861747531798,"realisasi":55016720212332,"persentase":"96,76 %"}},{"kodeakun":"654112","text":"Dana Tunjangan Khusus Guru PNSD di Daerah Khusus","value":{"anggaran":1922303294676,"realisasi":1727102404861,"persentase":"89,85 %"}},{"kodeakun":"654211","text":"Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil","value":{"anggaran":879468864366,"realisasi":588007130574,"persentase":"66,86 %"}},{"kodeakun":"654311","text":"Dana Bantuan Operasional Sekolah","value":{"anggaran":46311243600000,"realisasi":44367351931006,"persentase":"95,80 %"}},{"kodeakun":"654711","text":"Dana Bantuan Operasional Kesehatan","value":{"anggaran":8551232700000,"realisasi":7454358890915,"persentase":"87,17 %"}},{"kodeakun":"654712","text":"Dana Bantuan Operasional Keluarga Berencana","value":{"anggaran":1808787300000,"realisasi":1753419417979,"persentase":"96,94 %"}},{"kodeakun":"654811","text":"Dana Peningkatan Kapasitas Koperasi","value":{"anggaran":100000000000,"realisasi":92221616897,"persentase":"92,22 %"}},{"kodeakun":"654814","text":"Dana Pelayanan Administrasi Kependudukan","value":{"anggaran":825000000000,"realisasi":745193742984,"persentase":"90,33 %"}},{"kodeakun":"654911","text":"Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini","value":{"anggaran":3741708600000,"realisasi":3555904601891,"persentase":"95,03 %"}}],"total":{"anggaran":121001491890840,"realisasi":115300279949439,"persentase":"95,29 %"}}},"total":{"anggaran":703712259647840,"realisasi":697657198541618,"persentase":"99,14 %"}},"2":{"text":"Dana Desa","child":{"1":{"text":"DANA DESA","child":[{"kodeakun":"662111","text":"Dana Desa","value":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}],"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"wilayah":"Nasional","judul":"POSTUR TKDD TA 2018","link_excel":"http:\/\/www.djpk.kemenkeu.go.id\/portal\/data\/csv\/2018\/--\/--\/tkdd","disclaimer":"Data bulan desember","header":["Alokasi","Realisasi"],"special_row":{"text":"Transfer ke daerah dan Dana Desa","total":{"anggaran":763712259647840,"realisasi":757533338915843,"persentase":"99,19 %"}}}')
no = 1

# 1 Total TKDD
tkdd_total_a = datapemda['special_row']['total']['anggaran']
tkdd_total_r = datapemda['special_row']['total']['realisasi']
tkdd_total_a_header = "Total TKDD Anggaran"
tkdd_total_r_header = "Total TKDD Realisasi"

# 2 Total TKD
tkd_total_a = datapemda['postur']['1']['total']['anggaran'] if len(datapemda['postur']['1']['total']['anggaran']) != 0
tkd_total_r = datapemda['postur']['1']['total']['realisasi']
tkd_total_a_header = "Total TKD Anggaran"
tkd_total_r_header = "Total TKD Realisasi"

# 3 Total DBH
dbh_total_a = datapemda['postur']['1']['child']['1']['total']['anggaran']
dbh_total_r = datapemda['postur']['1']['child']['1']['total']['realisasi']
dbh_total_a_header = "Total DBH Anggaran"
dbh_total_r_header = "Total DBH Realisasi"

# 4
dbh_pph21                 = datapemda['postur']['1']['child']['1']['child'][0]
dbh_pph21_total_a         = dbh_pph21['value']['anggaran']
dbh_pph21_total_r         = dbh_pph21['value']['realisasi']
dbh_pph21_header_a          = dbh_pph21['kodeakun'] + "-" + dbh_pph21['text'] + " Anggaran"
dbh_pph21_header_r          = dbh_pph21['kodeakun'] + "-" + dbh_pph21['text'] + " Realisasi"

# 5
dbh_pph2529opdn                 = datapemda['postur']['1']['child']['1']['child'][1]
dbh_pph2529opdn_total_a         = dbh_pph2529opdn['value']['anggaran']
dbh_pph2529opdn_total_r         = dbh_pph2529opdn['value']['realisasi']
dbh_pph2529opdn_header_a          = dbh_pph2529opdn['kodeakun'] + "-" + dbh_pph2529opdn['text']  + " Anggaran"
dbh_pph2529opdn_header_r          = dbh_pph2529opdn['kodeakun'] + "-" + dbh_pph2529opdn['text']  + " Realisasi"

# 6
dbh_pbbprovinsi                 = datapemda['postur']['1']['child']['1']['child'][2]
dbh_pbbprovinsi_total_a         = dbh_pbbprovinsi['value']['anggaran']
dbh_pbbprovinsi_total_r         = dbh_pbbprovinsi['value']['realisasi']
dbh_pbbprovinsi_total_p         = dbh_pbbprovinsi['value']['persentase']
dbh_pbbprovinsi_header_a          = dbh_pbbprovinsi['kodeakun'] + "-" + dbh_pbbprovinsi['text'] + " Anggaran"
dbh_pbbprovinsi_header_r          = dbh_pbbprovinsi['kodeakun'] + "-" + dbh_pbbprovinsi['text'] + " Realisasi"

# 7
dbh_pbbkabkot                 = datapemda['postur']['1']['child']['1']['child'][3]
dbh_pbbkabkot_total_a         = dbh_pbbkabkot['value']['anggaran']
dbh_pbbkabkot_total_r         = dbh_pbbkabkot['value']['realisasi']
dbh_pbbkabkot_total_p         = dbh_pbbkabkot['value']['persentase']
dbh_pbbkabkot_header_a        = dbh_pbbkabkot['kodeakun'] + "-" + dbh_pbbkabkot['text'] + " Anggaran"
dbh_pbbkabkot_header_r        = dbh_pbbkabkot['kodeakun'] + "-" + dbh_pbbkabkot['text'] + " Realisasi"

# 8
dbh_pbbupahpungutprov                 = datapemda['postur']['1']['child']['1']['child'][4]
dbh_pbbupahpungutprov_total_a         = dbh_pbbupahpungutprov['value']['anggaran']
dbh_pbbupahpungutprov_total_r         = dbh_pbbupahpungutprov['value']['realisasi']
dbh_pbbupahpungutprov_total_p         = dbh_pbbupahpungutprov['value']['persentase']
dbh_pbbupahpungutprov_header_a        = dbh_pbbupahpungutprov['kodeakun'] + "-" + dbh_pbbupahpungutprov['text'] + " Anggaran"
dbh_pbbupahpungutprov_header_r        = dbh_pbbupahpungutprov['kodeakun'] + "-" + dbh_pbbupahpungutprov['text'] + " Realisasi"


# 9
dbh_pbbupahpungutkabkot                 = datapemda['postur']['1']['child']['1']['child'][5]
dbh_pbbupahpungutkabkot_total_a         = dbh_pbbupahpungutkabkot['value']['anggaran']
dbh_pbbupahpungutkabkot_total_r         = dbh_pbbupahpungutkabkot['value']['realisasi']
dbh_pbbupahpungutkabkot_total_p         = dbh_pbbupahpungutkabkot['value']['persentase']
dbh_pbbupahpungutkabkot_header_a  = dbh_pbbupahpungutkabkot['kodeakun'] + "-" + dbh_pbbupahpungutkabkot['text'] + " Anggaran"
dbh_pbbupahpungutkabkot_header_r  = dbh_pbbupahpungutkabkot['kodeakun'] + "-" + dbh_pbbupahpungutkabkot['text'] + " Realisasi"

# 10
dbh_pbbbagpusatrata                 = datapemda['postur']['1']['child']['1']['child'][6]
dbh_pbbbagpusatrata_total_a         = dbh_pbbbagpusatrata['value']['anggaran']
dbh_pbbbagpusatrata_total_r         = dbh_pbbbagpusatrata['value']['realisasi']
dbh_pbbbagpusatrata_total_p         = dbh_pbbbagpusatrata['value']['persentase']
dbh_pbbbagpusatrata_header_a        = dbh_pbbbagpusatrata['kodeakun'] + "-" + dbh_pbbbagpusatrata['text'] + " Anggaran"
dbh_pbbbagpusatrata_header_r        = dbh_pbbbagpusatrata['kodeakun'] + "-" + dbh_pbbbagpusatrata['text'] + " Realisasi"

# 11
dbh_minyak                 = datapemda['postur']['1']['child']['1']['child'][7]
dbh_minyak_total_a         = dbh_minyak['value']['anggaran']
dbh_minyak_total_r         = dbh_minyak['value']['realisasi']
dbh_minyak_total_p         = dbh_minyak['value']['persentase']
dbh_minyak_header_a        = dbh_minyak['kodeakun'] + "-" + dbh_minyak['text'] + " Anggaran"
dbh_minyak_header_r        = dbh_minyak['kodeakun'] + "-" + dbh_minyak['text'] + " Realisasi"

# 12
dbh_minyak_setengah_persen                 = datapemda['postur']['1']['child']['1']['child'][8]
dbh_minyak_setengah_persen_total_a         = dbh_minyak_setengah_persen['value']['anggaran']
dbh_minyak_setengah_persen_total_r         = dbh_minyak_setengah_persen['value']['realisasi']
dbh_minyak_setengah_persen_total_p         = dbh_minyak_setengah_persen['value']['persentase']
dbh_minyak_setengah_persen_header_a        = dbh_minyak_setengah_persen['kodeakun'] + "-" + dbh_minyak_setengah_persen['text'] + " Anggaran"
dbh_minyak_setengah_persen_header_r        = dbh_minyak_setengah_persen['kodeakun'] + "-" + dbh_minyak_setengah_persen['text'] + " Realisasi"

# 13
dbh_minyak_otsus                 = datapemda['postur']['1']['child']['1']['child'][9]
dbh_minyak_otsus_total_a         = dbh_minyak_otsus['value']['anggaran']
dbh_minyak_otsus_total_r         = dbh_minyak_otsus['value']['realisasi']
dbh_minyak_otsus_total_p         = dbh_minyak_otsus['value']['persentase']
dbh_minyak_otsus_header_a        = dbh_minyak_otsus['kodeakun'] + "-" + dbh_minyak_otsus['text'] + " Anggaran"
dbh_minyak_otsus_header_r        = dbh_minyak_otsus['kodeakun'] + "-" + dbh_minyak_otsus['text'] + " Realisasi"

# 14
dbh_gasbumi                 = datapemda['postur']['1']['child']['1']['child'][10]
dbh_gasbumi_total_a         = dbh_gasbumi['value']['anggaran']
dbh_gasbumi_total_r         = dbh_gasbumi['value']['realisasi']
dbh_gasbumi_total_p         = dbh_gasbumi['value']['persentase']
dbh_gasbumi_header_a         = dbh_gasbumi['kodeakun'] + "-" + dbh_gasbumi['text'] + " Anggaran"
dbh_gasbumi_header_r         = dbh_gasbumi['kodeakun'] + "-" + dbh_gasbumi['text'] + " Realisasi"

# 15
dbh_gasbumi_setengah_persen                 = datapemda['postur']['1']['child']['1']['child'][11]
dbh_gasbumi_setengah_persen_total_a         = dbh_gasbumi_setengah_persen['value']['anggaran']
dbh_gasbumi_setengah_persen_total_r         = dbh_gasbumi_setengah_persen['value']['realisasi']
dbh_gasbumi_setengah_persen_total_p         = dbh_gasbumi_setengah_persen['value']['persentase']
dbh_gasbumi_setengah_persen_header_a        = dbh_gasbumi_setengah_persen['kodeakun'] + "-" + dbh_gasbumi_setengah_persen['text'] + " Anggaran"
dbh_gasbumi_setengah_persen_header_r        = dbh_gasbumi_setengah_persen['kodeakun'] + "-" + dbh_gasbumi_setengah_persen['text'] + " Realisasi"

# 16
dbh_gasbumi_otsus                 = datapemda['postur']['1']['child']['1']['child'][12]
dbh_gasbumi_otsus_total_a         = dbh_gasbumi_otsus['value']['anggaran']
dbh_gasbumi_otsus_total_r         = dbh_gasbumi_otsus['value']['realisasi']
dbh_gasbumi_otsus_total_p         = dbh_gasbumi_otsus['value']['persentase']
dbh_gasbumi_otsus_header_a        = dbh_gasbumi_otsus['kodeakun'] + "-" + dbh_gasbumi_otsus['text'] + " Anggaran"
dbh_gasbumi_otsus_header_r        = dbh_gasbumi_otsus['kodeakun'] + "-" + dbh_gasbumi_otsus['text'] + " Realisasi"

# 17 
dbh_pertum_iurantetap                 = datapemda['postur']['1']['child']['1']['child'][13]
dbh_pertum_iurantetap_total_a         = dbh_pertum_iurantetap['value']['anggaran']
dbh_pertum_iurantetap_total_r         = dbh_pertum_iurantetap['value']['realisasi']
dbh_pertum_iurantetap_total_p         = dbh_pertum_iurantetap['value']['persentase']
dbh_pertum_iurantetap_header_a        = dbh_pertum_iurantetap['kodeakun'] + "-" + dbh_pertum_iurantetap['text'] + "Anggaran"
dbh_pertum_iurantetap_header_r        = dbh_pertum_iurantetap['kodeakun'] + "-" + dbh_pertum_iurantetap['text'] + "Realisasi"

# 18
dbh_pertum_royalti                 = datapemda['postur']['1']['child']['1']['child'][14]
dbh_pertum_royalti_total_a         = dbh_pertum_royalti['value']['anggaran']
dbh_pertum_royalti_total_r         = dbh_pertum_royalti['value']['realisasi']
dbh_pertum_royalti_total_p         = dbh_pertum_royalti['value']['persentase']
dbh_pertum_royalti_header_a        = dbh_pertum_royalti['kodeakun'] + "-" + dbh_pertum_royalti['text'] + " Anggaran"
dbh_pertum_royalti_header_r        = dbh_pertum_royalti['kodeakun'] + "-" + dbh_pertum_royalti['text'] + " Realisasi"

# 19
dbh_pabum_setoranbagpemerintah                 = datapemda['postur']['1']['child']['1']['child'][15]
dbh_pabum_setoranbagpemerintah_total_a         = dbh_pabum_setoranbagpemerintah['value']['anggaran']
dbh_pabum_setoranbagpemerintah_total_r         = dbh_pabum_setoranbagpemerintah['value']['realisasi']
dbh_pabum_setoranbagpemerintah_total_p         = dbh_pabum_setoranbagpemerintah['value']['persentase']
dbh_pabum_setoranbagpemerintah_header_a        = dbh_pabum_setoranbagpemerintah['kodeakun'] + "-" + dbh_pabum_setoranbagpemerintah['text'] + " Anggaran"
dbh_pabum_setoranbagpemerintah_header_r        = dbh_pabum_setoranbagpemerintah['kodeakun'] + "-" + dbh_pabum_setoranbagpemerintah['text'] + " Realisasi"

# 20
dbh_pabum_iurantetap                 = datapemda['postur']['1']['child']['1']['child'][16]
dbh_pabum_iurantetap_total_a         = dbh_pabum_iurantetap['value']['anggaran']
dbh_pabum_iurantetap_total_r         = dbh_pabum_iurantetap['value']['realisasi']
dbh_pabum_iurantetap_total_p         = dbh_pabum_iurantetap['value']['persentase']
dbh_pabum_iurantetap_header_a        = dbh_pabum_iurantetap['kodeakun'] + "-" + dbh_pabum_iurantetap['text'] + " Anggaran"
dbh_pabum_iurantetap_header_r        = dbh_pabum_iurantetap['kodeakun'] + "-" + dbh_pabum_iurantetap['text'] + " Realisasi"

# 21
dbh_pabum_royalti                 = datapemda['postur']['1']['child']['1']['child'][17]
dbh_pabum_royalti_total_a         = dbh_pabum_royalti['value']['anggaran']
dbh_pabum_royalti_total_r         = dbh_pabum_royalti['value']['realisasi']
dbh_pabum_royalti_total_p         = dbh_pabum_royalti['value']['persentase']
dbh_pabum_royalti_header_a        = dbh_pabum_royalti['kodeakun'] + "-" + dbh_pabum_royalti['text'] + " Anggaran"
dbh_pabum_royalti_header_r        = dbh_pabum_royalti['kodeakun'] + "-" + dbh_pabum_royalti['text'] + " Realisasi"

# 22
dbh_kehutanan_iiuph                 = datapemda['postur']['1']['child']['1']['child'][18]
dbh_kehutanan_iiuph_total_a         = dbh_kehutanan_iiuph['value']['anggaran']
dbh_kehutanan_iiuph_total_r         = dbh_kehutanan_iiuph['value']['realisasi']
dbh_kehutanan_iiuph_total_p         = dbh_kehutanan_iiuph['value']['persentase']
dbh_kehutanan_iiuph_header_a        = dbh_kehutanan_iiuph['kodeakun'] + "-" + dbh_kehutanan_iiuph['text'] + " Anggaran"
dbh_kehutanan_iiuph_header_r        = dbh_kehutanan_iiuph['kodeakun'] + "-" + dbh_kehutanan_iiuph['text'] + " Realisasi"

# 23
dbh_kehutanan_psdh                 = datapemda['postur']['1']['child']['1']['child'][19]
dbh_kehutanan_psdh_total_a         = dbh_kehutanan_psdh['value']['anggaran']
dbh_kehutanan_psdh_total_r         = dbh_kehutanan_psdh['value']['realisasi']
dbh_kehutanan_psdh_total_p         = dbh_kehutanan_psdh['value']['persentase']
dbh_kehutanan_psdh_header_a        = dbh_kehutanan_psdh['kodeakun'] + "-" + dbh_kehutanan_psdh['text'] + " Anggaran"
dbh_kehutanan_psdh_header_r        = dbh_kehutanan_psdh['kodeakun'] + "-" + dbh_kehutanan_psdh['text'] + " Realisasi"

# 24
dbh_kehutanan_danareboisasi                 = datapemda['postur']['1']['child']['1']['child'][20]
dbh_kehutanan_danareboisasi_total_a         = dbh_kehutanan_danareboisasi['value']['anggaran']
dbh_kehutanan_danareboisasi_total_r         = dbh_kehutanan_danareboisasi['value']['realisasi']
dbh_kehutanan_danareboisasi_total_p         = dbh_kehutanan_danareboisasi['value']['persentase']
dbh_kehutanan_danareboisasi_header_a        = dbh_kehutanan_danareboisasi['kodeakun'] + "-" + dbh_kehutanan_danareboisasi['text'] + "Anggaran"
dbh_kehutanan_danareboisasi_header_r        = dbh_kehutanan_danareboisasi['kodeakun'] + "-" + dbh_kehutanan_danareboisasi['text'] + "Realisasi"

# 25
dbh_perikanan                 = datapemda['postur']['1']['child']['1']['child'][21]
dbh_perikanan_total_a         = dbh_perikanan['value']['anggaran']
dbh_perikanan_total_r         = dbh_perikanan['value']['realisasi']
dbh_perikanan_total_p         = dbh_perikanan['value']['persentase']
dbh_perikanan_header_a        = dbh_perikanan['kodeakun'] + "-" + dbh_perikanan['text'] + " Anggaran"
dbh_perikanan_header_r        = dbh_perikanan['kodeakun'] + "-" + dbh_perikanan['text'] + " Realisasi"

# 26
dbh_cht                 = datapemda['postur']['1']['child']['1']['child'][22]
dbh_cht_total_a         = dbh_cht['value']['anggaran']
dbh_cht_total_r         = dbh_cht['value']['realisasi']
dbh_cht_total_p         = dbh_cht['value']['persentase']
dbh_cht_header_a        = dbh_cht['kodeakun'] + "-" + dbh_cht['text'] + " Anggaran"
dbh_cht_header_r        = dbh_cht['kodeakun'] + "-" + dbh_cht['text'] + " Realisasi"

# 27 Total DAU
dau_total_a = datapemda['postur']['1']['child']['2']['total']['anggaran']
dau_total_r = datapemda['postur']['1']['child']['2']['total']['realisasi']
dau_total_p = datapemda['postur']['1']['child']['2']['total']['persentase']
dau_header_a  = " Total DAU Anggaran"
dau_header_r  = " Total DAU Realisasi"

# 28 
dau                 = datapemda['postur']['1']['child']['2']['child'][0]
dau_value_a         = dau['value']['anggaran']
dau_value_r         = dau['value']['realisasi']
dau_value_p         = dau['value']['persentase']
dau_value_header_a        = dau['kodeakun'] + "-" + dau['text'] + " Anggaran"
dau_value_header_r        = dau['kodeakun'] + "-" + dau['text'] + " Realisasi"

# 29 Total DAK Fisik
dak_fisik_total_a = datapemda['postur']['1']['child']['3']['total']['anggaran']
dak_fisik_total_r = datapemda['postur']['1']['child']['3']['total']['realisasi']
dak_fisik_total_p = datapemda['postur']['1']['child']['3']['total']['persentase']
dak_fisik_total_header_a = "Total DAK Fisik Anggaran"
dak_fisik_total_header_r = "Total DAK Fisik Realisasi"

# 30
dak_fisik                 = datapemda['postur']['1']['child']['3']['child'][0]
dak_fisik_value_a         = dak_fisik['value']['anggaran']
dak_fisik_value_r         = dak_fisik['value']['realisasi']
dak_fisik_value_p         = dak_fisik['value']['persentase']
dak_fisik_header_a        = dak_fisik['kodeakun'] + "-" + dak_fisik['text'] + " Anggaran"
dak_fisik_header_r        = dak_fisik['kodeakun'] + "-" + dak_fisik['text'] + " Realisasi"

# 31
dak_fisik_infrastruktur                 = datapemda['postur']['1']['child']['3']['child'][1]
dak_fisik_infrastruktur_total_a         = dak_fisik_infrastruktur['value']['anggaran']
dak_fisik_infrastruktur_total_r         = dak_fisik_infrastruktur['value']['realisasi']
dak_fisik_infrastruktur_total_p         = dak_fisik_infrastruktur['value']['persentase']
dak_fisik_infrastruktur_header_a        = dak_fisik_infrastruktur['kodeakun'] + "-" + dak_fisik_infrastruktur['text'] + " Anggaran"
dak_fisik_infrastruktur_header_r        = dak_fisik_infrastruktur['kodeakun'] + "-" + dak_fisik_infrastruktur['text'] + " Realisasi"

# 32
dak_fisik_penugasan                 = datapemda['postur']['1']['child']['3']['child'][2]
dak_fisik_penugasan_total_a         = dak_fisik_penugasan['value']['anggaran']
dak_fisik_penugasan_total_r         = dak_fisik_penugasan['value']['realisasi']
dak_fisik_penugasan_total_p         = dak_fisik_penugasan['value']['persentase']
dak_fisik_penugasan_header_a        = dak_fisik_penugasan['kodeakun'] + "-" + dak_fisik_penugasan['text'] + " Anggaran"
dak_fisik_penugasan_header_r        = dak_fisik_penugasan['kodeakun'] + "-" + dak_fisik_penugasan['text'] + " Realisasi"

# 33
dak_fisik_afirmasi                 = datapemda['postur']['1']['child']['3']['child'][3]
dak_fisik_afirmasi_total_a         = dak_fisik_afirmasi['value']['anggaran']
dak_fisik_afirmasi_total_r         = dak_fisik_afirmasi['value']['realisasi']
dak_fisik_afirmasi_total_p         = dak_fisik_afirmasi['value']['persentase']
dak_fisik_afirmasi_header_a        = dak_fisik_afirmasi['kodeakun'] + "-" + dak_fisik_afirmasi['text'] + " Anggaran"
dak_fisik_afirmasi_header_r        = dak_fisik_afirmasi['kodeakun'] + "-" + dak_fisik_afirmasi['text'] + " Realisasi"

# 34
dak_nonfisik_total_a = datapemda['postur']['1']['child']['4']['total']['anggaran']
dak_nonfisik_total_r = datapemda['postur']['1']['child']['4']['total']['realisasi']
dak_nonfisik_total_p = datapemda['postur']['1']['child']['4']['total']['persentase']
dak_nonfisik_header_a = "Total DAK Non Fisik Anggaran"
dak_nonfisik_header_r = "Total DAK Non Fisik Realisasi"

# 35
dak_nonfisik_tpg                 = datapemda['postur']['1']['child']['4']['child'][0]
dak_nonfisik_tpg_total_a         = dak_nonfisik_tpg['value']['anggaran']
dak_nonfisik_tpg_total_r         = dak_nonfisik_tpg['value']['realisasi']
dak_nonfisik_tpg_total_p         = dak_nonfisik_tpg['value']['persentase']
dak_nonfisik_tpg_header_a = dak_nonfisik_tpg['kodeakun'] + "-" + dak_nonfisik_tpg['text'] + " Anggaran"
dak_nonfisik_tpg_header_r = dak_nonfisik_tpg['kodeakun'] + "-" + dak_nonfisik_tpg['text'] + " Realisasi"

# 36
dak_nonfisik_tkg_dasus                 = datapemda['postur']['1']['child']['4']['child'][1]
dak_nonfisik_tkg_dasus_total_a         = dak_nonfisik_tkg_dasus['value']['anggaran']
dak_nonfisik_tkg_dasus_total_r         = dak_nonfisik_tkg_dasus['value']['realisasi']
dak_nonfisik_tkg_dasus_total_p         = dak_nonfisik_tkg_dasus['value']['persentase']
dak_nonfisik_tkg_dasus_header_a = dak_nonfisik_tkg_dasus['kodeakun'] + "-" + dak_nonfisik_tkg_dasus['text'] + " Anggaran"
dak_nonfisik_tkg_dasus_header_r = dak_nonfisik_tkg_dasus['kodeakun'] + "-" + dak_nonfisik_tkg_dasus['text'] + " Realisasi"

# 37
dak_nonfisik_tamsil                 = datapemda['postur']['1']['child']['4']['child'][2]
dak_nonfisik_tamsil_total_a         = dak_nonfisik_tamsil['value']['anggaran']
dak_nonfisik_tamsil_total_r         = dak_nonfisik_tamsil['value']['realisasi']
dak_nonfisik_tamsil_total_p         = dak_nonfisik_tamsil['value']['persentase']
dak_nonfisik_tamsil_header_a        = dak_nonfisik_tamsil['kodeakun'] + "-" + dak_nonfisik_tamsil['text'] + " Anggaran"
dak_nonfisik_tamsil_header_r        = dak_nonfisik_tamsil['kodeakun'] + "-" + dak_nonfisik_tamsil['text'] + " Realisasi"

# 38
dak_nonfisik_bos                 = datapemda['postur']['1']['child']['4']['child'][3]
dak_nonfisik_bos_total_a         = dak_nonfisik_bos['value']['anggaran']
dak_nonfisik_bos_total_r         = dak_nonfisik_bos['value']['realisasi']
dak_nonfisik_bos_total_p         = dak_nonfisik_bos['value']['persentase']
dak_nonfisik_bos_header_a        = dak_nonfisik_bos['kodeakun'] + "-" + dak_nonfisik_bos['text'] + " Anggaran"
dak_nonfisik_bos_header_r        = dak_nonfisik_bos['kodeakun'] + "-" + dak_nonfisik_bos['text'] + " Realisasi"

# 39
dak_nonfisik_bos_kesehatan                 = datapemda['postur']['1']['child']['4']['child'][4]
dak_nonfisik_bos_kesehatan_total_a         = dak_nonfisik_bos_kesehatan['value']['anggaran']
dak_nonfisik_bos_kesehatan_total_r         = dak_nonfisik_bos_kesehatan['value']['realisasi']
dak_nonfisik_bos_kesehatan_total_p         = dak_nonfisik_bos_kesehatan['value']['persentase']
dak_nonfisik_bos_kesehatan_header_a        = dak_nonfisik_bos_kesehatan['kodeakun'] + "-" + dak_nonfisik_bos_kesehatan['text'] + " Anggaran"
dak_nonfisik_bos_kesehatan_header_r        = dak_nonfisik_bos_kesehatan['kodeakun'] + "-" + dak_nonfisik_bos_kesehatan['text'] + " Realisasi"

# 40
dak_nonfisik_bos_kb                 = datapemda['postur']['1']['child']['4']['child'][5]
dak_nonfisik_bos_kb_total_a         = dak_nonfisik_bos_kb['value']['anggaran']
dak_nonfisik_bos_kb_total_r         = dak_nonfisik_bos_kb['value']['realisasi']
dak_nonfisik_bos_kb_total_p         = dak_nonfisik_bos_kb['value']['persentase']
dak_nonfisik_bos_kb_header_a        = dak_nonfisik_bos_kb['kodeakun'] + "-" + dak_nonfisik_bos_kb['text'] + " Anggaran"
dak_nonfisik_bos_kb_header_r        = dak_nonfisik_bos_kb['kodeakun'] + "-" + dak_nonfisik_bos_kb['text'] + " Realisasi"

# 41
dak_nonfisik_koperasi                 = datapemda['postur']['1']['child']['4']['child'][6]
dak_nonfisik_koperasi_total_a         = dak_nonfisik_koperasi['value']['anggaran']
dak_nonfisik_koperasi_total_r         = dak_nonfisik_koperasi['value']['realisasi']
dak_nonfisik_koperasi_total_p         = dak_nonfisik_koperasi['value']['persentase']
dak_nonfisik_koperasi_header_a        = dak_nonfisik_koperasi['kodeakun'] + "-" + dak_nonfisik_koperasi['text'] + " Anggaran"
dak_nonfisik_koperasi_header_r        = dak_nonfisik_koperasi['kodeakun'] + "-" + dak_nonfisik_koperasi['text'] + " Realisasi"

# 42
dak_nonfisik_adminduk                 = datapemda['postur']['1']['child']['4']['child'][7]
dak_nonfisik_adminduk_total_a         = dak_nonfisik_adminduk['value']['anggaran']
dak_nonfisik_adminduk_total_r         = dak_nonfisik_adminduk['value']['realisasi']
dak_nonfisik_adminduk_total_p         = dak_nonfisik_adminduk['value']['persentase']
dak_nonfisik_adminduk_header_a        = dak_nonfisik_adminduk['kodeakun'] + "-" + dak_nonfisik_adminduk['text'] + " Anggaran"
dak_nonfisik_adminduk_header_r        = dak_nonfisik_adminduk['kodeakun'] + "-" + dak_nonfisik_adminduk['text'] + " Realisasi"

# 43
dak_nonfisik_bos_paud                 = datapemda['postur']['1']['child']['4']['child'][8]
dak_nonfisik_bos_paud_total_a         = dak_nonfisik_bos_paud['value']['anggaran']
dak_nonfisik_bos_paud_total_r         = dak_nonfisik_bos_paud['value']['realisasi']
dak_nonfisik_bos_paud_total_p         = dak_nonfisik_bos_paud['value']['persentase']
dak_nonfisik_bos_paud_header_a = dak_nonfisik_bos_paud['kodeakun'] + "-" + dak_nonfisik_bos_paud['text'] + " Anggaran"
dak_nonfisik_bos_paud_header_r = dak_nonfisik_bos_paud['kodeakun'] + "-" + dak_nonfisik_bos_paud['text'] + " Realisasi"

# 44 Total DID
did_total_a = datapemda['postur']['1']['child']['5']['total']['anggaran']
did_total_r = datapemda['postur']['1']['child']['5']['total']['realisasi']
did_total_p = datapemda['postur']['1']['child']['5']['total']['persentase']
did_total_header_a =  "Total DID Anggaran"
did_total_header_r =  "Total DID Realisasi"

# 45
did_value                 = datapemda['postur']['1']['child']['5']['child'][0]
did_value_total_a         = did_value['value']['anggaran']
did_value_total_r         = did_value['value']['realisasi']
did_value_total_p         = did_value['value']['persentase']
did_value_header_a        = did_value['kodeakun'] + "-" + did_value['text'] + " Anggaran"
did_value_header_r        = did_value['kodeakun'] + "-" + did_value['text'] + " Realisasi"

# 46 Total Otsus
otsus_total_a = datapemda['postur']['1']['child']['6']['total']['anggaran']
otsus_total_r = datapemda['postur']['1']['child']['6']['total']['realisasi']
otsus_total_p = datapemda['postur']['1']['child']['6']['total']['persentase']
otsus_total_header_a = "Total Otsus Anggaran"
otsus_total_header_r = "Total Otsus Realisasi"

# 47
otsus_aceh                 = datapemda['postur']['1']['child']['6']['child'][0]
otsus_aceh_total_a         = otsus_aceh['value']['anggaran']
otsus_aceh_total_r         = otsus_aceh['value']['realisasi']
otsus_aceh_total_p         = otsus_aceh['value']['persentase']
otsus_aceh_header_a        = otsus_aceh['kodeakun'] + "-" + otsus_aceh['text'] + " Anggaran"
otsus_aceh_header_r         = otsus_aceh['kodeakun'] + "-" + otsus_aceh['text'] + " Realisasi"

# 48
otsus_papua                 = datapemda['postur']['1']['child']['6']['child'][1]
otsus_papua_total_a         = otsus_papua['value']['anggaran']
otsus_papua_total_r         = otsus_papua['value']['realisasi']
otsus_papua_total_p         = otsus_papua['value']['persentase']
otsus_papua_header_a        = otsus_papua['kodeakun'] + "-" + otsus_papua['text'] + " Anggaran"
otsus_papua_header_r        = otsus_papua['kodeakun'] + "-" + otsus_papua['text'] + " Realisasi"

# 49
infrastruktur_papua                 = datapemda['postur']['1']['child']['6']['child'][2]
infrastruktur_papua_total_a         = infrastruktur_papua['value']['anggaran']
infrastruktur_papua_total_r         = infrastruktur_papua['value']['realisasi']
infrastruktur_papua_total_p         = infrastruktur_papua['value']['persentase']
infrastruktur_papua_header_a        = infrastruktur_papua['kodeakun'] + "-" + infrastruktur_papua['text'] + " Anggaran"
infrastruktur_papua_header_r        = infrastruktur_papua['kodeakun'] + "-" + infrastruktur_papua['text'] + " Realisasi"

# 50
otsus_papuabarat                 = datapemda['postur']['1']['child']['6']['child'][3]
otsus_papuabarat_total_a         = otsus_papuabarat['value']['anggaran']
otsus_papuabarat_total_r         = otsus_papuabarat['value']['realisasi']
otsus_papuabarat_total_p         = otsus_papuabarat['value']['persentase']
otsus_papuabarat_header_a        = otsus_papuabarat['kodeakun'] + "-" + otsus_papuabarat['text'] + " Anggaran"
otsus_papuabarat_header_r        = otsus_papuabarat['kodeakun'] + "-" + otsus_papuabarat['text'] + " Realisasi"

# 51
infrastruktur_papuabarat                 = datapemda['postur']['1']['child']['6']['child'][4]
infrastruktur_papuabarat_total_a         = infrastruktur_papuabarat['value']['anggaran']
infrastruktur_papuabarat_total_r         = infrastruktur_papuabarat['value']['realisasi']
infrastruktur_papuabarat_total_p         = infrastruktur_papuabarat['value']['persentase']
infrastruktur_papuabarat_header_a        = infrastruktur_papuabarat['kodeakun'] + "-" + infrastruktur_papuabarat['text'] + " Anggaran"
infrastruktur_papuabarat_header_r        = infrastruktur_papuabarat['kodeakun'] + "-" + infrastruktur_papuabarat['text'] + " Realisasi"

# 52 Total Dais
dais_total_a = datapemda['postur']['1']['child']['7']['total']['anggaran']
dais_total_r = datapemda['postur']['1']['child']['7']['total']['realisasi']
dais_total_p = datapemda['postur']['1']['child']['7']['total']['persentase']
dais_total_header_a = " Total Dais Anggaran"
dais_total_header_r = " Total Dais Realisasi"

# 53
dais_value                 = datapemda['postur']['1']['child']['7']['child'][0]
dais_value_total_a         = dais_value['value']['anggaran']
dais_value_total_r         = dais_value['value']['realisasi']
dais_value_total_p         = dais_value['value']['persentase']
dais_value_header_a        = dais_value['kodeakun'] + "-" + dais_value['text'] + " Anggaran"
dais_value_header_r        = dais_value['kodeakun'] + "-" + dais_value['text'] + " Realisasi"

# 54 Dana Desa
dd_total_a = datapemda['postur']['1']['total']['anggaran']
dd_total_r = datapemda['postur']['1']['total']['realisasi']
dd_total_p = datapemda['postur']['1']['total']['persentase']
dd_total_header_a = "Total Dana Desa Anggaran"
dd_total_header_r = "Total Dana Desa Realisasi"

# 55
dd_value                 = datapemda['postur']['2']['child']['1']['child'][0]
dd_value_total_a         = dd_value['value']['anggaran']
dd_value_total_r         = dd_value['value']['realisasi']
dd_value_total_p         = dd_value['value']['persentase']
dd_value_header_a        = dd_value['kodeakun'] + "-" + dd_value['text'] + " Anggaran"
dd_value_header_r        = dd_value['kodeakun'] + "-" + dd_value['text'] + " Realisasi"


headers = ["no", 
           # tahun, wilayah, provinsi, disclaimer, 
           tkdd_total_a_header, tkdd_total_r_header, 
           tkd_total_a_header, tkd_total_r_header,
           dbh_total_a_header, dbh_total_r_header, 
           # 4
           dbh_pph21_header_a, dbh_pph21_header_r,
           #5 
           dbh_pph2529opdn_header_a, dbh_pph2529opdn_header_r,
           #6 
           dbh_pbbprovinsi_header_a, dbh_pbbprovinsi_header_r,
           #7
           dbh_pbbkabkot_header_a, dbh_pbbkabkot_header_r,
           #8
           dbh_pbbupahpungutprov_header_a, dbh_pbbupahpungutprov_header_r,
           #9
           dbh_pbbupahpungutkabkot_header_a, dbh_pbbupahpungutkabkot_header_r,
           # 10
           dbh_pbbbagpusatrata_header_a, dbh_pbbbagpusatrata_header_r,
           # 11
           dbh_minyak_header_a, dbh_minyak_header_r,
           # 12
           dbh_minyak_setengah_persen_header_a, dbh_minyak_setengah_persen_header_r,
           #13
           dbh_minyak_otsus_header_a, dbh_minyak_otsus_header_r,
           # 14
           dbh_gasbumi_header_a, dbh_gasbumi_header_r,
           # 15
           dbh_gasbumi_setengah_persen_header_a, dbh_gasbumi_setengah_persen_header_r,
           # 16
           dbh_gasbumi_otsus_header_a, dbh_gasbumi_otsus_header_r,
           # 17
           dbh_pertum_iurantetap_header_a, dbh_pertum_iurantetap_header_r,
           # 18
           dbh_pertum_royalti_header_a, dbh_pertum_royalti_header_r,
           # 19
           dbh_pabum_setoranbagpemerintah_header_a, dbh_pabum_setoranbagpemerintah_header_r, 
           # 20
           dbh_pabum_iurantetap_header_a, dbh_pabum_iurantetap_header_r,
           # 21
           dbh_pabum_royalti_header_a, dbh_pabum_royalti_header_r,
           # 22
           dbh_kehutanan_iiuph_header_a, dbh_kehutanan_iiuph_header_r,
           # 23
           dbh_kehutanan_psdh_header_a, dbh_kehutanan_psdh_header_r,
           # 24
           dbh_kehutanan_danareboisasi_header_a, dbh_kehutanan_danareboisasi_header_r,
           # 25
           dbh_perikanan_header_a, dbh_perikanan_header_r,
           # 26
           dbh_cht_header_a, dbh_cht_header_r,
           #27
           dau_header_a, dau_header_r,
           # 28
           dau_value_header_a, dau_value_header_r,  
           # 29
           dak_fisik_total_header_a, dak_fisik_total_header_r,
           # 30
           dak_fisik_header_a, dak_fisik_header_r,
           # 31
           dak_fisik_infrastruktur_header_a, dak_fisik_infrastruktur_header_r,
           #33
           dak_fisik_penugasan_header_a, dak_fisik_penugasan_header_r,
           #33
           dak_fisik_afirmasi_header_a, dak_fisik_afirmasi_header_r,
           #34
           dak_nonfisik_header_a, dak_nonfisik_header_r,
           #35
           dak_nonfisik_tpg_header_a, dak_nonfisik_tpg_header_r,
           #36
           dak_nonfisik_tkg_dasus_header_a, dak_nonfisik_tkg_dasus_header_r,
           #37
           dak_nonfisik_tamsil_header_a, dak_nonfisik_tamsil_header_r,
           #38
           dak_nonfisik_bos_header_a, dak_nonfisik_bos_header_r,
           #39
           dak_nonfisik_bos_kesehatan_header_a, dak_nonfisik_bos_kesehatan_header_r,
           #40
           dak_nonfisik_bos_kb_header_a, dak_nonfisik_bos_kb_header_r,
           #41
           dak_nonfisik_koperasi_header_a, dak_nonfisik_koperasi_header_r,
           #42
           dak_nonfisik_adminduk_header_a, dak_nonfisik_adminduk_header_r,
           #43
           dak_nonfisik_bos_paud_header_a, dak_nonfisik_bos_paud_header_r,
           #44
           did_total_header_a, did_total_header_r,
           # 45 
           did_value_header_a, did_value_header_r,
           # 46
           otsus_total_header_a, otsus_total_header_r,
           # 47
           otsus_aceh_header_a, otsus_aceh_header_r,
           # 48
           otsus_papua_header_a, otsus_papua_header_r,
           # 49
           infrastruktur_papua_header_a, infrastruktur_papua_header_r,
           #50 
           otsus_papuabarat_header_a, otsus_papuabarat_header_r,
           #51
           infrastruktur_papuabarat_header_a, infrastruktur_papuabarat_header_r,
           #52
           dais_total_header_a, dais_total_header_r,
           # 53
           dais_value_header_a, dais_value_header_r,
           # 54
           dd_total_header_a, dd_total_header_r,
           # 55
           dd_value_header_a, dd_value_header_r
]





isi =[no, 
      #tahun, wilayah, provinsi, disclaimer, 
      tkdd_total_a, tkdd_total_r, 
      tkd_total_a, tkd_total_r,
      dbh_total_a, dbh_total_r, 
      dbh_pph21_total_a, dbh_pph21_total_r, 
      dbh_pph2529opdn_total_a,       dbh_pph2529opdn_total_r, 
      dbh_pbbprovinsi_total_a, dbh_pbbprovinsi_total_r,
      dbh_pbbkabkot_total_a, dbh_pbbkabkot_total_r, 
      dbh_pbbupahpungutprov_total_a, dbh_pbbupahpungutprov_total_r, 
      dbh_pbbupahpungutkabkot_total_a, dbh_pbbupahpungutkabkot_total_r,
      dbh_pbbbagpusatrata_total_a, dbh_pbbbagpusatrata_total_r,
      dbh_minyak_total_a, dbh_minyak_total_r, 
      dbh_minyak_setengah_persen_total_a, dbh_minyak_setengah_persen_total_r, 
      dbh_minyak_otsus_total_a, dbh_minyak_otsus_total_r,
      dbh_gasbumi_total_a, dbh_gasbumi_total_r, 
      dbh_gasbumi_setengah_persen_total_a, dbh_gasbumi_setengah_persen_total_r,
      dbh_gasbumi_otsus_total_a, dbh_gasbumi_otsus_total_r, 
      dbh_pertum_iurantetap_total_a, dbh_pertum_iurantetap_total_r,
      dbh_pertum_royalti_total_a, dbh_pertum_royalti_total_r, 
      dbh_pabum_setoranbagpemerintah_total_a, dbh_pabum_setoranbagpemerintah_total_r, 
      dbh_pabum_iurantetap_total_a, dbh_pabum_iurantetap_total_r,
      dbh_pabum_royalti_total_a, dbh_pabum_royalti_total_r,
      dbh_kehutanan_iiuph_total_a, dbh_kehutanan_iiuph_total_r,
      dbh_kehutanan_psdh_total_a, dbh_kehutanan_psdh_total_r, 
      dbh_kehutanan_danareboisasi_total_a, dbh_kehutanan_danareboisasi_total_r, 
      dbh_perikanan_total_a, dbh_perikanan_total_r,
      dbh_cht_total_a, dbh_cht_total_r,
      dau_total_a, dau_total_r, 
      dau_total_a, dau_total_r,
      dak_fisik_total_a, dak_fisik_total_r,
      dak_fisik_total_a, dak_fisik_total_r,
      dak_fisik_infrastruktur_total_a, dak_fisik_infrastruktur_total_r,
      dak_fisik_penugasan_total_a, dak_fisik_penugasan_total_r, 
      dak_fisik_afirmasi_total_a, dak_fisik_afirmasi_total_r,
      dak_nonfisik_total_a, dak_nonfisik_total_r, 
      dak_nonfisik_tpg_total_a, dak_nonfisik_tpg_total_r,
      dak_nonfisik_tkg_dasus_total_a, dak_nonfisik_tkg_dasus_total_r,
      dak_nonfisik_tamsil_total_a, dak_nonfisik_tamsil_total_r,
      dak_nonfisik_bos_total_a, dak_nonfisik_bos_total_r,
      dak_nonfisik_bos_kesehatan_total_a, dak_nonfisik_bos_kesehatan_total_r,
      dak_nonfisik_bos_kb_total_a, dak_nonfisik_bos_kb_total_r,
      dak_nonfisik_koperasi_total_a, dak_nonfisik_koperasi_total_r,
      dak_nonfisik_adminduk_total_a, dak_nonfisik_adminduk_total_r,
      dak_nonfisik_bos_paud_total_a, dak_nonfisik_bos_paud_total_r, 
      did_total_a, did_total_r, 
      did_value_total_a, did_value_total_r, 
      otsus_total_a, otsus_total_r,
      otsus_aceh_total_a, otsus_aceh_total_r, 
      otsus_papua_total_a, otsus_papua_total_r, 
      infrastruktur_papua_total_a, infrastruktur_papua_total_r,
      otsus_papuabarat_total_a, otsus_papuabarat_total_r,
      infrastruktur_papuabarat_total_a, infrastruktur_papuabarat_total_r, 
      dais_total_a, dais_total_r, 
      dais_value_total_a, dais_value_total_r,
      #DanaDesa, 
      dd_total_a, dd_total_r, dd_value_total_a, dd_value_total_r]
print(isi)

print("Panjang header: " + str(len(headers)))
print("Panjang isi: " + str(len(isi)))

[1, 763712259647840, 757533338915843, 703712259647840, 697657198541618, 89225342014000, 93705559169270, 32986586000000, 27458081283272, 4441882000000, 2533606329484, 2857407829000, 2737579489820, 11211563981000, 10840669654158, 86252212000, 84199185177, 398764072000, 390261862177, 1736910100000, 1650781100705, 8581982674000, 12376493484658, 282732216000, 407714843494, 370605160000, 481008154249, 5966265026000, 8745074752035, 96882460000, 124683590580, 270583164000, 340653879134, 615858746000, 574574040835, 13670958915000, 19233330257405, 545092142000, 814945712335, 14680315000, 17093550748, 703068000, 897327311, 164730247000, 165979915021, 736821430000, 706696713825, 743479609000, 741420178157, 480000648000, 357622699445, 2964600000000, 2922191165245, 401489579649000, 401489579649000, 401489579649000, 401489579649000, 62436262908000, 58212071587909, 62436262908000, 58212071587909, 0, None, 24463658880000, 22492927163424, 6621768074000, 6259405435103, 121001491890840, 115300279949439, 5

# Portal TKDD dipetani

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import urllib3
import json

session = None
# Snippet Save to Google Drive
from datetime import datetime
from pytz import timezone
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

def set_global_session():
    global session
    if not session:
        session = requests.Session()
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        session.verify = False

def get_alldata():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            token = bsoup.find("input", {"name":"_token"})['value']
            options_provinsi = bsoup.find("select", {"id":"sel_prov"})
            allprovinsi = {}
            for option_provinsi in options_provinsi.find_all("option"):
                allprovinsi[option_provinsi['value']]=option_provinsi.text 
            return [token, allprovinsi]

def get_token():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            token = bsoup.find("input", {"name":"_token"})['value']
            return token

def get_provinsi():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            options_provinsi = bsoup.find("select", {"id":"sel_prov"})
            allprovinsi = []
            for option_provinsi in options_provinsi.find_all("option"):
                allprovinsi[option_provinsi['value']]=option_provinsi['value'].text
            return allprovinsi

def get_pemdas(provinsi):
    with session.get('http://www.djpk.kemenkeu.go.id/portal/pemda/' + provinsi) as response:
        if response.status_code==200 and len(response.text) != 0:
            pemdas = json.loads(response.text)
            return pemdas

def get_headers(token, tahun=2018):
    with session.post('http://www.djpk.kemenkeu.go.id/portal/filter', 
                data={'_token': token, 'data': 'tkdd', 'tahun': tahun, 'provinsi': '--', 'pemda': '--'}) as response:# query nasional saja as response
        if response.status_code==200 and len(response.text) != 0:
            print(response.text)
            # olah gaes
            # {"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Kabupaten\/kota","value":{"anggaran":398764072000,"realisasi":390261862177,"persentase":"97,87 %"}},{"kodeakun":"611215","text":"Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten\/Kota","value":{"anggaran":1736910100000,"realisasi":1650781100705,"persentase":"95,04 %"}},{"kodeakun":"612111","text":"Transfer DBH Minyak Bumi","value":{"anggaran":8581982674000,"realisasi":12376493484658,"persentase":"144,21 %"}},{"kodeakun":"612112","text":"Transfer DBH Minyak Bumi 0.5%","value":{"anggaran":282732216000,"realisasi":407714843494,"persentase":"144,21 %"}},{"kodeakun":"612113","text":"Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus","value":{"anggaran":370605160000,"realisasi":481008154249,"persentase":"129,79 %"}},{"kodeakun":"612211","text":"Transfer DBH Gas Bumi","value":{"anggaran":5966265026000,"realisasi":8745074752035,"persentase":"146,58 %"}},{"kodeakun":"612212","text":"Transfer DBH Gas Bumi 0.5%","value":{"anggaran":96882460000,"realisasi":124683590580,"persentase":"128,70 %"}},{"kodeakun":"612213","text":"Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus","value":{"anggaran":270583164000,"realisasi":340653879134,"persentase":"125,90 %"}},{"kodeakun":"612311","text":"Transfer DBH Pertambangan Umum - Iuran Tetap","value":{"anggaran":615858746000,"realisasi":574574040835,"persentase":"93,30 %"}},{"kodeakun":"612312","text":"Transfer DBH Pertambangan Umum - Royalti","value":{"anggaran":13670958915000,"realisasi":19233330257405,"persentase":"140,69 %"}},{"kodeakun":"612411","text":"Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah","value":{"anggaran":545092142000,"realisasi":814945712335,"persentase":"149,51 %"}},{"kodeakun":"612412","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Tetap","value":{"anggaran":14680315000,"realisasi":17093550748,"persentase":"116,44 %"}},{"kodeakun":"612413","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)","value":{"anggaran":703068000,"realisasi":897327311,"persentase":"127,63 %"}},{"kodeakun":"612511","text":"Transfer DBH Kehutanan - IIUPH\/IHPH","value":{"anggaran":164730247000,"realisasi":165979915021,"persentase":"100,76 %"}},{"kodeakun":"612512","text":"Transfer DBH Kehutanan - PSDH","value":{"anggaran":736821430000,"realisasi":706696713825,"persentase":"95,91 %"}},{"kodeakun":"612513","text":"Transfer DBH Kehutanan - Dana Reboisasi","value":{"anggaran":743479609000,"realisasi":741420178157,"persentase":"99,72 %"}},{"kodeakun":"612611","text":"Transfer DBH Perikanan","value":{"anggaran":480000648000,"realisasi":357622699445,"persentase":"74,50 %"}},{"kodeakun":"613111","text":"Transfer DBH Cukai Hasil Tembakau","value":{"anggaran":2964600000000,"realisasi":2922191165245,"persentase":"98,57 %"}}],"total":{"anggaran":89225342014000,"realisasi":93705559169270,"persentase":"105,02 %"}},"2":{"text":"DAU","child":[{"kodeakun":"621111","text":"Dana Alokasi Umum","value":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}}],"total":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}},"3":{"text":"DAK FISIK","child":[{"kodeakun":"631111","text":"Dana Alokasi Khusus","value":{"anggaran":31350835954000,"realisasi":29459738989382,"persentase":"93,97 %"}},{"kodeakun":"631211","text":"Dana Alokasi Khusus Infrastruktur Publik Daerah","value":{"anggaran":0,"realisasi":null,"persentase":"-"}},{"kodeakun":"631212","text":"Dana Alokasi Khusus Penugasan","value":{"anggaran":24463658880000,"realisasi":22492927163424,"persentase":"91,94 %"}},{"kodeakun":"631311","text":"Dana Alokasi Khusus Affirmasi","value":{"anggaran":6621768074000,"realisasi":6259405435103,"persentase":"94,53 %"}}],"total":{"anggaran":62436262908000,"realisasi":58212071587909,"persentase":"93,23 %"}},"6":{"text":"OTSUS","child":[{"kodeakun":"641111","text":"Dana Otonomi Khusus Provinsi Aceh","value":{"anggaran":8029791593000,"realisasi":8029791593000,"persentase":"100,00 %"}},{"kodeakun":"641211","text":"Dana Otonomi Khusus Provinsi Papua","value":{"anggaran":5620854115000,"realisasi":5620854115000,"persentase":"100,00 %"}},{"kodeakun":"641212","text":"Dana Tambahan Infrastruktur Provinsi Papua","value":{"anggaran":2400000000000,"realisasi":2400000000000,"persentase":"100,00 %"}},{"kodeakun":"641311","text":"Dana Otonomi Khusus Provinsi Papua Barat","value":{"anggaran":2408937478000,"realisasi":2408937478000,"persentase":"100,00 %"}},{"kodeakun":"641312","text":"Dana Tambahan Infrastruktur Provinsi Papua Barat","value":{"anggaran":1600000000000,"realisasi":1600000000000,"persentase":"100,00 %"}}],"total":{"anggaran":20059583186000,"realisasi":20059583186000,"persentase":"100,00 %"}},"7":{"text":"KEISTIMEWAAN DIY","child":[{"kodeakun":"642111","text":"Dana Keistimewaan Daerah Istimewa Yogyakarta","value":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}}],"total":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}},"5":{"text":"DID","child":[{"kodeakun":"643111","text":"Dana Insentif Daerah","value":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}}],"total":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}},"4":{"text":"DAK NON FISIK","child":[{"kodeakun":"654111","text":"Dana Tunjangan Profesi Guru PNSD","value":{"anggaran":56861747531798,"realisasi":55016720212332,"persentase":"96,76 %"}},{"kodeakun":"654112","text":"Dana Tunjangan Khusus Guru PNSD di Daerah Khusus","value":{"anggaran":1922303294676,"realisasi":1727102404861,"persentase":"89,85 %"}},{"kodeakun":"654211","text":"Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil","value":{"anggaran":879468864366,"realisasi":588007130574,"persentase":"66,86 %"}},{"kodeakun":"654311","text":"Dana Bantuan Operasional Sekolah","value":{"anggaran":46311243600000,"realisasi":44367351931006,"persentase":"95,80 %"}},{"kodeakun":"654711","text":"Dana Bantuan Operasional Kesehatan","value":{"anggaran":8551232700000,"realisasi":7454358890915,"persentase":"87,17 %"}},{"kodeakun":"654712","text":"Dana Bantuan Operasional Keluarga Berencana","value":{"anggaran":1808787300000,"realisasi":1753419417979,"persentase":"96,94 %"}},{"kodeakun":"654811","text":"Dana Peningkatan Kapasitas Koperasi","value":{"anggaran":100000000000,"realisasi":92221616897,"persentase":"92,22 %"}},{"kodeakun":"654814","text":"Dana Pelayanan Administrasi Kependudukan","value":{"anggaran":825000000000,"realisasi":745193742984,"persentase":"90,33 %"}},{"kodeakun":"654911","text":"Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini","value":{"anggaran":3741708600000,"realisasi":3555904601891,"persentase":"95,03 %"}}],"total":{"anggaran":121001491890840,"realisasi":115300279949439,"persentase":"95,29 %"}}},"total":{"anggaran":703712259647840,"realisasi":697657198541618,"persentase":"99,14 %"}},"2":{"text":"Dana Desa","child":{"1":{"text":"DANA DESA","child":[{"kodeakun":"662111","text":"Dana Desa","value":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}],"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"wilayah":"Nasional","judul":"POSTUR TKDD TA 2018","link_excel":"http:\/\/www.djpk.kemenkeu.go.id\/portal\/data\/csv\/2018\/--\/--\/tkdd","disclaimer":"Data bulan desember","header":["Alokasi","Realisasi"],"special_row":{"text":"Transfer ke daerah dan Dana Desa","total":{"anggaran":763712259647840,"realisasi":757533338915843,"persentase":"99,19 %"}}}

            kodeakun = 0

            return kodeakun

def get_data_tkdd(token, provinsi, pemda, tahun=2018):
    try:
        with session.post('http://www.djpk.kemenkeu.go.id/portal/filter', 
                    data={'_token': token, 'data': 'tkdd', 'tahun': tahun, 'provinsi': provinsi, 'pemda': pemda}) as response:
            if response.status_code==200 and len(response.text) != 0:
                return response.text
    except requests.exceptions.ConnectionError as e:
        return False
        pass
    except Exception as e:
        logger.error(e)
        randomtime = random.randint(1,5)
        logger.warn('ERROR - Retrying again website %s, retrying in %d secs' % (url, randomtime))
        time.sleep(randomtime)
        pass
        return False

def save_to_gdrive():
    path = F"/content/drive/My Drive/Colab Notebooks/Data DJSN/"
    filename = "data-tkdd-"+ datetime.now(timezone('Asia/Jakarta')).strftime("%Y-%m-%d--%H-%M") + ".csv"
    return True

def mapping(datapemda, provinsi):
    wilayah = datapemda['wilayah']
    disclaimer = datapemda['disclaimer']

    # 1 Total TKDD
    tkdd_total_a = datapemda['special_row']['total']['anggaran']
    tkdd_total_r = datapemda['special_row']['total']['realisasi']
    tkdd_total_a_header = "Total TKDD Anggaran"
    tkdd_total_r_header = "Total TKDD Realisasi"

    # 2 Total TKD
    tkd_total_a = datapemda['postur']['1']['total']['anggaran']
    tkd_total_r = datapemda['postur']['1']['total']['realisasi']
    tkd_total_a_header = "Total TKD Anggaran"
    tkd_total_r_header = "Total TKD Realisasi"

    # 3 Total DBH
    dbh_total_a = datapemda['postur']['1']['child']['1']['total']['anggaran']
    dbh_total_r = datapemda['postur']['1']['child']['1']['total']['realisasi']
    dbh_total_a_header = "Total DBH Anggaran"
    dbh_total_r_header = "Total DBH Realisasi"

    # hitung brp dbh yang ada
    print("Jumlah jenis DBH yg diterima: " + str(len(datapemda['postur']['1']['child']['1']['child'])))
    
    # 4
    dbh_pph21                 = datapemda['postur']['1']['child']['1']['child'][0]
    dbh_pph21_total_a         = dbh_pph21['value']['anggaran']
    dbh_pph21_total_r         = dbh_pph21['value']['realisasi']
    dbh_pph21_header_a          = dbh_pph21['kodeakun'] + "-" + dbh_pph21['text'] + " Anggaran"
    dbh_pph21_header_r          = dbh_pph21['kodeakun'] + "-" + dbh_pph21['text'] + " Realisasi"

    # 5
    dbh_pph2529opdn                 = datapemda['postur']['1']['child']['1']['child'][1]
    dbh_pph2529opdn_total_a         = dbh_pph2529opdn['value']['anggaran']
    dbh_pph2529opdn_total_r         = dbh_pph2529opdn['value']['realisasi']
    dbh_pph2529opdn_header_a          = dbh_pph2529opdn['kodeakun'] + "-" + dbh_pph2529opdn['text']  + " Anggaran"
    dbh_pph2529opdn_header_r          = dbh_pph2529opdn['kodeakun'] + "-" + dbh_pph2529opdn['text']  + " Realisasi"

    # 6
    dbh_pbbprovinsi                 = datapemda['postur']['1']['child']['1']['child'][2]
    dbh_pbbprovinsi_total_a         = dbh_pbbprovinsi['value']['anggaran']
    dbh_pbbprovinsi_total_r         = dbh_pbbprovinsi['value']['realisasi']
    dbh_pbbprovinsi_total_p         = dbh_pbbprovinsi['value']['persentase']
    dbh_pbbprovinsi_header_a        = dbh_pbbprovinsi['kodeakun'] + "-" + dbh_pbbprovinsi['text'] + " Anggaran"
    dbh_pbbprovinsi_header_r        = dbh_pbbprovinsi['kodeakun'] + "-" + dbh_pbbprovinsi['text'] + " Realisasi"

    # 7
    dbh_pbbkabkot                 = datapemda['postur']['1']['child']['1']['child'][3]
    dbh_pbbkabkot_total_a         = dbh_pbbkabkot['value']['anggaran']
    dbh_pbbkabkot_total_r         = dbh_pbbkabkot['value']['realisasi']
    dbh_pbbkabkot_total_p         = dbh_pbbkabkot['value']['persentase']
    dbh_pbbkabkot_header_a        = dbh_pbbkabkot['kodeakun'] + "-" + dbh_pbbkabkot['text'] + " Anggaran"
    dbh_pbbkabkot_header_r        = dbh_pbbkabkot['kodeakun'] + "-" + dbh_pbbkabkot['text'] + " Realisasi"

    # 8
    dbh_pbbupahpungutprov                 = datapemda['postur']['1']['child']['1']['child'][4]
    dbh_pbbupahpungutprov_total_a         = dbh_pbbupahpungutprov['value']['anggaran']
    dbh_pbbupahpungutprov_total_r         = dbh_pbbupahpungutprov['value']['realisasi']
    dbh_pbbupahpungutprov_total_p         = dbh_pbbupahpungutprov['value']['persentase']
    dbh_pbbupahpungutprov_header_a        = dbh_pbbupahpungutprov['kodeakun'] + "-" + dbh_pbbupahpungutprov['text'] + " Anggaran"
    dbh_pbbupahpungutprov_header_r        = dbh_pbbupahpungutprov['kodeakun'] + "-" + dbh_pbbupahpungutprov['text'] + " Realisasi"


    # 9
    dbh_pbbupahpungutkabkot                 = datapemda['postur']['1']['child']['1']['child'][5]
    dbh_pbbupahpungutkabkot_total_a         = dbh_pbbupahpungutkabkot['value']['anggaran']
    dbh_pbbupahpungutkabkot_total_r         = dbh_pbbupahpungutkabkot['value']['realisasi']
    dbh_pbbupahpungutkabkot_total_p         = dbh_pbbupahpungutkabkot['value']['persentase']
    dbh_pbbupahpungutkabkot_header_a  = dbh_pbbupahpungutkabkot['kodeakun'] + "-" + dbh_pbbupahpungutkabkot['text'] + " Anggaran"
    dbh_pbbupahpungutkabkot_header_r  = dbh_pbbupahpungutkabkot['kodeakun'] + "-" + dbh_pbbupahpungutkabkot['text'] + " Realisasi"

    # 10
    dbh_pbbbagpusatrata                 = datapemda['postur']['1']['child']['1']['child'][6]
    dbh_pbbbagpusatrata_total_a         = dbh_pbbbagpusatrata['value']['anggaran']
    dbh_pbbbagpusatrata_total_r         = dbh_pbbbagpusatrata['value']['realisasi']
    dbh_pbbbagpusatrata_total_p         = dbh_pbbbagpusatrata['value']['persentase']
    dbh_pbbbagpusatrata_header_a        = dbh_pbbbagpusatrata['kodeakun'] + "-" + dbh_pbbbagpusatrata['text'] + " Anggaran"
    dbh_pbbbagpusatrata_header_r        = dbh_pbbbagpusatrata['kodeakun'] + "-" + dbh_pbbbagpusatrata['text'] + " Realisasi"

    # 11
    dbh_minyak                 = datapemda['postur']['1']['child']['1']['child'][7]
    dbh_minyak_total_a         = dbh_minyak['value']['anggaran']
    dbh_minyak_total_r         = dbh_minyak['value']['realisasi']
    dbh_minyak_total_p         = dbh_minyak['value']['persentase']
    dbh_minyak_header_a        = dbh_minyak['kodeakun'] + "-" + dbh_minyak['text'] + " Anggaran"
    dbh_minyak_header_r        = dbh_minyak['kodeakun'] + "-" + dbh_minyak['text'] + " Realisasi"

    # 12
    if len(datapemda['postur']['1']['child']['1']['child'][8]) != 0:
        dbh_minyak_setengah_persen                 = datapemda['postur']['1']['child']['1']['child'][8] 
        dbh_minyak_setengah_persen_total_a         = dbh_minyak_setengah_persen['value']['anggaran']
        dbh_minyak_setengah_persen_total_r         = dbh_minyak_setengah_persen['value']['realisasi']
        dbh_minyak_setengah_persen_total_p         = dbh_minyak_setengah_persen['value']['persentase']
        dbh_minyak_setengah_persen_header_a        = dbh_minyak_setengah_persen['kodeakun'] + "-" + dbh_minyak_setengah_persen['text'] + " Anggaran"
        dbh_minyak_setengah_persen_header_r        = dbh_minyak_setengah_persen['kodeakun'] + "-" + dbh_minyak_setengah_persen['text'] + " Realisasi"

    # 13
    dbh_minyak_otsus                 = datapemda['postur']['1']['child']['1']['child'][9]
    dbh_minyak_otsus_total_a         = dbh_minyak_otsus['value']['anggaran']
    dbh_minyak_otsus_total_r         = dbh_minyak_otsus['value']['realisasi']
    dbh_minyak_otsus_total_p         = dbh_minyak_otsus['value']['persentase']
    dbh_minyak_otsus_header_a        = dbh_minyak_otsus['kodeakun'] + "-" + dbh_minyak_otsus['text'] + " Anggaran"
    dbh_minyak_otsus_header_r        = dbh_minyak_otsus['kodeakun'] + "-" + dbh_minyak_otsus['text'] + " Realisasi"

    # 14
    dbh_gasbumi                 = datapemda['postur']['1']['child']['1']['child'][10]
    dbh_gasbumi_total_a         = dbh_gasbumi['value']['anggaran']
    dbh_gasbumi_total_r         = dbh_gasbumi['value']['realisasi']
    dbh_gasbumi_total_p         = dbh_gasbumi['value']['persentase']
    dbh_gasbumi_header_a         = dbh_gasbumi['kodeakun'] + "-" + dbh_gasbumi['text'] + " Anggaran"
    dbh_gasbumi_header_r         = dbh_gasbumi['kodeakun'] + "-" + dbh_gasbumi['text'] + " Realisasi"

    # 15
    dbh_gasbumi_setengah_persen                 = datapemda['postur']['1']['child']['1']['child'][11]
    dbh_gasbumi_setengah_persen_total_a         = dbh_gasbumi_setengah_persen['value']['anggaran']
    dbh_gasbumi_setengah_persen_total_r         = dbh_gasbumi_setengah_persen['value']['realisasi']
    dbh_gasbumi_setengah_persen_total_p         = dbh_gasbumi_setengah_persen['value']['persentase']
    dbh_gasbumi_setengah_persen_header_a        = dbh_gasbumi_setengah_persen['kodeakun'] + "-" + dbh_gasbumi_setengah_persen['text'] + " Anggaran"
    dbh_gasbumi_setengah_persen_header_r        = dbh_gasbumi_setengah_persen['kodeakun'] + "-" + dbh_gasbumi_setengah_persen['text'] + " Realisasi"

    # 16
    dbh_gasbumi_otsus                 = datapemda['postur']['1']['child']['1']['child'][12]
    dbh_gasbumi_otsus_total_a         = dbh_gasbumi_otsus['value']['anggaran']
    dbh_gasbumi_otsus_total_r         = dbh_gasbumi_otsus['value']['realisasi']
    dbh_gasbumi_otsus_total_p         = dbh_gasbumi_otsus['value']['persentase']
    dbh_gasbumi_otsus_header_a        = dbh_gasbumi_otsus['kodeakun'] + "-" + dbh_gasbumi_otsus['text'] + " Anggaran"
    dbh_gasbumi_otsus_header_r        = dbh_gasbumi_otsus['kodeakun'] + "-" + dbh_gasbumi_otsus['text'] + " Realisasi"

    # 17 
    dbh_pertum_iurantetap                 = datapemda['postur']['1']['child']['1']['child'][13]
    dbh_pertum_iurantetap_total_a         = dbh_pertum_iurantetap['value']['anggaran']
    dbh_pertum_iurantetap_total_r         = dbh_pertum_iurantetap['value']['realisasi']
    dbh_pertum_iurantetap_total_p         = dbh_pertum_iurantetap['value']['persentase']
    dbh_pertum_iurantetap_header_a        = dbh_pertum_iurantetap['kodeakun'] + "-" + dbh_pertum_iurantetap['text'] + "Anggaran"
    dbh_pertum_iurantetap_header_r        = dbh_pertum_iurantetap['kodeakun'] + "-" + dbh_pertum_iurantetap['text'] + "Realisasi"

    # 18
    dbh_pertum_royalti                 = datapemda['postur']['1']['child']['1']['child'][14]
    dbh_pertum_royalti_total_a         = dbh_pertum_royalti['value']['anggaran']
    dbh_pertum_royalti_total_r         = dbh_pertum_royalti['value']['realisasi']
    dbh_pertum_royalti_total_p         = dbh_pertum_royalti['value']['persentase']
    dbh_pertum_royalti_header_a        = dbh_pertum_royalti['kodeakun'] + "-" + dbh_pertum_royalti['text'] + " Anggaran"
    dbh_pertum_royalti_header_r        = dbh_pertum_royalti['kodeakun'] + "-" + dbh_pertum_royalti['text'] + " Realisasi"

    # 19
    dbh_pabum_setoranbagpemerintah                 = datapemda['postur']['1']['child']['1']['child'][15]
    dbh_pabum_setoranbagpemerintah_total_a         = dbh_pabum_setoranbagpemerintah['value']['anggaran']
    dbh_pabum_setoranbagpemerintah_total_r         = dbh_pabum_setoranbagpemerintah['value']['realisasi']
    dbh_pabum_setoranbagpemerintah_total_p         = dbh_pabum_setoranbagpemerintah['value']['persentase']
    dbh_pabum_setoranbagpemerintah_header_a        = dbh_pabum_setoranbagpemerintah['kodeakun'] + "-" + dbh_pabum_setoranbagpemerintah['text'] + " Anggaran"
    dbh_pabum_setoranbagpemerintah_header_r        = dbh_pabum_setoranbagpemerintah['kodeakun'] + "-" + dbh_pabum_setoranbagpemerintah['text'] + " Realisasi"

    # 20
    dbh_pabum_iurantetap                 = datapemda['postur']['1']['child']['1']['child'][16]
    dbh_pabum_iurantetap_total_a         = dbh_pabum_iurantetap['value']['anggaran']
    dbh_pabum_iurantetap_total_r         = dbh_pabum_iurantetap['value']['realisasi']
    dbh_pabum_iurantetap_total_p         = dbh_pabum_iurantetap['value']['persentase']
    dbh_pabum_iurantetap_header_a        = dbh_pabum_iurantetap['kodeakun'] + "-" + dbh_pabum_iurantetap['text'] + " Anggaran"
    dbh_pabum_iurantetap_header_r        = dbh_pabum_iurantetap['kodeakun'] + "-" + dbh_pabum_iurantetap['text'] + " Realisasi"

    # 21
    dbh_pabum_royalti                 = datapemda['postur']['1']['child']['1']['child'][17]
    dbh_pabum_royalti_total_a         = dbh_pabum_royalti['value']['anggaran']
    dbh_pabum_royalti_total_r         = dbh_pabum_royalti['value']['realisasi']
    dbh_pabum_royalti_total_p         = dbh_pabum_royalti['value']['persentase']
    dbh_pabum_royalti_header_a        = dbh_pabum_royalti['kodeakun'] + "-" + dbh_pabum_royalti['text'] + " Anggaran"
    dbh_pabum_royalti_header_r        = dbh_pabum_royalti['kodeakun'] + "-" + dbh_pabum_royalti['text'] + " Realisasi"

    # 22
    dbh_kehutanan_iiuph                 = datapemda['postur']['1']['child']['1']['child'][18]
    dbh_kehutanan_iiuph_total_a         = dbh_kehutanan_iiuph['value']['anggaran']
    dbh_kehutanan_iiuph_total_r         = dbh_kehutanan_iiuph['value']['realisasi']
    dbh_kehutanan_iiuph_total_p         = dbh_kehutanan_iiuph['value']['persentase']
    dbh_kehutanan_iiuph_header_a        = dbh_kehutanan_iiuph['kodeakun'] + "-" + dbh_kehutanan_iiuph['text'] + " Anggaran"
    dbh_kehutanan_iiuph_header_r        = dbh_kehutanan_iiuph['kodeakun'] + "-" + dbh_kehutanan_iiuph['text'] + " Realisasi"

    # 23
    dbh_kehutanan_psdh                 = datapemda['postur']['1']['child']['1']['child'][19]
    dbh_kehutanan_psdh_total_a         = dbh_kehutanan_psdh['value']['anggaran']
    dbh_kehutanan_psdh_total_r         = dbh_kehutanan_psdh['value']['realisasi']
    dbh_kehutanan_psdh_total_p         = dbh_kehutanan_psdh['value']['persentase']
    dbh_kehutanan_psdh_header_a        = dbh_kehutanan_psdh['kodeakun'] + "-" + dbh_kehutanan_psdh['text'] + " Anggaran"
    dbh_kehutanan_psdh_header_r        = dbh_kehutanan_psdh['kodeakun'] + "-" + dbh_kehutanan_psdh['text'] + " Realisasi"

    # 24
    dbh_kehutanan_danareboisasi                 = datapemda['postur']['1']['child']['1']['child'][20]
    dbh_kehutanan_danareboisasi_total_a         = dbh_kehutanan_danareboisasi['value']['anggaran']
    dbh_kehutanan_danareboisasi_total_r         = dbh_kehutanan_danareboisasi['value']['realisasi']
    dbh_kehutanan_danareboisasi_total_p         = dbh_kehutanan_danareboisasi['value']['persentase']
    dbh_kehutanan_danareboisasi_header_a        = dbh_kehutanan_danareboisasi['kodeakun'] + "-" + dbh_kehutanan_danareboisasi['text'] + "Anggaran"
    dbh_kehutanan_danareboisasi_header_r        = dbh_kehutanan_danareboisasi['kodeakun'] + "-" + dbh_kehutanan_danareboisasi['text'] + "Realisasi"

    # 25
    dbh_perikanan                 = datapemda['postur']['1']['child']['1']['child'][21]
    dbh_perikanan_total_a         = dbh_perikanan['value']['anggaran']
    dbh_perikanan_total_r         = dbh_perikanan['value']['realisasi']
    dbh_perikanan_total_p         = dbh_perikanan['value']['persentase']
    dbh_perikanan_header_a        = dbh_perikanan['kodeakun'] + "-" + dbh_perikanan['text'] + " Anggaran"
    dbh_perikanan_header_r        = dbh_perikanan['kodeakun'] + "-" + dbh_perikanan['text'] + " Realisasi"

    # 26
    dbh_cht                 = datapemda['postur']['1']['child']['1']['child'][22]
    dbh_cht_total_a         = dbh_cht['value']['anggaran']
    dbh_cht_total_r         = dbh_cht['value']['realisasi']
    dbh_cht_total_p         = dbh_cht['value']['persentase']
    dbh_cht_header_a        = dbh_cht['kodeakun'] + "-" + dbh_cht['text'] + " Anggaran"
    dbh_cht_header_r        = dbh_cht['kodeakun'] + "-" + dbh_cht['text'] + " Realisasi"

    # 27 Total DAU
    dau_total_a = datapemda['postur']['1']['child']['2']['total']['anggaran']
    dau_total_r = datapemda['postur']['1']['child']['2']['total']['realisasi']
    dau_total_p = datapemda['postur']['1']['child']['2']['total']['persentase']
    dau_header_a  = " Total DAU Anggaran"
    dau_header_r  = " Total DAU Realisasi"

    # 28 
    dau                 = datapemda['postur']['1']['child']['2']['child'][0]
    dau_value_a         = dau['value']['anggaran']
    dau_value_r         = dau['value']['realisasi']
    dau_value_p         = dau['value']['persentase']
    dau_value_header_a        = dau['kodeakun'] + "-" + dau['text'] + " Anggaran"
    dau_value_header_r        = dau['kodeakun'] + "-" + dau['text'] + " Realisasi"

    # 29 Total DAK Fisik
    dak_fisik_total_a = datapemda['postur']['1']['child']['3']['total']['anggaran']
    dak_fisik_total_r = datapemda['postur']['1']['child']['3']['total']['realisasi']
    dak_fisik_total_p = datapemda['postur']['1']['child']['3']['total']['persentase']
    dak_fisik_total_header_a = "Total DAK Fisik Anggaran"
    dak_fisik_total_header_r = "Total DAK Fisik Realisasi"

    # 30
    dak_fisik                 = datapemda['postur']['1']['child']['3']['child'][0]
    dak_fisik_value_a         = dak_fisik['value']['anggaran']
    dak_fisik_value_r         = dak_fisik['value']['realisasi']
    dak_fisik_value_p         = dak_fisik['value']['persentase']
    dak_fisik_header_a        = dak_fisik['kodeakun'] + "-" + dak_fisik['text'] + " Anggaran"
    dak_fisik_header_r        = dak_fisik['kodeakun'] + "-" + dak_fisik['text'] + " Realisasi"

    # 31
    dak_fisik_infrastruktur                 = datapemda['postur']['1']['child']['3']['child'][1]
    dak_fisik_infrastruktur_total_a         = dak_fisik_infrastruktur['value']['anggaran']
    dak_fisik_infrastruktur_total_r         = dak_fisik_infrastruktur['value']['realisasi']
    dak_fisik_infrastruktur_total_p         = dak_fisik_infrastruktur['value']['persentase']
    dak_fisik_infrastruktur_header_a        = dak_fisik_infrastruktur['kodeakun'] + "-" + dak_fisik_infrastruktur['text'] + " Anggaran"
    dak_fisik_infrastruktur_header_r        = dak_fisik_infrastruktur['kodeakun'] + "-" + dak_fisik_infrastruktur['text'] + " Realisasi"

    # 32
    dak_fisik_penugasan                 = datapemda['postur']['1']['child']['3']['child'][2]
    dak_fisik_penugasan_total_a         = dak_fisik_penugasan['value']['anggaran']
    dak_fisik_penugasan_total_r         = dak_fisik_penugasan['value']['realisasi']
    dak_fisik_penugasan_total_p         = dak_fisik_penugasan['value']['persentase']
    dak_fisik_penugasan_header_a        = dak_fisik_penugasan['kodeakun'] + "-" + dak_fisik_penugasan['text'] + " Anggaran"
    dak_fisik_penugasan_header_r        = dak_fisik_penugasan['kodeakun'] + "-" + dak_fisik_penugasan['text'] + " Realisasi"

    # 33
    dak_fisik_afirmasi                 = datapemda['postur']['1']['child']['3']['child'][3]
    dak_fisik_afirmasi_total_a         = dak_fisik_afirmasi['value']['anggaran']
    dak_fisik_afirmasi_total_r         = dak_fisik_afirmasi['value']['realisasi']
    dak_fisik_afirmasi_total_p         = dak_fisik_afirmasi['value']['persentase']
    dak_fisik_afirmasi_header_a        = dak_fisik_afirmasi['kodeakun'] + "-" + dak_fisik_afirmasi['text'] + " Anggaran"
    dak_fisik_afirmasi_header_r        = dak_fisik_afirmasi['kodeakun'] + "-" + dak_fisik_afirmasi['text'] + " Realisasi"

    # 34
    dak_nonfisik_total_a = datapemda['postur']['1']['child']['4']['total']['anggaran']
    dak_nonfisik_total_r = datapemda['postur']['1']['child']['4']['total']['realisasi']
    dak_nonfisik_total_p = datapemda['postur']['1']['child']['4']['total']['persentase']
    dak_nonfisik_header_a = "Total DAK Non Fisik Anggaran"
    dak_nonfisik_header_r = "Total DAK Non Fisik Realisasi"

    # 35
    dak_nonfisik_tpg                 = datapemda['postur']['1']['child']['4']['child'][0]
    dak_nonfisik_tpg_total_a         = dak_nonfisik_tpg['value']['anggaran']
    dak_nonfisik_tpg_total_r         = dak_nonfisik_tpg['value']['realisasi']
    dak_nonfisik_tpg_total_p         = dak_nonfisik_tpg['value']['persentase']
    dak_nonfisik_tpg_header_a = dak_nonfisik_tpg['kodeakun'] + "-" + dak_nonfisik_tpg['text'] + " Anggaran"
    dak_nonfisik_tpg_header_r = dak_nonfisik_tpg['kodeakun'] + "-" + dak_nonfisik_tpg['text'] + " Realisasi"

    # 36
    dak_nonfisik_tkg_dasus                 = datapemda['postur']['1']['child']['4']['child'][1]
    dak_nonfisik_tkg_dasus_total_a         = dak_nonfisik_tkg_dasus['value']['anggaran']
    dak_nonfisik_tkg_dasus_total_r         = dak_nonfisik_tkg_dasus['value']['realisasi']
    dak_nonfisik_tkg_dasus_total_p         = dak_nonfisik_tkg_dasus['value']['persentase']
    dak_nonfisik_tkg_dasus_header_a = dak_nonfisik_tkg_dasus['kodeakun'] + "-" + dak_nonfisik_tkg_dasus['text'] + " Anggaran"
    dak_nonfisik_tkg_dasus_header_r = dak_nonfisik_tkg_dasus['kodeakun'] + "-" + dak_nonfisik_tkg_dasus['text'] + " Realisasi"

    # 37
    dak_nonfisik_tamsil                 = datapemda['postur']['1']['child']['4']['child'][2]
    dak_nonfisik_tamsil_total_a         = dak_nonfisik_tamsil['value']['anggaran']
    dak_nonfisik_tamsil_total_r         = dak_nonfisik_tamsil['value']['realisasi']
    dak_nonfisik_tamsil_total_p         = dak_nonfisik_tamsil['value']['persentase']
    dak_nonfisik_tamsil_header_a        = dak_nonfisik_tamsil['kodeakun'] + "-" + dak_nonfisik_tamsil['text'] + " Anggaran"
    dak_nonfisik_tamsil_header_r        = dak_nonfisik_tamsil['kodeakun'] + "-" + dak_nonfisik_tamsil['text'] + " Realisasi"

    # 38
    dak_nonfisik_bos                 = datapemda['postur']['1']['child']['4']['child'][3]
    dak_nonfisik_bos_total_a         = dak_nonfisik_bos['value']['anggaran']
    dak_nonfisik_bos_total_r         = dak_nonfisik_bos['value']['realisasi']
    dak_nonfisik_bos_total_p         = dak_nonfisik_bos['value']['persentase']
    dak_nonfisik_bos_header_a        = dak_nonfisik_bos['kodeakun'] + "-" + dak_nonfisik_bos['text'] + " Anggaran"
    dak_nonfisik_bos_header_r        = dak_nonfisik_bos['kodeakun'] + "-" + dak_nonfisik_bos['text'] + " Realisasi"

    # 39
    dak_nonfisik_bos_kesehatan                 = datapemda['postur']['1']['child']['4']['child'][4]
    dak_nonfisik_bos_kesehatan_total_a         = dak_nonfisik_bos_kesehatan['value']['anggaran']
    dak_nonfisik_bos_kesehatan_total_r         = dak_nonfisik_bos_kesehatan['value']['realisasi']
    dak_nonfisik_bos_kesehatan_total_p         = dak_nonfisik_bos_kesehatan['value']['persentase']
    dak_nonfisik_bos_kesehatan_header_a        = dak_nonfisik_bos_kesehatan['kodeakun'] + "-" + dak_nonfisik_bos_kesehatan['text'] + " Anggaran"
    dak_nonfisik_bos_kesehatan_header_r        = dak_nonfisik_bos_kesehatan['kodeakun'] + "-" + dak_nonfisik_bos_kesehatan['text'] + " Realisasi"

    # 40
    dak_nonfisik_bos_kb                 = datapemda['postur']['1']['child']['4']['child'][5]
    dak_nonfisik_bos_kb_total_a         = dak_nonfisik_bos_kb['value']['anggaran']
    dak_nonfisik_bos_kb_total_r         = dak_nonfisik_bos_kb['value']['realisasi']
    dak_nonfisik_bos_kb_total_p         = dak_nonfisik_bos_kb['value']['persentase']
    dak_nonfisik_bos_kb_header_a        = dak_nonfisik_bos_kb['kodeakun'] + "-" + dak_nonfisik_bos_kb['text'] + " Anggaran"
    dak_nonfisik_bos_kb_header_r        = dak_nonfisik_bos_kb['kodeakun'] + "-" + dak_nonfisik_bos_kb['text'] + " Realisasi"

    # 41
    dak_nonfisik_koperasi                 = datapemda['postur']['1']['child']['4']['child'][6]
    dak_nonfisik_koperasi_total_a         = dak_nonfisik_koperasi['value']['anggaran']
    dak_nonfisik_koperasi_total_r         = dak_nonfisik_koperasi['value']['realisasi']
    dak_nonfisik_koperasi_total_p         = dak_nonfisik_koperasi['value']['persentase']
    dak_nonfisik_koperasi_header_a        = dak_nonfisik_koperasi['kodeakun'] + "-" + dak_nonfisik_koperasi['text'] + " Anggaran"
    dak_nonfisik_koperasi_header_r        = dak_nonfisik_koperasi['kodeakun'] + "-" + dak_nonfisik_koperasi['text'] + " Realisasi"

    # 42
    dak_nonfisik_adminduk                 = datapemda['postur']['1']['child']['4']['child'][7]
    dak_nonfisik_adminduk_total_a         = dak_nonfisik_adminduk['value']['anggaran']
    dak_nonfisik_adminduk_total_r         = dak_nonfisik_adminduk['value']['realisasi']
    dak_nonfisik_adminduk_total_p         = dak_nonfisik_adminduk['value']['persentase']
    dak_nonfisik_adminduk_header_a        = dak_nonfisik_adminduk['kodeakun'] + "-" + dak_nonfisik_adminduk['text'] + " Anggaran"
    dak_nonfisik_adminduk_header_r        = dak_nonfisik_adminduk['kodeakun'] + "-" + dak_nonfisik_adminduk['text'] + " Realisasi"

    # 43
    dak_nonfisik_bos_paud                 = datapemda['postur']['1']['child']['4']['child'][8]
    dak_nonfisik_bos_paud_total_a         = dak_nonfisik_bos_paud['value']['anggaran']
    dak_nonfisik_bos_paud_total_r         = dak_nonfisik_bos_paud['value']['realisasi']
    dak_nonfisik_bos_paud_total_p         = dak_nonfisik_bos_paud['value']['persentase']
    dak_nonfisik_bos_paud_header_a = dak_nonfisik_bos_paud['kodeakun'] + "-" + dak_nonfisik_bos_paud['text'] + " Anggaran"
    dak_nonfisik_bos_paud_header_r = dak_nonfisik_bos_paud['kodeakun'] + "-" + dak_nonfisik_bos_paud['text'] + " Realisasi"

    # 44 Total DID
    did_total_a = datapemda['postur']['1']['child']['5']['total']['anggaran']
    did_total_r = datapemda['postur']['1']['child']['5']['total']['realisasi']
    did_total_p = datapemda['postur']['1']['child']['5']['total']['persentase']
    did_total_header_a =  "Total DID Anggaran"
    did_total_header_r =  "Total DID Realisasi"

    # 45
    did_value                 = datapemda['postur']['1']['child']['5']['child'][0]
    did_value_total_a         = did_value['value']['anggaran']
    did_value_total_r         = did_value['value']['realisasi']
    did_value_total_p         = did_value['value']['persentase']
    did_value_header_a        = did_value['kodeakun'] + "-" + did_value['text'] + " Anggaran"
    did_value_header_r        = did_value['kodeakun'] + "-" + did_value['text'] + " Realisasi"

    # 46 Total Otsus
    otsus_total_a = datapemda['postur']['1']['child']['6']['total']['anggaran']
    otsus_total_r = datapemda['postur']['1']['child']['6']['total']['realisasi']
    otsus_total_p = datapemda['postur']['1']['child']['6']['total']['persentase']
    otsus_total_header_a = "Total Otsus Anggaran"
    otsus_total_header_r = "Total Otsus Realisasi"

    # 47
    otsus_aceh                 = datapemda['postur']['1']['child']['6']['child'][0]
    otsus_aceh_total_a         = otsus_aceh['value']['anggaran']
    otsus_aceh_total_r         = otsus_aceh['value']['realisasi']
    otsus_aceh_total_p         = otsus_aceh['value']['persentase']
    otsus_aceh_header_a        = otsus_aceh['kodeakun'] + "-" + otsus_aceh['text'] + " Anggaran"
    otsus_aceh_header_r         = otsus_aceh['kodeakun'] + "-" + otsus_aceh['text'] + " Realisasi"

    # 48
    otsus_papua                 = datapemda['postur']['1']['child']['6']['child'][1]
    otsus_papua_total_a         = otsus_papua['value']['anggaran']
    otsus_papua_total_r         = otsus_papua['value']['realisasi']
    otsus_papua_total_p         = otsus_papua['value']['persentase']
    otsus_papua_header_a        = otsus_papua['kodeakun'] + "-" + otsus_papua['text'] + " Anggaran"
    otsus_papua_header_r        = otsus_papua['kodeakun'] + "-" + otsus_papua['text'] + " Realisasi"

    # 49
    infrastruktur_papua                 = datapemda['postur']['1']['child']['6']['child'][2]
    infrastruktur_papua_total_a         = infrastruktur_papua['value']['anggaran']
    infrastruktur_papua_total_r         = infrastruktur_papua['value']['realisasi']
    infrastruktur_papua_total_p         = infrastruktur_papua['value']['persentase']
    infrastruktur_papua_header_a        = infrastruktur_papua['kodeakun'] + "-" + infrastruktur_papua['text'] + " Anggaran"
    infrastruktur_papua_header_r        = infrastruktur_papua['kodeakun'] + "-" + infrastruktur_papua['text'] + " Realisasi"

    # 50
    otsus_papuabarat                 = datapemda['postur']['1']['child']['6']['child'][3]
    otsus_papuabarat_total_a         = otsus_papuabarat['value']['anggaran']
    otsus_papuabarat_total_r         = otsus_papuabarat['value']['realisasi']
    otsus_papuabarat_total_p         = otsus_papuabarat['value']['persentase']
    otsus_papuabarat_header_a        = otsus_papuabarat['kodeakun'] + "-" + otsus_papuabarat['text'] + " Anggaran"
    otsus_papuabarat_header_r        = otsus_papuabarat['kodeakun'] + "-" + otsus_papuabarat['text'] + " Realisasi"

    # 51
    infrastruktur_papuabarat                 = datapemda['postur']['1']['child']['6']['child'][4]
    infrastruktur_papuabarat_total_a         = infrastruktur_papuabarat['value']['anggaran']
    infrastruktur_papuabarat_total_r         = infrastruktur_papuabarat['value']['realisasi']
    infrastruktur_papuabarat_total_p         = infrastruktur_papuabarat['value']['persentase']
    infrastruktur_papuabarat_header_a        = infrastruktur_papuabarat['kodeakun'] + "-" + infrastruktur_papuabarat['text'] + " Anggaran"
    infrastruktur_papuabarat_header_r        = infrastruktur_papuabarat['kodeakun'] + "-" + infrastruktur_papuabarat['text'] + " Realisasi"

    # 52 Total Dais
    dais_total_a = datapemda['postur']['1']['child']['7']['total']['anggaran']
    dais_total_r = datapemda['postur']['1']['child']['7']['total']['realisasi']
    dais_total_p = datapemda['postur']['1']['child']['7']['total']['persentase']
    dais_total_header_a = " Total Dais Anggaran"
    dais_total_header_r = " Total Dais Realisasi"

    # 53
    dais_value                 = datapemda['postur']['1']['child']['7']['child'][0]
    dais_value_total_a         = dais_value['value']['anggaran']
    dais_value_total_r         = dais_value['value']['realisasi']
    dais_value_total_p         = dais_value['value']['persentase']
    dais_value_header_a        = dais_value['kodeakun'] + "-" + dais_value['text'] + " Anggaran"
    dais_value_header_r        = dais_value['kodeakun'] + "-" + dais_value['text'] + " Realisasi"

    # 54 Dana Desa
    dd_total_a = datapemda['postur']['1']['total']['anggaran']
    dd_total_r = datapemda['postur']['1']['total']['realisasi']
    dd_total_p = datapemda['postur']['1']['total']['persentase']
    dd_total_header_a = "Total Dana Desa Anggaran"
    dd_total_header_r = "Total Dana Desa Realisasi"

    # 55
    dd_value                 = datapemda['postur']['2']['child']['1']['child'][0]
    dd_value_total_a         = dd_value['value']['anggaran']
    dd_value_total_r         = dd_value['value']['realisasi']
    dd_value_total_p         = dd_value['value']['persentase']
    dd_value_header_a        = dd_value['kodeakun'] + "-" + dd_value['text'] + " Anggaran"
    dd_value_header_r        = dd_value['kodeakun'] + "-" + dd_value['text'] + " Realisasi"


    headers = ["no", 
              "tahun", "wilayah", "provinsi", "disclaimer", 
              tkdd_total_a_header, tkdd_total_r_header, 
              tkd_total_a_header, tkd_total_r_header,
              dbh_total_a_header, dbh_total_r_header, 
              # 4
              dbh_pph21_header_a, dbh_pph21_header_r,
              #5 
              dbh_pph2529opdn_header_a, dbh_pph2529opdn_header_r,
              #6 
              dbh_pbbprovinsi_header_a, dbh_pbbprovinsi_header_r,
              #7
              dbh_pbbkabkot_header_a, dbh_pbbkabkot_header_r,
              #8
              dbh_pbbupahpungutprov_header_a, dbh_pbbupahpungutprov_header_r,
              #9
              dbh_pbbupahpungutkabkot_header_a, dbh_pbbupahpungutkabkot_header_r,
              # 10
              dbh_pbbbagpusatrata_header_a, dbh_pbbbagpusatrata_header_r,
              # 11
              dbh_minyak_header_a, dbh_minyak_header_r,
              # 12
              dbh_minyak_setengah_persen_header_a, dbh_minyak_setengah_persen_header_r,
              #13
              dbh_minyak_otsus_header_a, dbh_minyak_otsus_header_r,
              # 14
              dbh_gasbumi_header_a, dbh_gasbumi_header_r,
              # 15
              dbh_gasbumi_setengah_persen_header_a, dbh_gasbumi_setengah_persen_header_r,
              # 16
              dbh_gasbumi_otsus_header_a, dbh_gasbumi_otsus_header_r,
              # 17
              dbh_pertum_iurantetap_header_a, dbh_pertum_iurantetap_header_r,
              # 18
              dbh_pertum_royalti_header_a, dbh_pertum_royalti_header_r,
              # 19
              dbh_pabum_setoranbagpemerintah_header_a, dbh_pabum_setoranbagpemerintah_header_r, 
              # 20
              dbh_pabum_iurantetap_header_a, dbh_pabum_iurantetap_header_r,
              # 21
              dbh_pabum_royalti_header_a, dbh_pabum_royalti_header_r,
              # 22
              dbh_kehutanan_iiuph_header_a, dbh_kehutanan_iiuph_header_r,
              # 23
              dbh_kehutanan_psdh_header_a, dbh_kehutanan_psdh_header_r,
              # 24
              dbh_kehutanan_danareboisasi_header_a, dbh_kehutanan_danareboisasi_header_r,
              # 25
              dbh_perikanan_header_a, dbh_perikanan_header_r,
              # 26
              dbh_cht_header_a, dbh_cht_header_r,
              #27
              dau_header_a, dau_header_r,
              # 28
              dau_value_header_a, dau_value_header_r,  
              # 29
              dak_fisik_total_header_a, dak_fisik_total_header_r,
              # 30
              dak_fisik_header_a, dak_fisik_header_r,
              # 31
              dak_fisik_infrastruktur_header_a, dak_fisik_infrastruktur_header_r,
              #33
              dak_fisik_penugasan_header_a, dak_fisik_penugasan_header_r,
              #33
              dak_fisik_afirmasi_header_a, dak_fisik_afirmasi_header_r,
              #34
              dak_nonfisik_header_a, dak_nonfisik_header_r,
              #35
              dak_nonfisik_tpg_header_a, dak_nonfisik_tpg_header_r,
              #36
              dak_nonfisik_tkg_dasus_header_a, dak_nonfisik_tkg_dasus_header_r,
              #37
              dak_nonfisik_tamsil_header_a, dak_nonfisik_tamsil_header_r,
              #38
              dak_nonfisik_bos_header_a, dak_nonfisik_bos_header_r,
              #39
              dak_nonfisik_bos_kesehatan_header_a, dak_nonfisik_bos_kesehatan_header_r,
              #40
              dak_nonfisik_bos_kb_header_a, dak_nonfisik_bos_kb_header_r,
              #41
              dak_nonfisik_koperasi_header_a, dak_nonfisik_koperasi_header_r,
              #42
              dak_nonfisik_adminduk_header_a, dak_nonfisik_adminduk_header_r,
              #43
              dak_nonfisik_bos_paud_header_a, dak_nonfisik_bos_paud_header_r,
              #44
              did_total_header_a, did_total_header_r,
              # 45 
              did_value_header_a, did_value_header_r,
              # 46
              otsus_total_header_a, otsus_total_header_r,
              # 47
              otsus_aceh_header_a, otsus_aceh_header_r,
              # 48
              otsus_papua_header_a, otsus_papua_header_r,
              # 49
              infrastruktur_papua_header_a, infrastruktur_papua_header_r,
              #50 
              otsus_papuabarat_header_a, otsus_papuabarat_header_r,
              #51
              infrastruktur_papuabarat_header_a, infrastruktur_papuabarat_header_r,
              #52
              dais_total_header_a, dais_total_header_r,
              # 53
              dais_value_header_a, dais_value_header_r,
              # 54
              dd_total_header_a, dd_total_header_r,
              # 55
              dd_value_header_a, dd_value_header_r
    ]





    isi =[no, 
          tahun, wilayah, provinsi, disclaimer, 
          tkdd_total_a, tkdd_total_r, 
          tkd_total_a, tkd_total_r,
          dbh_total_a, dbh_total_r, 
          dbh_pph21_total_a, dbh_pph21_total_r, 
          dbh_pph2529opdn_total_a,       dbh_pph2529opdn_total_r, 
          dbh_pbbprovinsi_total_a, dbh_pbbprovinsi_total_r,
          dbh_pbbkabkot_total_a, dbh_pbbkabkot_total_r, 
          dbh_pbbupahpungutprov_total_a, dbh_pbbupahpungutprov_total_r, 
          dbh_pbbupahpungutkabkot_total_a, dbh_pbbupahpungutkabkot_total_r,
          dbh_pbbbagpusatrata_total_a, dbh_pbbbagpusatrata_total_r,
          dbh_minyak_total_a, dbh_minyak_total_r, 
          dbh_minyak_setengah_persen_total_a, dbh_minyak_setengah_persen_total_r, 
          dbh_minyak_otsus_total_a, dbh_minyak_otsus_total_r,
          dbh_gasbumi_total_a, dbh_gasbumi_total_r, 
          dbh_gasbumi_setengah_persen_total_a, dbh_gasbumi_setengah_persen_total_r,
          dbh_gasbumi_otsus_total_a, dbh_gasbumi_otsus_total_r, 
          dbh_pertum_iurantetap_total_a, dbh_pertum_iurantetap_total_r,
          dbh_pertum_royalti_total_a, dbh_pertum_royalti_total_r, 
          dbh_pabum_setoranbagpemerintah_total_a, dbh_pabum_setoranbagpemerintah_total_r, 
          dbh_pabum_iurantetap_total_a, dbh_pabum_iurantetap_total_r,
          dbh_pabum_royalti_total_a, dbh_pabum_royalti_total_r,
          dbh_kehutanan_iiuph_total_a, dbh_kehutanan_iiuph_total_r,
          dbh_kehutanan_psdh_total_a, dbh_kehutanan_psdh_total_r, 
          dbh_kehutanan_danareboisasi_total_a, dbh_kehutanan_danareboisasi_total_r, 
          dbh_perikanan_total_a, dbh_perikanan_total_r,
          dbh_cht_total_a, dbh_cht_total_r,
          dau_total_a, dau_total_r, 
          dau_total_a, dau_total_r,
          dak_fisik_total_a, dak_fisik_total_r,
          dak_fisik_total_a, dak_fisik_total_r,
          dak_fisik_infrastruktur_total_a, dak_fisik_infrastruktur_total_r,
          dak_fisik_penugasan_total_a, dak_fisik_penugasan_total_r, 
          dak_fisik_afirmasi_total_a, dak_fisik_afirmasi_total_r,
          dak_nonfisik_total_a, dak_nonfisik_total_r, 
          dak_nonfisik_tpg_total_a, dak_nonfisik_tpg_total_r,
          dak_nonfisik_tkg_dasus_total_a, dak_nonfisik_tkg_dasus_total_r,
          dak_nonfisik_tamsil_total_a, dak_nonfisik_tamsil_total_r,
          dak_nonfisik_bos_total_a, dak_nonfisik_bos_total_r,
          dak_nonfisik_bos_kesehatan_total_a, dak_nonfisik_bos_kesehatan_total_r,
          dak_nonfisik_bos_kb_total_a, dak_nonfisik_bos_kb_total_r,
          dak_nonfisik_koperasi_total_a, dak_nonfisik_koperasi_total_r,
          dak_nonfisik_adminduk_total_a, dak_nonfisik_adminduk_total_r,
          dak_nonfisik_bos_paud_total_a, dak_nonfisik_bos_paud_total_r, 
          did_total_a, did_total_r, 
          did_value_total_a, did_value_total_r, 
          otsus_total_a, otsus_total_r,
          otsus_aceh_total_a, otsus_aceh_total_r, 
          otsus_papua_total_a, otsus_papua_total_r, 
          infrastruktur_papua_total_a, infrastruktur_papua_total_r,
          otsus_papuabarat_total_a, otsus_papuabarat_total_r,
          infrastruktur_papuabarat_total_a, infrastruktur_papuabarat_total_r, 
          dais_total_a, dais_total_r, 
          dais_value_total_a, dais_value_total_r,
          #DanaDesa, 
          dd_total_a, dd_total_r, dd_value_total_a, dd_value_total_r]

    return [headers, isi]
    
if __name__ == "__main__":
    set_global_session()
    data = []
    allprovinsi = []
    no = 1
    # tahun yg ada datanya hanya 2018-2019
    token, allprovinsi = get_alldata()
    # loop tahun
    for tahun in range(2018, 2021):
        if tahun == '2000':
            continue
        # loop provinsi
        for provinsi in allprovinsi:
            pemdas = get_pemdas(provinsi)
        
            # loop pemda per provinsi
            for pemda in pemdas.keys():
                print(get_data_tkdd(token, provinsi, pemda))
                datapemda = json.loads(get_data_tkdd(token, provinsi, pemda))
                #print(datapemda)
                if len(datapemda) == 0:
                     continue
                
                # create headers only in 1st loop
                if no == 1:
                    headers = mapping(datapemda, allprovinsi[provinsi])
                                
                # isi data
                isi = mapping(datapemda, allprovinsi[provinsi])
                data.append(isi[1])
                no = no + 1
                

    # save via DataFrame
    df = pd.DataFrame(data, headers=headers[0])
    df.to_csv( path+filename , index=False, encoding="utf-8")                

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transf

IndexError: ignored

# Enumerate Dict

In [ ]:
import json
datapemda = json.loads('{"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Kabupaten\/kota","value":{"anggaran":398764072000,"realisasi":390261862177,"persentase":"97,87 %"}},{"kodeakun":"611215","text":"Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten\/Kota","value":{"anggaran":1736910100000,"realisasi":1650781100705,"persentase":"95,04 %"}},{"kodeakun":"612111","text":"Transfer DBH Minyak Bumi","value":{"anggaran":8581982674000,"realisasi":12376493484658,"persentase":"144,21 %"}},{"kodeakun":"612112","text":"Transfer DBH Minyak Bumi 0.5%","value":{"anggaran":282732216000,"realisasi":407714843494,"persentase":"144,21 %"}},{"kodeakun":"612113","text":"Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus","value":{"anggaran":370605160000,"realisasi":481008154249,"persentase":"129,79 %"}},{"kodeakun":"612211","text":"Transfer DBH Gas Bumi","value":{"anggaran":5966265026000,"realisasi":8745074752035,"persentase":"146,58 %"}},{"kodeakun":"612212","text":"Transfer DBH Gas Bumi 0.5%","value":{"anggaran":96882460000,"realisasi":124683590580,"persentase":"128,70 %"}},{"kodeakun":"612213","text":"Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus","value":{"anggaran":270583164000,"realisasi":340653879134,"persentase":"125,90 %"}},{"kodeakun":"612311","text":"Transfer DBH Pertambangan Umum - Iuran Tetap","value":{"anggaran":615858746000,"realisasi":574574040835,"persentase":"93,30 %"}},{"kodeakun":"612312","text":"Transfer DBH Pertambangan Umum - Royalti","value":{"anggaran":13670958915000,"realisasi":19233330257405,"persentase":"140,69 %"}},{"kodeakun":"612411","text":"Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah","value":{"anggaran":545092142000,"realisasi":814945712335,"persentase":"149,51 %"}},{"kodeakun":"612412","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Tetap","value":{"anggaran":14680315000,"realisasi":17093550748,"persentase":"116,44 %"}},{"kodeakun":"612413","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)","value":{"anggaran":703068000,"realisasi":897327311,"persentase":"127,63 %"}},{"kodeakun":"612511","text":"Transfer DBH Kehutanan - IIUPH\/IHPH","value":{"anggaran":164730247000,"realisasi":165979915021,"persentase":"100,76 %"}},{"kodeakun":"612512","text":"Transfer DBH Kehutanan - PSDH","value":{"anggaran":736821430000,"realisasi":706696713825,"persentase":"95,91 %"}},{"kodeakun":"612513","text":"Transfer DBH Kehutanan - Dana Reboisasi","value":{"anggaran":743479609000,"realisasi":741420178157,"persentase":"99,72 %"}},{"kodeakun":"612611","text":"Transfer DBH Perikanan","value":{"anggaran":480000648000,"realisasi":357622699445,"persentase":"74,50 %"}},{"kodeakun":"613111","text":"Transfer DBH Cukai Hasil Tembakau","value":{"anggaran":2964600000000,"realisasi":2922191165245,"persentase":"98,57 %"}}],"total":{"anggaran":89225342014000,"realisasi":93705559169270,"persentase":"105,02 %"}},"2":{"text":"DAU","child":[{"kodeakun":"621111","text":"Dana Alokasi Umum","value":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}}],"total":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}},"3":{"text":"DAK FISIK","child":[{"kodeakun":"631111","text":"Dana Alokasi Khusus","value":{"anggaran":31350835954000,"realisasi":29459738989382,"persentase":"93,97 %"}},{"kodeakun":"631211","text":"Dana Alokasi Khusus Infrastruktur Publik Daerah","value":{"anggaran":0,"realisasi":null,"persentase":"-"}},{"kodeakun":"631212","text":"Dana Alokasi Khusus Penugasan","value":{"anggaran":24463658880000,"realisasi":22492927163424,"persentase":"91,94 %"}},{"kodeakun":"631311","text":"Dana Alokasi Khusus Affirmasi","value":{"anggaran":6621768074000,"realisasi":6259405435103,"persentase":"94,53 %"}}],"total":{"anggaran":62436262908000,"realisasi":58212071587909,"persentase":"93,23 %"}},"6":{"text":"OTSUS","child":[{"kodeakun":"641111","text":"Dana Otonomi Khusus Provinsi Aceh","value":{"anggaran":8029791593000,"realisasi":8029791593000,"persentase":"100,00 %"}},{"kodeakun":"641211","text":"Dana Otonomi Khusus Provinsi Papua","value":{"anggaran":5620854115000,"realisasi":5620854115000,"persentase":"100,00 %"}},{"kodeakun":"641212","text":"Dana Tambahan Infrastruktur Provinsi Papua","value":{"anggaran":2400000000000,"realisasi":2400000000000,"persentase":"100,00 %"}},{"kodeakun":"641311","text":"Dana Otonomi Khusus Provinsi Papua Barat","value":{"anggaran":2408937478000,"realisasi":2408937478000,"persentase":"100,00 %"}},{"kodeakun":"641312","text":"Dana Tambahan Infrastruktur Provinsi Papua Barat","value":{"anggaran":1600000000000,"realisasi":1600000000000,"persentase":"100,00 %"}}],"total":{"anggaran":20059583186000,"realisasi":20059583186000,"persentase":"100,00 %"}},"7":{"text":"KEISTIMEWAAN DIY","child":[{"kodeakun":"642111","text":"Dana Keistimewaan Daerah Istimewa Yogyakarta","value":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}}],"total":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}},"5":{"text":"DID","child":[{"kodeakun":"643111","text":"Dana Insentif Daerah","value":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}}],"total":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}},"4":{"text":"DAK NON FISIK","child":[{"kodeakun":"654111","text":"Dana Tunjangan Profesi Guru PNSD","value":{"anggaran":56861747531798,"realisasi":55016720212332,"persentase":"96,76 %"}},{"kodeakun":"654112","text":"Dana Tunjangan Khusus Guru PNSD di Daerah Khusus","value":{"anggaran":1922303294676,"realisasi":1727102404861,"persentase":"89,85 %"}},{"kodeakun":"654211","text":"Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil","value":{"anggaran":879468864366,"realisasi":588007130574,"persentase":"66,86 %"}},{"kodeakun":"654311","text":"Dana Bantuan Operasional Sekolah","value":{"anggaran":46311243600000,"realisasi":44367351931006,"persentase":"95,80 %"}},{"kodeakun":"654711","text":"Dana Bantuan Operasional Kesehatan","value":{"anggaran":8551232700000,"realisasi":7454358890915,"persentase":"87,17 %"}},{"kodeakun":"654712","text":"Dana Bantuan Operasional Keluarga Berencana","value":{"anggaran":1808787300000,"realisasi":1753419417979,"persentase":"96,94 %"}},{"kodeakun":"654811","text":"Dana Peningkatan Kapasitas Koperasi","value":{"anggaran":100000000000,"realisasi":92221616897,"persentase":"92,22 %"}},{"kodeakun":"654814","text":"Dana Pelayanan Administrasi Kependudukan","value":{"anggaran":825000000000,"realisasi":745193742984,"persentase":"90,33 %"}},{"kodeakun":"654911","text":"Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini","value":{"anggaran":3741708600000,"realisasi":3555904601891,"persentase":"95,03 %"}}],"total":{"anggaran":121001491890840,"realisasi":115300279949439,"persentase":"95,29 %"}}},"total":{"anggaran":703712259647840,"realisasi":697657198541618,"persentase":"99,14 %"}},"2":{"text":"Dana Desa","child":{"1":{"text":"DANA DESA","child":[{"kodeakun":"662111","text":"Dana Desa","value":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}],"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"wilayah":"Nasional","judul":"POSTUR TKDD TA 2018","link_excel":"http:\/\/www.djpk.kemenkeu.go.id\/portal\/data\/csv\/2018\/--\/--\/tkdd","disclaimer":"Data bulan desember","header":["Alokasi","Realisasi"],"special_row":{"text":"Transfer ke daerah dan Dana Desa","total":{"anggaran":763712259647840,"realisasi":757533338915843,"persentase":"99,19 %"}}}')

#for i, j in enumerate(datapemda['postur']['1']):
#    print(i, j)

#dir(datapemda)
#print(datapemda.items())

#for k, v in {**datapemda}.items():
#    print(k, '->', v)

#for i, (k, v) in enumerate(datapemda.items()):
#    if isinstance(v, dict):
#        print("Lanjut")

#    print(i, k, v)

def beta(jsondata):
    for i, (k, v) in enumerate(jsondata.items()):
        print(i, k, v)
        if isinstance(v, dict):
            beta(v)
        return True

beta(datapemda)

0 postur {'1': {'text': 'Transfer ke Daerah', 'child': {'1': {'text': 'DBH', 'child': [{'kodeakun': '611111', 'text': 'Transfer DBH PPh Pasal 21', 'value': {'anggaran': 32986586000000, 'realisasi': 27458081283272, 'persentase': '83,24 %'}}, {'kodeakun': '611112', 'text': 'Transfer DBH PPh Pasal 25/29 OP', 'value': {'anggaran': 4441882000000, 'realisasi': 2533606329484, 'persentase': '57,04 %'}}, {'kodeakun': '611211', 'text': 'Transfer DBH PBB untuk Propinsi', 'value': {'anggaran': 2857407829000, 'realisasi': 2737579489820, 'persentase': '95,81 %'}}, {'kodeakun': '611212', 'text': 'Transfer DBH PBB untuk Kabupaten/Kota', 'value': {'anggaran': 11211563981000, 'realisasi': 10840669654158, 'persentase': '96,69 %'}}, {'kodeakun': '611213', 'text': 'Transfer DBH Biaya/Upah Pungut PBB untuk Propinsi', 'value': {'anggaran': 86252212000, 'realisasi': 84199185177, 'persentase': '97,62 %'}}, {'kodeakun': '611214', 'text': 'Transfer DBH Biaya/Upah Pungut PBB untuk Kabupaten/kota', 'value': {'angg

True

# Compare Kodeakun tiap tahun

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import urllib3
import json

session = None
# Snippet Save to Google Drive
from datetime import datetime
from pytz import timezone
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

def set_global_session():
    global session
    if not session:
        session = requests.Session()
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        session.verify = False

def get_alldata():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            token = bsoup.find("input", {"name":"_token"})['value']
            options_provinsi = bsoup.find("select", {"id":"sel_prov"})
            allprovinsi = {}
            for option_provinsi in options_provinsi.find_all("option"):
                allprovinsi[option_provinsi['value']]=option_provinsi.text 
            return [token, allprovinsi]

def get_token():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            token = bsoup.find("input", {"name":"_token"})['value']
            return token

def get_provinsi():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            options_provinsi = bsoup.find("select", {"id":"sel_prov"})
            allprovinsi = []
            for option_provinsi in options_provinsi.find_all("option"):
                allprovinsi[option_provinsi['value']]=option_provinsi['value'].text
            return allprovinsi

def get_pemdas(provinsi):
    with session.get('http://www.djpk.kemenkeu.go.id/portal/pemda/' + provinsi) as response:
        if response.status_code==200 and len(response.text) != 0:
            pemdas = json.loads(response.text)
            return pemdas

def get_headers(token, tahun=2018):
    with session.post('http://www.djpk.kemenkeu.go.id/portal/filter', 
                data={'_token': token, 'data': 'tkdd', 'tahun': tahun, 'provinsi': '--', 'pemda': '--'}) as response:# query nasional saja as response
        if response.status_code==200 and len(response.text) != 0:
            print(response.text)
            # olah gaes
            # {"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Kabupaten\/kota","value":{"anggaran":398764072000,"realisasi":390261862177,"persentase":"97,87 %"}},{"kodeakun":"611215","text":"Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten\/Kota","value":{"anggaran":1736910100000,"realisasi":1650781100705,"persentase":"95,04 %"}},{"kodeakun":"612111","text":"Transfer DBH Minyak Bumi","value":{"anggaran":8581982674000,"realisasi":12376493484658,"persentase":"144,21 %"}},{"kodeakun":"612112","text":"Transfer DBH Minyak Bumi 0.5%","value":{"anggaran":282732216000,"realisasi":407714843494,"persentase":"144,21 %"}},{"kodeakun":"612113","text":"Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus","value":{"anggaran":370605160000,"realisasi":481008154249,"persentase":"129,79 %"}},{"kodeakun":"612211","text":"Transfer DBH Gas Bumi","value":{"anggaran":5966265026000,"realisasi":8745074752035,"persentase":"146,58 %"}},{"kodeakun":"612212","text":"Transfer DBH Gas Bumi 0.5%","value":{"anggaran":96882460000,"realisasi":124683590580,"persentase":"128,70 %"}},{"kodeakun":"612213","text":"Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus","value":{"anggaran":270583164000,"realisasi":340653879134,"persentase":"125,90 %"}},{"kodeakun":"612311","text":"Transfer DBH Pertambangan Umum - Iuran Tetap","value":{"anggaran":615858746000,"realisasi":574574040835,"persentase":"93,30 %"}},{"kodeakun":"612312","text":"Transfer DBH Pertambangan Umum - Royalti","value":{"anggaran":13670958915000,"realisasi":19233330257405,"persentase":"140,69 %"}},{"kodeakun":"612411","text":"Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah","value":{"anggaran":545092142000,"realisasi":814945712335,"persentase":"149,51 %"}},{"kodeakun":"612412","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Tetap","value":{"anggaran":14680315000,"realisasi":17093550748,"persentase":"116,44 %"}},{"kodeakun":"612413","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)","value":{"anggaran":703068000,"realisasi":897327311,"persentase":"127,63 %"}},{"kodeakun":"612511","text":"Transfer DBH Kehutanan - IIUPH\/IHPH","value":{"anggaran":164730247000,"realisasi":165979915021,"persentase":"100,76 %"}},{"kodeakun":"612512","text":"Transfer DBH Kehutanan - PSDH","value":{"anggaran":736821430000,"realisasi":706696713825,"persentase":"95,91 %"}},{"kodeakun":"612513","text":"Transfer DBH Kehutanan - Dana Reboisasi","value":{"anggaran":743479609000,"realisasi":741420178157,"persentase":"99,72 %"}},{"kodeakun":"612611","text":"Transfer DBH Perikanan","value":{"anggaran":480000648000,"realisasi":357622699445,"persentase":"74,50 %"}},{"kodeakun":"613111","text":"Transfer DBH Cukai Hasil Tembakau","value":{"anggaran":2964600000000,"realisasi":2922191165245,"persentase":"98,57 %"}}],"total":{"anggaran":89225342014000,"realisasi":93705559169270,"persentase":"105,02 %"}},"2":{"text":"DAU","child":[{"kodeakun":"621111","text":"Dana Alokasi Umum","value":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}}],"total":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}},"3":{"text":"DAK FISIK","child":[{"kodeakun":"631111","text":"Dana Alokasi Khusus","value":{"anggaran":31350835954000,"realisasi":29459738989382,"persentase":"93,97 %"}},{"kodeakun":"631211","text":"Dana Alokasi Khusus Infrastruktur Publik Daerah","value":{"anggaran":0,"realisasi":null,"persentase":"-"}},{"kodeakun":"631212","text":"Dana Alokasi Khusus Penugasan","value":{"anggaran":24463658880000,"realisasi":22492927163424,"persentase":"91,94 %"}},{"kodeakun":"631311","text":"Dana Alokasi Khusus Affirmasi","value":{"anggaran":6621768074000,"realisasi":6259405435103,"persentase":"94,53 %"}}],"total":{"anggaran":62436262908000,"realisasi":58212071587909,"persentase":"93,23 %"}},"6":{"text":"OTSUS","child":[{"kodeakun":"641111","text":"Dana Otonomi Khusus Provinsi Aceh","value":{"anggaran":8029791593000,"realisasi":8029791593000,"persentase":"100,00 %"}},{"kodeakun":"641211","text":"Dana Otonomi Khusus Provinsi Papua","value":{"anggaran":5620854115000,"realisasi":5620854115000,"persentase":"100,00 %"}},{"kodeakun":"641212","text":"Dana Tambahan Infrastruktur Provinsi Papua","value":{"anggaran":2400000000000,"realisasi":2400000000000,"persentase":"100,00 %"}},{"kodeakun":"641311","text":"Dana Otonomi Khusus Provinsi Papua Barat","value":{"anggaran":2408937478000,"realisasi":2408937478000,"persentase":"100,00 %"}},{"kodeakun":"641312","text":"Dana Tambahan Infrastruktur Provinsi Papua Barat","value":{"anggaran":1600000000000,"realisasi":1600000000000,"persentase":"100,00 %"}}],"total":{"anggaran":20059583186000,"realisasi":20059583186000,"persentase":"100,00 %"}},"7":{"text":"KEISTIMEWAAN DIY","child":[{"kodeakun":"642111","text":"Dana Keistimewaan Daerah Istimewa Yogyakarta","value":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}}],"total":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}},"5":{"text":"DID","child":[{"kodeakun":"643111","text":"Dana Insentif Daerah","value":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}}],"total":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}},"4":{"text":"DAK NON FISIK","child":[{"kodeakun":"654111","text":"Dana Tunjangan Profesi Guru PNSD","value":{"anggaran":56861747531798,"realisasi":55016720212332,"persentase":"96,76 %"}},{"kodeakun":"654112","text":"Dana Tunjangan Khusus Guru PNSD di Daerah Khusus","value":{"anggaran":1922303294676,"realisasi":1727102404861,"persentase":"89,85 %"}},{"kodeakun":"654211","text":"Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil","value":{"anggaran":879468864366,"realisasi":588007130574,"persentase":"66,86 %"}},{"kodeakun":"654311","text":"Dana Bantuan Operasional Sekolah","value":{"anggaran":46311243600000,"realisasi":44367351931006,"persentase":"95,80 %"}},{"kodeakun":"654711","text":"Dana Bantuan Operasional Kesehatan","value":{"anggaran":8551232700000,"realisasi":7454358890915,"persentase":"87,17 %"}},{"kodeakun":"654712","text":"Dana Bantuan Operasional Keluarga Berencana","value":{"anggaran":1808787300000,"realisasi":1753419417979,"persentase":"96,94 %"}},{"kodeakun":"654811","text":"Dana Peningkatan Kapasitas Koperasi","value":{"anggaran":100000000000,"realisasi":92221616897,"persentase":"92,22 %"}},{"kodeakun":"654814","text":"Dana Pelayanan Administrasi Kependudukan","value":{"anggaran":825000000000,"realisasi":745193742984,"persentase":"90,33 %"}},{"kodeakun":"654911","text":"Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini","value":{"anggaran":3741708600000,"realisasi":3555904601891,"persentase":"95,03 %"}}],"total":{"anggaran":121001491890840,"realisasi":115300279949439,"persentase":"95,29 %"}}},"total":{"anggaran":703712259647840,"realisasi":697657198541618,"persentase":"99,14 %"}},"2":{"text":"Dana Desa","child":{"1":{"text":"DANA DESA","child":[{"kodeakun":"662111","text":"Dana Desa","value":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}],"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"wilayah":"Nasional","judul":"POSTUR TKDD TA 2018","link_excel":"http:\/\/www.djpk.kemenkeu.go.id\/portal\/data\/csv\/2018\/--\/--\/tkdd","disclaimer":"Data bulan desember","header":["Alokasi","Realisasi"],"special_row":{"text":"Transfer ke daerah dan Dana Desa","total":{"anggaran":763712259647840,"realisasi":757533338915843,"persentase":"99,19 %"}}}

            kodeakun = 0

            return kodeakun

def get_data_tkdd(token, provinsi, pemda, tahun=2018):
    try:
        with session.post('http://www.djpk.kemenkeu.go.id/portal/filter', 
                    data={'_token': token, 'data': 'tkdd', 'tahun': tahun, 'provinsi': provinsi, 'pemda': pemda}) as response:
            if response.status_code==200 and len(response.text) != 0:
                return response.text
    except requests.exceptions.ConnectionError as e:
        return False
        pass
    except Exception as e:
        logger.error(e)
        randomtime = random.randint(1,5)
        logger.warn('ERROR - Retrying again website %s, retrying in %d secs' % (url, randomtime))
        time.sleep(randomtime)
        pass
        return False

def save_to_gdrive():
    path = F"/content/drive/My Drive/Colab Notebooks/Portal APBD/"
    filename = "data-tkdd-"+ datetime.now(timezone('Asia/Jakarta')).strftime("%Y-%m-%d--%H-%M") + ".csv"
    return True

if __name__ == "__main__":
    set_global_session()
    data = []
    allprovinsi = []
    # tahun yg ada datanya hanya 2018-2019
    token, allprovinsi = get_alldata()
    # get headers
    print("Headers 2018:")
    print(get_headers(token, tahun=2018))
    print("Headers 2019:")
    print(get_headers(token, tahun=2019))
    print("Headers 2021:")
    print(get_headers(token, tahun=2021))
    
                

    # save via DataFrame
    #df = pd.DataFrame(data)
    #df.to_csv( path+filename , index=False, encoding="utf-8")                

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Headers 2018:
{"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214",

# Regex every kodeakun

In [ ]:
import re
datapemda = '{"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Kabupaten\/kota","value":{"anggaran":398764072000,"realisasi":390261862177,"persentase":"97,87 %"}},{"kodeakun":"611215","text":"Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten\/Kota","value":{"anggaran":1736910100000,"realisasi":1650781100705,"persentase":"95,04 %"}},{"kodeakun":"612111","text":"Transfer DBH Minyak Bumi","value":{"anggaran":8581982674000,"realisasi":12376493484658,"persentase":"144,21 %"}},{"kodeakun":"612112","text":"Transfer DBH Minyak Bumi 0.5%","value":{"anggaran":282732216000,"realisasi":407714843494,"persentase":"144,21 %"}},{"kodeakun":"612113","text":"Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus","value":{"anggaran":370605160000,"realisasi":481008154249,"persentase":"129,79 %"}},{"kodeakun":"612211","text":"Transfer DBH Gas Bumi","value":{"anggaran":5966265026000,"realisasi":8745074752035,"persentase":"146,58 %"}},{"kodeakun":"612212","text":"Transfer DBH Gas Bumi 0.5%","value":{"anggaran":96882460000,"realisasi":124683590580,"persentase":"128,70 %"}},{"kodeakun":"612213","text":"Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus","value":{"anggaran":270583164000,"realisasi":340653879134,"persentase":"125,90 %"}},{"kodeakun":"612311","text":"Transfer DBH Pertambangan Umum - Iuran Tetap","value":{"anggaran":615858746000,"realisasi":574574040835,"persentase":"93,30 %"}},{"kodeakun":"612312","text":"Transfer DBH Pertambangan Umum - Royalti","value":{"anggaran":13670958915000,"realisasi":19233330257405,"persentase":"140,69 %"}},{"kodeakun":"612411","text":"Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah","value":{"anggaran":545092142000,"realisasi":814945712335,"persentase":"149,51 %"}},{"kodeakun":"612412","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Tetap","value":{"anggaran":14680315000,"realisasi":17093550748,"persentase":"116,44 %"}},{"kodeakun":"612413","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)","value":{"anggaran":703068000,"realisasi":897327311,"persentase":"127,63 %"}},{"kodeakun":"612511","text":"Transfer DBH Kehutanan - IIUPH\/IHPH","value":{"anggaran":164730247000,"realisasi":165979915021,"persentase":"100,76 %"}},{"kodeakun":"612512","text":"Transfer DBH Kehutanan - PSDH","value":{"anggaran":736821430000,"realisasi":706696713825,"persentase":"95,91 %"}},{"kodeakun":"612513","text":"Transfer DBH Kehutanan - Dana Reboisasi","value":{"anggaran":743479609000,"realisasi":741420178157,"persentase":"99,72 %"}},{"kodeakun":"612611","text":"Transfer DBH Perikanan","value":{"anggaran":480000648000,"realisasi":357622699445,"persentase":"74,50 %"}},{"kodeakun":"613111","text":"Transfer DBH Cukai Hasil Tembakau","value":{"anggaran":2964600000000,"realisasi":2922191165245,"persentase":"98,57 %"}}],"total":{"anggaran":89225342014000,"realisasi":93705559169270,"persentase":"105,02 %"}},"2":{"text":"DAU","child":[{"kodeakun":"621111","text":"Dana Alokasi Umum","value":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}}],"total":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}},"3":{"text":"DAK FISIK","child":[{"kodeakun":"631111","text":"Dana Alokasi Khusus","value":{"anggaran":31350835954000,"realisasi":29459738989382,"persentase":"93,97 %"}},{"kodeakun":"631211","text":"Dana Alokasi Khusus Infrastruktur Publik Daerah","value":{"anggaran":0,"realisasi":null,"persentase":"-"}},{"kodeakun":"631212","text":"Dana Alokasi Khusus Penugasan","value":{"anggaran":24463658880000,"realisasi":22492927163424,"persentase":"91,94 %"}},{"kodeakun":"631311","text":"Dana Alokasi Khusus Affirmasi","value":{"anggaran":6621768074000,"realisasi":6259405435103,"persentase":"94,53 %"}}],"total":{"anggaran":62436262908000,"realisasi":58212071587909,"persentase":"93,23 %"}},"6":{"text":"OTSUS","child":[{"kodeakun":"641111","text":"Dana Otonomi Khusus Provinsi Aceh","value":{"anggaran":8029791593000,"realisasi":8029791593000,"persentase":"100,00 %"}},{"kodeakun":"641211","text":"Dana Otonomi Khusus Provinsi Papua","value":{"anggaran":5620854115000,"realisasi":5620854115000,"persentase":"100,00 %"}},{"kodeakun":"641212","text":"Dana Tambahan Infrastruktur Provinsi Papua","value":{"anggaran":2400000000000,"realisasi":2400000000000,"persentase":"100,00 %"}},{"kodeakun":"641311","text":"Dana Otonomi Khusus Provinsi Papua Barat","value":{"anggaran":2408937478000,"realisasi":2408937478000,"persentase":"100,00 %"}},{"kodeakun":"641312","text":"Dana Tambahan Infrastruktur Provinsi Papua Barat","value":{"anggaran":1600000000000,"realisasi":1600000000000,"persentase":"100,00 %"}}],"total":{"anggaran":20059583186000,"realisasi":20059583186000,"persentase":"100,00 %"}},"7":{"text":"KEISTIMEWAAN DIY","child":[{"kodeakun":"642111","text":"Dana Keistimewaan Daerah Istimewa Yogyakarta","value":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}}],"total":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}},"5":{"text":"DID","child":[{"kodeakun":"643111","text":"Dana Insentif Daerah","value":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}}],"total":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}},"4":{"text":"DAK NON FISIK","child":[{"kodeakun":"654111","text":"Dana Tunjangan Profesi Guru PNSD","value":{"anggaran":56861747531798,"realisasi":55016720212332,"persentase":"96,76 %"}},{"kodeakun":"654112","text":"Dana Tunjangan Khusus Guru PNSD di Daerah Khusus","value":{"anggaran":1922303294676,"realisasi":1727102404861,"persentase":"89,85 %"}},{"kodeakun":"654211","text":"Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil","value":{"anggaran":879468864366,"realisasi":588007130574,"persentase":"66,86 %"}},{"kodeakun":"654311","text":"Dana Bantuan Operasional Sekolah","value":{"anggaran":46311243600000,"realisasi":44367351931006,"persentase":"95,80 %"}},{"kodeakun":"654711","text":"Dana Bantuan Operasional Kesehatan","value":{"anggaran":8551232700000,"realisasi":7454358890915,"persentase":"87,17 %"}},{"kodeakun":"654712","text":"Dana Bantuan Operasional Keluarga Berencana","value":{"anggaran":1808787300000,"realisasi":1753419417979,"persentase":"96,94 %"}},{"kodeakun":"654811","text":"Dana Peningkatan Kapasitas Koperasi","value":{"anggaran":100000000000,"realisasi":92221616897,"persentase":"92,22 %"}},{"kodeakun":"654814","text":"Dana Pelayanan Administrasi Kependudukan","value":{"anggaran":825000000000,"realisasi":745193742984,"persentase":"90,33 %"}},{"kodeakun":"654911","text":"Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini","value":{"anggaran":3741708600000,"realisasi":3555904601891,"persentase":"95,03 %"}}],"total":{"anggaran":121001491890840,"realisasi":115300279949439,"persentase":"95,29 %"}}},"total":{"anggaran":703712259647840,"realisasi":697657198541618,"persentase":"99,14 %"}},"2":{"text":"Dana Desa","child":{"1":{"text":"DANA DESA","child":[{"kodeakun":"662111","text":"Dana Desa","value":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}],"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"wilayah":"Nasional","judul":"POSTUR TKDD TA 2018","link_excel":"http:\/\/www.djpk.kemenkeu.go.id\/portal\/data\/csv\/2018\/--\/--\/tkdd","disclaimer":"Data bulan desember","header":["Alokasi","Realisasi"],"special_row":{"text":"Transfer ke daerah dan Dana Desa","total":{"anggaran":763712259647840,"realisasi":757533338915843,"persentase":"99,19 %"}}}'

kodeakun = "{"611111" : "Transfer DBH PPh Pasal 21",  "611112" : "Transfer DBH PPh Pasal 25/29 OP",  "611211" : "Transfer DBH PBB untuk Propinsi",  "611212" : "Transfer DBH PBB untuk Kabupaten/Kota",  "611213" : "Transfer DBH Biaya/Upah Pungut PBB untuk Propinsi",  "611214" : "Transfer DBH Biaya/Upah Pungut PBB untuk Kabupaten/kota",  "611215" : "Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten/Kota",  "612111" : "Transfer DBH Minyak Bumi",  "612112" : "Transfer DBH Minyak Bumi 0.5%",  "612113" : "Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus",  "612211" : "Transfer DBH Gas Bumi",  "612212" : "Transfer DBH Gas Bumi 0.5%",  "612213" : "Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus",  "612311" : "Transfer DBH Pertambangan Umum - Iuran Tetap",  "612312" : "Transfer DBH Pertambangan Umum - Royalti",  "612411" : "Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah",  "612412" : "Transfer DBH Pertambangan Panas Bumi - Iuran Tetap",  "612413" : "Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)",  "612511" : "Transfer DBH Kehutanan - IIUPH/IHPH",  "612512" : "Transfer DBH Kehutanan - PSDH",  "612513" : "Transfer DBH Kehutanan - Dana Reboisasi",  "612611" : "Transfer DBH Perikanan",  "613111" : "Transfer DBH Cukai Hasil Tembakau",  "621111" : "Dana Alokasi Umum",  "631111" : "Dana Alokasi Khusus",  "631211" : "Dana Alokasi Khusus Infrastruktur Publik Daerah",  "631212" : "Dana Alokasi Khusus Penugasan",  "631311" : "Dana Alokasi Khusus Affirmasi",  "641111" : "Dana Otonomi Khusus Provinsi Aceh",  "641211" : "Dana Otonomi Khusus Provinsi Papua",  "641212" : "Dana Tambahan Infrastruktur Provinsi Papua",  "641311" : "Dana Otonomi Khusus Provinsi Papua Barat",  "641312" : "Dana Tambahan Infrastruktur Provinsi Papua Barat",  "642111" : "Dana Keistimewaan Daerah Istimewa Yogyakarta",  "643111" : "Dana Insentif Daerah",  "654111" : "Dana Tunjangan Profesi Guru PNSD",  "654112" : "Dana Tunjangan Khusus Guru PNSD di Daerah Khusus",  "654211" : "Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil",  "654311" : "Dana Bantuan Operasional Sekolah",  "654711" : "Dana Bantuan Operasional Kesehatan",  "654712" : "Dana Bantuan Operasional Keluarga Berencana",  "654811" : "Dana Peningkatan Kapasitas Koperasi",  "654814" : "Dana Pelayanan Administrasi Kependudukan",  "654911" : "Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini",  "654721" : "Dana Bantuan Biaya Layanan Pengolahan Sampah (BLPS)",  "654821" : "Dana Pelayanan Kepariwisataan",  "654912" : "Dana Bantuan Operasional Penyelenggaraan (BOP) Pendidikan Kesetaraan",  "654921" : "Dana Bantuan Operasioanal Penyelenggaraan (BOP) Museum dan Taman Budaya",  "655211" : "Dana Pelayanan Perlindungan Perempuan dan Anak",  "655311" : "Dana Fasilitasi Penanaman Modal",  "655411" : "Dana Ketahanan Pangan dan Pertanian"}"
print(kodeakun)

SyntaxError: ignored

# Flatten json

In [ ]:
import json
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

datapemda = json.loads('{"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Kabupaten\/kota","value":{"anggaran":398764072000,"realisasi":390261862177,"persentase":"97,87 %"}},{"kodeakun":"611215","text":"Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten\/Kota","value":{"anggaran":1736910100000,"realisasi":1650781100705,"persentase":"95,04 %"}},{"kodeakun":"612111","text":"Transfer DBH Minyak Bumi","value":{"anggaran":8581982674000,"realisasi":12376493484658,"persentase":"144,21 %"}},{"kodeakun":"612112","text":"Transfer DBH Minyak Bumi 0.5%","value":{"anggaran":282732216000,"realisasi":407714843494,"persentase":"144,21 %"}},{"kodeakun":"612113","text":"Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus","value":{"anggaran":370605160000,"realisasi":481008154249,"persentase":"129,79 %"}},{"kodeakun":"612211","text":"Transfer DBH Gas Bumi","value":{"anggaran":5966265026000,"realisasi":8745074752035,"persentase":"146,58 %"}},{"kodeakun":"612212","text":"Transfer DBH Gas Bumi 0.5%","value":{"anggaran":96882460000,"realisasi":124683590580,"persentase":"128,70 %"}},{"kodeakun":"612213","text":"Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus","value":{"anggaran":270583164000,"realisasi":340653879134,"persentase":"125,90 %"}},{"kodeakun":"612311","text":"Transfer DBH Pertambangan Umum - Iuran Tetap","value":{"anggaran":615858746000,"realisasi":574574040835,"persentase":"93,30 %"}},{"kodeakun":"612312","text":"Transfer DBH Pertambangan Umum - Royalti","value":{"anggaran":13670958915000,"realisasi":19233330257405,"persentase":"140,69 %"}},{"kodeakun":"612411","text":"Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah","value":{"anggaran":545092142000,"realisasi":814945712335,"persentase":"149,51 %"}},{"kodeakun":"612412","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Tetap","value":{"anggaran":14680315000,"realisasi":17093550748,"persentase":"116,44 %"}},{"kodeakun":"612413","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)","value":{"anggaran":703068000,"realisasi":897327311,"persentase":"127,63 %"}},{"kodeakun":"612511","text":"Transfer DBH Kehutanan - IIUPH\/IHPH","value":{"anggaran":164730247000,"realisasi":165979915021,"persentase":"100,76 %"}},{"kodeakun":"612512","text":"Transfer DBH Kehutanan - PSDH","value":{"anggaran":736821430000,"realisasi":706696713825,"persentase":"95,91 %"}},{"kodeakun":"612513","text":"Transfer DBH Kehutanan - Dana Reboisasi","value":{"anggaran":743479609000,"realisasi":741420178157,"persentase":"99,72 %"}},{"kodeakun":"612611","text":"Transfer DBH Perikanan","value":{"anggaran":480000648000,"realisasi":357622699445,"persentase":"74,50 %"}},{"kodeakun":"613111","text":"Transfer DBH Cukai Hasil Tembakau","value":{"anggaran":2964600000000,"realisasi":2922191165245,"persentase":"98,57 %"}}],"total":{"anggaran":89225342014000,"realisasi":93705559169270,"persentase":"105,02 %"}},"2":{"text":"DAU","child":[{"kodeakun":"621111","text":"Dana Alokasi Umum","value":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}}],"total":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}},"3":{"text":"DAK FISIK","child":[{"kodeakun":"631111","text":"Dana Alokasi Khusus","value":{"anggaran":31350835954000,"realisasi":29459738989382,"persentase":"93,97 %"}},{"kodeakun":"631211","text":"Dana Alokasi Khusus Infrastruktur Publik Daerah","value":{"anggaran":0,"realisasi":null,"persentase":"-"}},{"kodeakun":"631212","text":"Dana Alokasi Khusus Penugasan","value":{"anggaran":24463658880000,"realisasi":22492927163424,"persentase":"91,94 %"}},{"kodeakun":"631311","text":"Dana Alokasi Khusus Affirmasi","value":{"anggaran":6621768074000,"realisasi":6259405435103,"persentase":"94,53 %"}}],"total":{"anggaran":62436262908000,"realisasi":58212071587909,"persentase":"93,23 %"}},"6":{"text":"OTSUS","child":[{"kodeakun":"641111","text":"Dana Otonomi Khusus Provinsi Aceh","value":{"anggaran":8029791593000,"realisasi":8029791593000,"persentase":"100,00 %"}},{"kodeakun":"641211","text":"Dana Otonomi Khusus Provinsi Papua","value":{"anggaran":5620854115000,"realisasi":5620854115000,"persentase":"100,00 %"}},{"kodeakun":"641212","text":"Dana Tambahan Infrastruktur Provinsi Papua","value":{"anggaran":2400000000000,"realisasi":2400000000000,"persentase":"100,00 %"}},{"kodeakun":"641311","text":"Dana Otonomi Khusus Provinsi Papua Barat","value":{"anggaran":2408937478000,"realisasi":2408937478000,"persentase":"100,00 %"}},{"kodeakun":"641312","text":"Dana Tambahan Infrastruktur Provinsi Papua Barat","value":{"anggaran":1600000000000,"realisasi":1600000000000,"persentase":"100,00 %"}}],"total":{"anggaran":20059583186000,"realisasi":20059583186000,"persentase":"100,00 %"}},"7":{"text":"KEISTIMEWAAN DIY","child":[{"kodeakun":"642111","text":"Dana Keistimewaan Daerah Istimewa Yogyakarta","value":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}}],"total":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}},"5":{"text":"DID","child":[{"kodeakun":"643111","text":"Dana Insentif Daerah","value":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}}],"total":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}},"4":{"text":"DAK NON FISIK","child":[{"kodeakun":"654111","text":"Dana Tunjangan Profesi Guru PNSD","value":{"anggaran":56861747531798,"realisasi":55016720212332,"persentase":"96,76 %"}},{"kodeakun":"654112","text":"Dana Tunjangan Khusus Guru PNSD di Daerah Khusus","value":{"anggaran":1922303294676,"realisasi":1727102404861,"persentase":"89,85 %"}},{"kodeakun":"654211","text":"Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil","value":{"anggaran":879468864366,"realisasi":588007130574,"persentase":"66,86 %"}},{"kodeakun":"654311","text":"Dana Bantuan Operasional Sekolah","value":{"anggaran":46311243600000,"realisasi":44367351931006,"persentase":"95,80 %"}},{"kodeakun":"654711","text":"Dana Bantuan Operasional Kesehatan","value":{"anggaran":8551232700000,"realisasi":7454358890915,"persentase":"87,17 %"}},{"kodeakun":"654712","text":"Dana Bantuan Operasional Keluarga Berencana","value":{"anggaran":1808787300000,"realisasi":1753419417979,"persentase":"96,94 %"}},{"kodeakun":"654811","text":"Dana Peningkatan Kapasitas Koperasi","value":{"anggaran":100000000000,"realisasi":92221616897,"persentase":"92,22 %"}},{"kodeakun":"654814","text":"Dana Pelayanan Administrasi Kependudukan","value":{"anggaran":825000000000,"realisasi":745193742984,"persentase":"90,33 %"}},{"kodeakun":"654911","text":"Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini","value":{"anggaran":3741708600000,"realisasi":3555904601891,"persentase":"95,03 %"}}],"total":{"anggaran":121001491890840,"realisasi":115300279949439,"persentase":"95,29 %"}}},"total":{"anggaran":703712259647840,"realisasi":697657198541618,"persentase":"99,14 %"}},"2":{"text":"Dana Desa","child":{"1":{"text":"DANA DESA","child":[{"kodeakun":"662111","text":"Dana Desa","value":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}],"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"wilayah":"Nasional","judul":"POSTUR TKDD TA 2018","link_excel":"http:\/\/www.djpk.kemenkeu.go.id\/portal\/data\/csv\/2018\/--\/--\/tkdd","disclaimer":"Data bulan desember","header":["Alokasi","Realisasi"],"special_row":{"text":"Transfer ke daerah dan Dana Desa","total":{"anggaran":763712259647840,"realisasi":757533338915843,"persentase":"99,19 %"}}}')
dataflatten = flatten_json(datapemda)
print(dataflatten)

{'postur_1_text': 'Transfer ke Daerah', 'postur_1_child_1_text': 'DBH', 'postur_1_child_1_child_0_kodeakun': '611111', 'postur_1_child_1_child_0_text': 'Transfer DBH PPh Pasal 21', 'postur_1_child_1_child_0_value_anggaran': 32986586000000, 'postur_1_child_1_child_0_value_realisasi': 27458081283272, 'postur_1_child_1_child_0_value_persentase': '83,24 %', 'postur_1_child_1_child_1_kodeakun': '611112', 'postur_1_child_1_child_1_text': 'Transfer DBH PPh Pasal 25/29 OP', 'postur_1_child_1_child_1_value_anggaran': 4441882000000, 'postur_1_child_1_child_1_value_realisasi': 2533606329484, 'postur_1_child_1_child_1_value_persentase': '57,04 %', 'postur_1_child_1_child_2_kodeakun': '611211', 'postur_1_child_1_child_2_text': 'Transfer DBH PBB untuk Propinsi', 'postur_1_child_1_child_2_value_anggaran': 2857407829000, 'postur_1_child_1_child_2_value_realisasi': 2737579489820, 'postur_1_child_1_child_2_value_persentase': '95,81 %', 'postur_1_child_1_child_3_kodeakun': '611212', 'postur_1_child_1_chi

# Only save available kodeakun value

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import urllib3
import json

session = None
# Snippet Save to Google Drive
from datetime import datetime
from pytz import timezone
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

def set_global_session():
    global session
    if not session:
        session = requests.Session()
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        session.verify = False

def get_alldata():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            token = bsoup.find("input", {"name":"_token"})['value']
            options_provinsi = bsoup.find("select", {"id":"sel_prov"})
            allprovinsi = {}
            for option_provinsi in options_provinsi.find_all("option"):
                allprovinsi[option_provinsi['value']]=option_provinsi.text 
            return [token, allprovinsi]

def get_token():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            token = bsoup.find("input", {"name":"_token"})['value']
            return token

def get_provinsi():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            options_provinsi = bsoup.find("select", {"id":"sel_prov"})
            allprovinsi = []
            for option_provinsi in options_provinsi.find_all("option"):
                allprovinsi[option_provinsi['value']]=option_provinsi['value'].text
            return allprovinsi

def get_pemdas(provinsi):
    with session.get('http://www.djpk.kemenkeu.go.id/portal/pemda/' + provinsi) as response:
        if response.status_code==200 and len(response.text) != 0:
            pemdas = json.loads(response.text)
            return pemdas

def get_headers(token, tahun=2018):
    with session.post('http://www.djpk.kemenkeu.go.id/portal/filter', 
                data={'_token': token, 'data': 'tkdd', 'tahun': tahun, 'provinsi': '--', 'pemda': '--'}) as response:# query nasional saja as response
        if response.status_code==200 and len(response.text) != 0:
            print(response.text)
            # olah gaes
            # {"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Kabupaten\/kota","value":{"anggaran":398764072000,"realisasi":390261862177,"persentase":"97,87 %"}},{"kodeakun":"611215","text":"Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten\/Kota","value":{"anggaran":1736910100000,"realisasi":1650781100705,"persentase":"95,04 %"}},{"kodeakun":"612111","text":"Transfer DBH Minyak Bumi","value":{"anggaran":8581982674000,"realisasi":12376493484658,"persentase":"144,21 %"}},{"kodeakun":"612112","text":"Transfer DBH Minyak Bumi 0.5%","value":{"anggaran":282732216000,"realisasi":407714843494,"persentase":"144,21 %"}},{"kodeakun":"612113","text":"Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus","value":{"anggaran":370605160000,"realisasi":481008154249,"persentase":"129,79 %"}},{"kodeakun":"612211","text":"Transfer DBH Gas Bumi","value":{"anggaran":5966265026000,"realisasi":8745074752035,"persentase":"146,58 %"}},{"kodeakun":"612212","text":"Transfer DBH Gas Bumi 0.5%","value":{"anggaran":96882460000,"realisasi":124683590580,"persentase":"128,70 %"}},{"kodeakun":"612213","text":"Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus","value":{"anggaran":270583164000,"realisasi":340653879134,"persentase":"125,90 %"}},{"kodeakun":"612311","text":"Transfer DBH Pertambangan Umum - Iuran Tetap","value":{"anggaran":615858746000,"realisasi":574574040835,"persentase":"93,30 %"}},{"kodeakun":"612312","text":"Transfer DBH Pertambangan Umum - Royalti","value":{"anggaran":13670958915000,"realisasi":19233330257405,"persentase":"140,69 %"}},{"kodeakun":"612411","text":"Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah","value":{"anggaran":545092142000,"realisasi":814945712335,"persentase":"149,51 %"}},{"kodeakun":"612412","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Tetap","value":{"anggaran":14680315000,"realisasi":17093550748,"persentase":"116,44 %"}},{"kodeakun":"612413","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)","value":{"anggaran":703068000,"realisasi":897327311,"persentase":"127,63 %"}},{"kodeakun":"612511","text":"Transfer DBH Kehutanan - IIUPH\/IHPH","value":{"anggaran":164730247000,"realisasi":165979915021,"persentase":"100,76 %"}},{"kodeakun":"612512","text":"Transfer DBH Kehutanan - PSDH","value":{"anggaran":736821430000,"realisasi":706696713825,"persentase":"95,91 %"}},{"kodeakun":"612513","text":"Transfer DBH Kehutanan - Dana Reboisasi","value":{"anggaran":743479609000,"realisasi":741420178157,"persentase":"99,72 %"}},{"kodeakun":"612611","text":"Transfer DBH Perikanan","value":{"anggaran":480000648000,"realisasi":357622699445,"persentase":"74,50 %"}},{"kodeakun":"613111","text":"Transfer DBH Cukai Hasil Tembakau","value":{"anggaran":2964600000000,"realisasi":2922191165245,"persentase":"98,57 %"}}],"total":{"anggaran":89225342014000,"realisasi":93705559169270,"persentase":"105,02 %"}},"2":{"text":"DAU","child":[{"kodeakun":"621111","text":"Dana Alokasi Umum","value":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}}],"total":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}},"3":{"text":"DAK FISIK","child":[{"kodeakun":"631111","text":"Dana Alokasi Khusus","value":{"anggaran":31350835954000,"realisasi":29459738989382,"persentase":"93,97 %"}},{"kodeakun":"631211","text":"Dana Alokasi Khusus Infrastruktur Publik Daerah","value":{"anggaran":0,"realisasi":null,"persentase":"-"}},{"kodeakun":"631212","text":"Dana Alokasi Khusus Penugasan","value":{"anggaran":24463658880000,"realisasi":22492927163424,"persentase":"91,94 %"}},{"kodeakun":"631311","text":"Dana Alokasi Khusus Affirmasi","value":{"anggaran":6621768074000,"realisasi":6259405435103,"persentase":"94,53 %"}}],"total":{"anggaran":62436262908000,"realisasi":58212071587909,"persentase":"93,23 %"}},"6":{"text":"OTSUS","child":[{"kodeakun":"641111","text":"Dana Otonomi Khusus Provinsi Aceh","value":{"anggaran":8029791593000,"realisasi":8029791593000,"persentase":"100,00 %"}},{"kodeakun":"641211","text":"Dana Otonomi Khusus Provinsi Papua","value":{"anggaran":5620854115000,"realisasi":5620854115000,"persentase":"100,00 %"}},{"kodeakun":"641212","text":"Dana Tambahan Infrastruktur Provinsi Papua","value":{"anggaran":2400000000000,"realisasi":2400000000000,"persentase":"100,00 %"}},{"kodeakun":"641311","text":"Dana Otonomi Khusus Provinsi Papua Barat","value":{"anggaran":2408937478000,"realisasi":2408937478000,"persentase":"100,00 %"}},{"kodeakun":"641312","text":"Dana Tambahan Infrastruktur Provinsi Papua Barat","value":{"anggaran":1600000000000,"realisasi":1600000000000,"persentase":"100,00 %"}}],"total":{"anggaran":20059583186000,"realisasi":20059583186000,"persentase":"100,00 %"}},"7":{"text":"KEISTIMEWAAN DIY","child":[{"kodeakun":"642111","text":"Dana Keistimewaan Daerah Istimewa Yogyakarta","value":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}}],"total":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}},"5":{"text":"DID","child":[{"kodeakun":"643111","text":"Dana Insentif Daerah","value":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}}],"total":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}},"4":{"text":"DAK NON FISIK","child":[{"kodeakun":"654111","text":"Dana Tunjangan Profesi Guru PNSD","value":{"anggaran":56861747531798,"realisasi":55016720212332,"persentase":"96,76 %"}},{"kodeakun":"654112","text":"Dana Tunjangan Khusus Guru PNSD di Daerah Khusus","value":{"anggaran":1922303294676,"realisasi":1727102404861,"persentase":"89,85 %"}},{"kodeakun":"654211","text":"Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil","value":{"anggaran":879468864366,"realisasi":588007130574,"persentase":"66,86 %"}},{"kodeakun":"654311","text":"Dana Bantuan Operasional Sekolah","value":{"anggaran":46311243600000,"realisasi":44367351931006,"persentase":"95,80 %"}},{"kodeakun":"654711","text":"Dana Bantuan Operasional Kesehatan","value":{"anggaran":8551232700000,"realisasi":7454358890915,"persentase":"87,17 %"}},{"kodeakun":"654712","text":"Dana Bantuan Operasional Keluarga Berencana","value":{"anggaran":1808787300000,"realisasi":1753419417979,"persentase":"96,94 %"}},{"kodeakun":"654811","text":"Dana Peningkatan Kapasitas Koperasi","value":{"anggaran":100000000000,"realisasi":92221616897,"persentase":"92,22 %"}},{"kodeakun":"654814","text":"Dana Pelayanan Administrasi Kependudukan","value":{"anggaran":825000000000,"realisasi":745193742984,"persentase":"90,33 %"}},{"kodeakun":"654911","text":"Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini","value":{"anggaran":3741708600000,"realisasi":3555904601891,"persentase":"95,03 %"}}],"total":{"anggaran":121001491890840,"realisasi":115300279949439,"persentase":"95,29 %"}}},"total":{"anggaran":703712259647840,"realisasi":697657198541618,"persentase":"99,14 %"}},"2":{"text":"Dana Desa","child":{"1":{"text":"DANA DESA","child":[{"kodeakun":"662111","text":"Dana Desa","value":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}],"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"wilayah":"Nasional","judul":"POSTUR TKDD TA 2018","link_excel":"http:\/\/www.djpk.kemenkeu.go.id\/portal\/data\/csv\/2018\/--\/--\/tkdd","disclaimer":"Data bulan desember","header":["Alokasi","Realisasi"],"special_row":{"text":"Transfer ke daerah dan Dana Desa","total":{"anggaran":763712259647840,"realisasi":757533338915843,"persentase":"99,19 %"}}}

            kodeakun = 0

            return kodeakun

def get_data_tkdd(token, provinsi, pemda, tahun=2018):
    try:
        with session.post('http://www.djpk.kemenkeu.go.id/portal/filter', 
                    data={'_token': token, 'data': 'tkdd', 'tahun': tahun, 'provinsi': provinsi, 'pemda': pemda}) as response:
            if response.status_code==200 and len(response.text) != 0:
                return response.text
    except requests.exceptions.ConnectionError as e:
        return False
        pass
    except Exception as e:
        logger.error(e)
        randomtime = random.randint(1,5)
        logger.warn('ERROR - Retrying again website %s, retrying in %d secs' % (url, randomtime))
        time.sleep(randomtime)
        pass
        return False

def save_to_gdrive():
    path = F"/content/drive/My Drive/Colab Notebooks/Data DJSN/"
    filename = "data-tkdd-"+ datetime.now(timezone('Asia/Jakarta')).strftime("%Y-%m-%d--%H-%M") + ".csv"
    return True

def mapping(datapemda, provinsi):
    wilayah = datapemda['wilayah']
    disclaimer = datapemda['disclaimer']
    
    # 4
    dbh_pph21                 = datapemda['postur']['1']['child']['1']['child'][0]
    dbh_pph21_total_a         = dbh_pph21['value']['anggaran']
    dbh_pph21_total_r         = dbh_pph21['value']['realisasi']
    dbh_pph21_header_a          = dbh_pph21['kodeakun'] + "-" + dbh_pph21['text'] + " Anggaran"
    dbh_pph21_header_r          = dbh_pph21['kodeakun'] + "-" + dbh_pph21['text'] + " Realisasi"


    headers = ["no", 
              "tahun", "wilayah", "provinsi", "disclaimer"]
    return [headers, isi]
    
if __name__ == "__main__":
    set_global_session()
    data = []
    allprovinsi = []
    no = 1
    # tahun yg ada datanya hanya 2018-2019
    token, allprovinsi = get_alldata()
    # loop tahun
    for tahun in range(2018, 2021):
        if tahun == '2000':
            continue
        # loop provinsi
        for provinsi in allprovinsi:
            pemdas = get_pemdas(provinsi)
        
            # loop pemda per provinsi
            for pemda in pemdas.keys():
                print(get_data_tkdd(token, provinsi, pemda))
                datapemda = json.loads(get_data_tkdd(token, provinsi, pemda))
                #print(datapemda)
                if len(datapemda) == 0:
                     continue
                
                # create headers only in 1st loop
                if no == 1:
                    headers = mapping(datapemda, allprovinsi[provinsi])
                                
                # isi data
                isi = mapping(datapemda, allprovinsi[provinsi])
                data.append(isi[1])
                no = no + 1
                

    # save via DataFrame
    df = pd.DataFrame(data, headers=headers[0])
    df.to_csv( path+filename , index=False, encoding="utf-8")                

# Hanya Cari Kodeakun

In [ ]:
import json
def flatten_json(y):
    out = {}
    kodeakun = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
                print("Dict: " + name + " with key: " + a + " and value " + str(x[a]) )
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
                print("List: " + name)
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

datapemda = json.loads('{"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Kabupaten\/kota","value":{"anggaran":398764072000,"realisasi":390261862177,"persentase":"97,87 %"}},{"kodeakun":"611215","text":"Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten\/Kota","value":{"anggaran":1736910100000,"realisasi":1650781100705,"persentase":"95,04 %"}},{"kodeakun":"612111","text":"Transfer DBH Minyak Bumi","value":{"anggaran":8581982674000,"realisasi":12376493484658,"persentase":"144,21 %"}},{"kodeakun":"612112","text":"Transfer DBH Minyak Bumi 0.5%","value":{"anggaran":282732216000,"realisasi":407714843494,"persentase":"144,21 %"}},{"kodeakun":"612113","text":"Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus","value":{"anggaran":370605160000,"realisasi":481008154249,"persentase":"129,79 %"}},{"kodeakun":"612211","text":"Transfer DBH Gas Bumi","value":{"anggaran":5966265026000,"realisasi":8745074752035,"persentase":"146,58 %"}},{"kodeakun":"612212","text":"Transfer DBH Gas Bumi 0.5%","value":{"anggaran":96882460000,"realisasi":124683590580,"persentase":"128,70 %"}},{"kodeakun":"612213","text":"Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus","value":{"anggaran":270583164000,"realisasi":340653879134,"persentase":"125,90 %"}},{"kodeakun":"612311","text":"Transfer DBH Pertambangan Umum - Iuran Tetap","value":{"anggaran":615858746000,"realisasi":574574040835,"persentase":"93,30 %"}},{"kodeakun":"612312","text":"Transfer DBH Pertambangan Umum - Royalti","value":{"anggaran":13670958915000,"realisasi":19233330257405,"persentase":"140,69 %"}},{"kodeakun":"612411","text":"Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah","value":{"anggaran":545092142000,"realisasi":814945712335,"persentase":"149,51 %"}},{"kodeakun":"612412","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Tetap","value":{"anggaran":14680315000,"realisasi":17093550748,"persentase":"116,44 %"}},{"kodeakun":"612413","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)","value":{"anggaran":703068000,"realisasi":897327311,"persentase":"127,63 %"}},{"kodeakun":"612511","text":"Transfer DBH Kehutanan - IIUPH\/IHPH","value":{"anggaran":164730247000,"realisasi":165979915021,"persentase":"100,76 %"}},{"kodeakun":"612512","text":"Transfer DBH Kehutanan - PSDH","value":{"anggaran":736821430000,"realisasi":706696713825,"persentase":"95,91 %"}},{"kodeakun":"612513","text":"Transfer DBH Kehutanan - Dana Reboisasi","value":{"anggaran":743479609000,"realisasi":741420178157,"persentase":"99,72 %"}},{"kodeakun":"612611","text":"Transfer DBH Perikanan","value":{"anggaran":480000648000,"realisasi":357622699445,"persentase":"74,50 %"}},{"kodeakun":"613111","text":"Transfer DBH Cukai Hasil Tembakau","value":{"anggaran":2964600000000,"realisasi":2922191165245,"persentase":"98,57 %"}}],"total":{"anggaran":89225342014000,"realisasi":93705559169270,"persentase":"105,02 %"}},"2":{"text":"DAU","child":[{"kodeakun":"621111","text":"Dana Alokasi Umum","value":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}}],"total":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}},"3":{"text":"DAK FISIK","child":[{"kodeakun":"631111","text":"Dana Alokasi Khusus","value":{"anggaran":31350835954000,"realisasi":29459738989382,"persentase":"93,97 %"}},{"kodeakun":"631211","text":"Dana Alokasi Khusus Infrastruktur Publik Daerah","value":{"anggaran":0,"realisasi":null,"persentase":"-"}},{"kodeakun":"631212","text":"Dana Alokasi Khusus Penugasan","value":{"anggaran":24463658880000,"realisasi":22492927163424,"persentase":"91,94 %"}},{"kodeakun":"631311","text":"Dana Alokasi Khusus Affirmasi","value":{"anggaran":6621768074000,"realisasi":6259405435103,"persentase":"94,53 %"}}],"total":{"anggaran":62436262908000,"realisasi":58212071587909,"persentase":"93,23 %"}},"6":{"text":"OTSUS","child":[{"kodeakun":"641111","text":"Dana Otonomi Khusus Provinsi Aceh","value":{"anggaran":8029791593000,"realisasi":8029791593000,"persentase":"100,00 %"}},{"kodeakun":"641211","text":"Dana Otonomi Khusus Provinsi Papua","value":{"anggaran":5620854115000,"realisasi":5620854115000,"persentase":"100,00 %"}},{"kodeakun":"641212","text":"Dana Tambahan Infrastruktur Provinsi Papua","value":{"anggaran":2400000000000,"realisasi":2400000000000,"persentase":"100,00 %"}},{"kodeakun":"641311","text":"Dana Otonomi Khusus Provinsi Papua Barat","value":{"anggaran":2408937478000,"realisasi":2408937478000,"persentase":"100,00 %"}},{"kodeakun":"641312","text":"Dana Tambahan Infrastruktur Provinsi Papua Barat","value":{"anggaran":1600000000000,"realisasi":1600000000000,"persentase":"100,00 %"}}],"total":{"anggaran":20059583186000,"realisasi":20059583186000,"persentase":"100,00 %"}},"7":{"text":"KEISTIMEWAAN DIY","child":[{"kodeakun":"642111","text":"Dana Keistimewaan Daerah Istimewa Yogyakarta","value":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}}],"total":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}},"5":{"text":"DID","child":[{"kodeakun":"643111","text":"Dana Insentif Daerah","value":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}}],"total":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}},"4":{"text":"DAK NON FISIK","child":[{"kodeakun":"654111","text":"Dana Tunjangan Profesi Guru PNSD","value":{"anggaran":56861747531798,"realisasi":55016720212332,"persentase":"96,76 %"}},{"kodeakun":"654112","text":"Dana Tunjangan Khusus Guru PNSD di Daerah Khusus","value":{"anggaran":1922303294676,"realisasi":1727102404861,"persentase":"89,85 %"}},{"kodeakun":"654211","text":"Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil","value":{"anggaran":879468864366,"realisasi":588007130574,"persentase":"66,86 %"}},{"kodeakun":"654311","text":"Dana Bantuan Operasional Sekolah","value":{"anggaran":46311243600000,"realisasi":44367351931006,"persentase":"95,80 %"}},{"kodeakun":"654711","text":"Dana Bantuan Operasional Kesehatan","value":{"anggaran":8551232700000,"realisasi":7454358890915,"persentase":"87,17 %"}},{"kodeakun":"654712","text":"Dana Bantuan Operasional Keluarga Berencana","value":{"anggaran":1808787300000,"realisasi":1753419417979,"persentase":"96,94 %"}},{"kodeakun":"654811","text":"Dana Peningkatan Kapasitas Koperasi","value":{"anggaran":100000000000,"realisasi":92221616897,"persentase":"92,22 %"}},{"kodeakun":"654814","text":"Dana Pelayanan Administrasi Kependudukan","value":{"anggaran":825000000000,"realisasi":745193742984,"persentase":"90,33 %"}},{"kodeakun":"654911","text":"Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini","value":{"anggaran":3741708600000,"realisasi":3555904601891,"persentase":"95,03 %"}}],"total":{"anggaran":121001491890840,"realisasi":115300279949439,"persentase":"95,29 %"}}},"total":{"anggaran":703712259647840,"realisasi":697657198541618,"persentase":"99,14 %"}},"2":{"text":"Dana Desa","child":{"1":{"text":"DANA DESA","child":[{"kodeakun":"662111","text":"Dana Desa","value":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}],"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"wilayah":"Nasional","judul":"POSTUR TKDD TA 2018","link_excel":"http:\/\/www.djpk.kemenkeu.go.id\/portal\/data\/csv\/2018\/--\/--\/tkdd","disclaimer":"Data bulan desember","header":["Alokasi","Realisasi"],"special_row":{"text":"Transfer ke daerah dan Dana Desa","total":{"anggaran":763712259647840,"realisasi":757533338915843,"persentase":"99,19 %"}}}')
dataflatten = flatten_json(datapemda)
print(dataflatten)

Dict: postur_1_ with key: text and value Transfer ke Daerah
Dict: postur_1_child_1_ with key: text and value DBH
Dict: postur_1_child_1_child_0_ with key: kodeakun and value 611111
Dict: postur_1_child_1_child_0_ with key: text and value Transfer DBH PPh Pasal 21
Dict: postur_1_child_1_child_0_value_ with key: anggaran and value 32986586000000
Dict: postur_1_child_1_child_0_value_ with key: realisasi and value 27458081283272
Dict: postur_1_child_1_child_0_value_ with key: persentase and value 83,24 %
Dict: postur_1_child_1_child_0_ with key: value and value {'anggaran': 32986586000000, 'realisasi': 27458081283272, 'persentase': '83,24 %'}
List: postur_1_child_1_child_
Dict: postur_1_child_1_child_1_ with key: kodeakun and value 611112
Dict: postur_1_child_1_child_1_ with key: text and value Transfer DBH PPh Pasal 25/29 OP
Dict: postur_1_child_1_child_1_value_ with key: anggaran and value 4441882000000
Dict: postur_1_child_1_child_1_value_ with key: realisasi and value 2533606329484
Dic

In [ ]:
import json 
datapemda = json.loads('{"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Kabupaten\/kota","value":{"anggaran":398764072000,"realisasi":390261862177,"persentase":"97,87 %"}},{"kodeakun":"611215","text":"Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten\/Kota","value":{"anggaran":1736910100000,"realisasi":1650781100705,"persentase":"95,04 %"}},{"kodeakun":"612111","text":"Transfer DBH Minyak Bumi","value":{"anggaran":8581982674000,"realisasi":12376493484658,"persentase":"144,21 %"}},{"kodeakun":"612112","text":"Transfer DBH Minyak Bumi 0.5%","value":{"anggaran":282732216000,"realisasi":407714843494,"persentase":"144,21 %"}},{"kodeakun":"612113","text":"Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus","value":{"anggaran":370605160000,"realisasi":481008154249,"persentase":"129,79 %"}},{"kodeakun":"612211","text":"Transfer DBH Gas Bumi","value":{"anggaran":5966265026000,"realisasi":8745074752035,"persentase":"146,58 %"}},{"kodeakun":"612212","text":"Transfer DBH Gas Bumi 0.5%","value":{"anggaran":96882460000,"realisasi":124683590580,"persentase":"128,70 %"}},{"kodeakun":"612213","text":"Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus","value":{"anggaran":270583164000,"realisasi":340653879134,"persentase":"125,90 %"}},{"kodeakun":"612311","text":"Transfer DBH Pertambangan Umum - Iuran Tetap","value":{"anggaran":615858746000,"realisasi":574574040835,"persentase":"93,30 %"}},{"kodeakun":"612312","text":"Transfer DBH Pertambangan Umum - Royalti","value":{"anggaran":13670958915000,"realisasi":19233330257405,"persentase":"140,69 %"}},{"kodeakun":"612411","text":"Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah","value":{"anggaran":545092142000,"realisasi":814945712335,"persentase":"149,51 %"}},{"kodeakun":"612412","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Tetap","value":{"anggaran":14680315000,"realisasi":17093550748,"persentase":"116,44 %"}},{"kodeakun":"612413","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)","value":{"anggaran":703068000,"realisasi":897327311,"persentase":"127,63 %"}},{"kodeakun":"612511","text":"Transfer DBH Kehutanan - IIUPH\/IHPH","value":{"anggaran":164730247000,"realisasi":165979915021,"persentase":"100,76 %"}},{"kodeakun":"612512","text":"Transfer DBH Kehutanan - PSDH","value":{"anggaran":736821430000,"realisasi":706696713825,"persentase":"95,91 %"}},{"kodeakun":"612513","text":"Transfer DBH Kehutanan - Dana Reboisasi","value":{"anggaran":743479609000,"realisasi":741420178157,"persentase":"99,72 %"}},{"kodeakun":"612611","text":"Transfer DBH Perikanan","value":{"anggaran":480000648000,"realisasi":357622699445,"persentase":"74,50 %"}},{"kodeakun":"613111","text":"Transfer DBH Cukai Hasil Tembakau","value":{"anggaran":2964600000000,"realisasi":2922191165245,"persentase":"98,57 %"}}],"total":{"anggaran":89225342014000,"realisasi":93705559169270,"persentase":"105,02 %"}},"2":{"text":"DAU","child":[{"kodeakun":"621111","text":"Dana Alokasi Umum","value":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}}],"total":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}},"3":{"text":"DAK FISIK","child":[{"kodeakun":"631111","text":"Dana Alokasi Khusus","value":{"anggaran":31350835954000,"realisasi":29459738989382,"persentase":"93,97 %"}},{"kodeakun":"631211","text":"Dana Alokasi Khusus Infrastruktur Publik Daerah","value":{"anggaran":0,"realisasi":null,"persentase":"-"}},{"kodeakun":"631212","text":"Dana Alokasi Khusus Penugasan","value":{"anggaran":24463658880000,"realisasi":22492927163424,"persentase":"91,94 %"}},{"kodeakun":"631311","text":"Dana Alokasi Khusus Affirmasi","value":{"anggaran":6621768074000,"realisasi":6259405435103,"persentase":"94,53 %"}}],"total":{"anggaran":62436262908000,"realisasi":58212071587909,"persentase":"93,23 %"}},"6":{"text":"OTSUS","child":[{"kodeakun":"641111","text":"Dana Otonomi Khusus Provinsi Aceh","value":{"anggaran":8029791593000,"realisasi":8029791593000,"persentase":"100,00 %"}},{"kodeakun":"641211","text":"Dana Otonomi Khusus Provinsi Papua","value":{"anggaran":5620854115000,"realisasi":5620854115000,"persentase":"100,00 %"}},{"kodeakun":"641212","text":"Dana Tambahan Infrastruktur Provinsi Papua","value":{"anggaran":2400000000000,"realisasi":2400000000000,"persentase":"100,00 %"}},{"kodeakun":"641311","text":"Dana Otonomi Khusus Provinsi Papua Barat","value":{"anggaran":2408937478000,"realisasi":2408937478000,"persentase":"100,00 %"}},{"kodeakun":"641312","text":"Dana Tambahan Infrastruktur Provinsi Papua Barat","value":{"anggaran":1600000000000,"realisasi":1600000000000,"persentase":"100,00 %"}}],"total":{"anggaran":20059583186000,"realisasi":20059583186000,"persentase":"100,00 %"}},"7":{"text":"KEISTIMEWAAN DIY","child":[{"kodeakun":"642111","text":"Dana Keistimewaan Daerah Istimewa Yogyakarta","value":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}}],"total":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}},"5":{"text":"DID","child":[{"kodeakun":"643111","text":"Dana Insentif Daerah","value":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}}],"total":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}},"4":{"text":"DAK NON FISIK","child":[{"kodeakun":"654111","text":"Dana Tunjangan Profesi Guru PNSD","value":{"anggaran":56861747531798,"realisasi":55016720212332,"persentase":"96,76 %"}},{"kodeakun":"654112","text":"Dana Tunjangan Khusus Guru PNSD di Daerah Khusus","value":{"anggaran":1922303294676,"realisasi":1727102404861,"persentase":"89,85 %"}},{"kodeakun":"654211","text":"Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil","value":{"anggaran":879468864366,"realisasi":588007130574,"persentase":"66,86 %"}},{"kodeakun":"654311","text":"Dana Bantuan Operasional Sekolah","value":{"anggaran":46311243600000,"realisasi":44367351931006,"persentase":"95,80 %"}},{"kodeakun":"654711","text":"Dana Bantuan Operasional Kesehatan","value":{"anggaran":8551232700000,"realisasi":7454358890915,"persentase":"87,17 %"}},{"kodeakun":"654712","text":"Dana Bantuan Operasional Keluarga Berencana","value":{"anggaran":1808787300000,"realisasi":1753419417979,"persentase":"96,94 %"}},{"kodeakun":"654811","text":"Dana Peningkatan Kapasitas Koperasi","value":{"anggaran":100000000000,"realisasi":92221616897,"persentase":"92,22 %"}},{"kodeakun":"654814","text":"Dana Pelayanan Administrasi Kependudukan","value":{"anggaran":825000000000,"realisasi":745193742984,"persentase":"90,33 %"}},{"kodeakun":"654911","text":"Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini","value":{"anggaran":3741708600000,"realisasi":3555904601891,"persentase":"95,03 %"}}],"total":{"anggaran":121001491890840,"realisasi":115300279949439,"persentase":"95,29 %"}}},"total":{"anggaran":703712259647840,"realisasi":697657198541618,"persentase":"99,14 %"}},"2":{"text":"Dana Desa","child":{"1":{"text":"DANA DESA","child":[{"kodeakun":"662111","text":"Dana Desa","value":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}],"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"wilayah":"Nasional","judul":"POSTUR TKDD TA 2018","link_excel":"http:\/\/www.djpk.kemenkeu.go.id\/portal\/data\/csv\/2018\/--\/--\/tkdd","disclaimer":"Data bulan desember","header":["Alokasi","Realisasi"],"special_row":{"text":"Transfer ke daerah dan Dana Desa","total":{"anggaran":763712259647840,"realisasi":757533338915843,"persentase":"99,19 %"}}}')
for a in datapemda:
    print(a)
    
def cari_kodeakun(y):
    out = {}

    def extract_kodeakun(x):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
                print("Dict: " + name + " with Values: " + a)
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
                print("List: " + name)
        else:
            out[name[:-1]] = x


postur
wilayah
judul
link_excel
disclaimer
header
special_row


# Saving excel TKDD

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import urllib3
import json

session = None
# Snippet Save to Google Drive
from datetime import datetime
from pytz import timezone
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

def set_global_session():
    global session
    if not session:
        session = requests.Session()
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        session.verify = False

def get_alldata():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            token = bsoup.find("input", {"name":"_token"})['value']
            options_provinsi = bsoup.find("select", {"id":"sel_prov"})
            allprovinsi = {}
            for option_provinsi in options_provinsi.find_all("option"):
                allprovinsi[option_provinsi['value']]=option_provinsi.text 
            return [token, allprovinsi]

def get_token():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            token = bsoup.find("input", {"name":"_token"})['value']
            return token

def get_provinsi():
    with session.get('http://www.djpk.kemenkeu.go.id/portal/data/apbd') as response:
        if response.status_code==200 and len(response.text) != 0:
            bsoup = BeautifulSoup(response.text, 'html.parser')
            options_provinsi = bsoup.find("select", {"id":"sel_prov"})
            allprovinsi = []
            for option_provinsi in options_provinsi.find_all("option"):
                allprovinsi[option_provinsi['value']]=option_provinsi['value'].text
            return allprovinsi

def get_pemdas(provinsi):
    with session.get('http://www.djpk.kemenkeu.go.id/portal/pemda/' + provinsi) as response:
        if response.status_code==200 and len(response.text) != 0:
            pemdas = json.loads(response.text)
            return pemdas

def get_headers(token, tahun=2018):
    with session.post('http://www.djpk.kemenkeu.go.id/portal/filter', 
                data={'_token': token, 'data': 'tkdd', 'tahun': tahun, 'provinsi': '--', 'pemda': '--'}) as response:# query nasional saja as response
        if response.status_code==200 and len(response.text) != 0:
            print(response.text)
            # olah gaes
            # {"postur":{"1":{"text":"Transfer ke Daerah","child":{"1":{"text":"DBH","child":[{"kodeakun":"611111","text":"Transfer DBH PPh Pasal 21","value":{"anggaran":32986586000000,"realisasi":27458081283272,"persentase":"83,24 %"}},{"kodeakun":"611112","text":"Transfer DBH PPh Pasal 25\/29 OP","value":{"anggaran":4441882000000,"realisasi":2533606329484,"persentase":"57,04 %"}},{"kodeakun":"611211","text":"Transfer DBH PBB untuk Propinsi","value":{"anggaran":2857407829000,"realisasi":2737579489820,"persentase":"95,81 %"}},{"kodeakun":"611212","text":"Transfer DBH PBB untuk Kabupaten\/Kota","value":{"anggaran":11211563981000,"realisasi":10840669654158,"persentase":"96,69 %"}},{"kodeakun":"611213","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Propinsi","value":{"anggaran":86252212000,"realisasi":84199185177,"persentase":"97,62 %"}},{"kodeakun":"611214","text":"Transfer DBH Biaya\/Upah Pungut PBB untuk Kabupaten\/kota","value":{"anggaran":398764072000,"realisasi":390261862177,"persentase":"97,87 %"}},{"kodeakun":"611215","text":"Transfer DBH PBB Bagian Pemerintah Pusat yang Dikembalikan Sama Rata ke Kabupaten\/Kota","value":{"anggaran":1736910100000,"realisasi":1650781100705,"persentase":"95,04 %"}},{"kodeakun":"612111","text":"Transfer DBH Minyak Bumi","value":{"anggaran":8581982674000,"realisasi":12376493484658,"persentase":"144,21 %"}},{"kodeakun":"612112","text":"Transfer DBH Minyak Bumi 0.5%","value":{"anggaran":282732216000,"realisasi":407714843494,"persentase":"144,21 %"}},{"kodeakun":"612113","text":"Tambahan Transfer DBH Minyak Bumi Dalam Rangka Otonomi Khusus","value":{"anggaran":370605160000,"realisasi":481008154249,"persentase":"129,79 %"}},{"kodeakun":"612211","text":"Transfer DBH Gas Bumi","value":{"anggaran":5966265026000,"realisasi":8745074752035,"persentase":"146,58 %"}},{"kodeakun":"612212","text":"Transfer DBH Gas Bumi 0.5%","value":{"anggaran":96882460000,"realisasi":124683590580,"persentase":"128,70 %"}},{"kodeakun":"612213","text":"Tambahan Transfer DBH Gas Bumi Dalam Rangka Otsus","value":{"anggaran":270583164000,"realisasi":340653879134,"persentase":"125,90 %"}},{"kodeakun":"612311","text":"Transfer DBH Pertambangan Umum - Iuran Tetap","value":{"anggaran":615858746000,"realisasi":574574040835,"persentase":"93,30 %"}},{"kodeakun":"612312","text":"Transfer DBH Pertambangan Umum - Royalti","value":{"anggaran":13670958915000,"realisasi":19233330257405,"persentase":"140,69 %"}},{"kodeakun":"612411","text":"Transfer DBH Pertambangan Panas Bumi - Setoran Bagian Pemerintah","value":{"anggaran":545092142000,"realisasi":814945712335,"persentase":"149,51 %"}},{"kodeakun":"612412","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Tetap","value":{"anggaran":14680315000,"realisasi":17093550748,"persentase":"116,44 %"}},{"kodeakun":"612413","text":"Transfer DBH Pertambangan Panas Bumi - Iuran Produksi (Royalti)","value":{"anggaran":703068000,"realisasi":897327311,"persentase":"127,63 %"}},{"kodeakun":"612511","text":"Transfer DBH Kehutanan - IIUPH\/IHPH","value":{"anggaran":164730247000,"realisasi":165979915021,"persentase":"100,76 %"}},{"kodeakun":"612512","text":"Transfer DBH Kehutanan - PSDH","value":{"anggaran":736821430000,"realisasi":706696713825,"persentase":"95,91 %"}},{"kodeakun":"612513","text":"Transfer DBH Kehutanan - Dana Reboisasi","value":{"anggaran":743479609000,"realisasi":741420178157,"persentase":"99,72 %"}},{"kodeakun":"612611","text":"Transfer DBH Perikanan","value":{"anggaran":480000648000,"realisasi":357622699445,"persentase":"74,50 %"}},{"kodeakun":"613111","text":"Transfer DBH Cukai Hasil Tembakau","value":{"anggaran":2964600000000,"realisasi":2922191165245,"persentase":"98,57 %"}}],"total":{"anggaran":89225342014000,"realisasi":93705559169270,"persentase":"105,02 %"}},"2":{"text":"DAU","child":[{"kodeakun":"621111","text":"Dana Alokasi Umum","value":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}}],"total":{"anggaran":401489579649000,"realisasi":401489579649000,"persentase":"100,00 %"}},"3":{"text":"DAK FISIK","child":[{"kodeakun":"631111","text":"Dana Alokasi Khusus","value":{"anggaran":31350835954000,"realisasi":29459738989382,"persentase":"93,97 %"}},{"kodeakun":"631211","text":"Dana Alokasi Khusus Infrastruktur Publik Daerah","value":{"anggaran":0,"realisasi":null,"persentase":"-"}},{"kodeakun":"631212","text":"Dana Alokasi Khusus Penugasan","value":{"anggaran":24463658880000,"realisasi":22492927163424,"persentase":"91,94 %"}},{"kodeakun":"631311","text":"Dana Alokasi Khusus Affirmasi","value":{"anggaran":6621768074000,"realisasi":6259405435103,"persentase":"94,53 %"}}],"total":{"anggaran":62436262908000,"realisasi":58212071587909,"persentase":"93,23 %"}},"6":{"text":"OTSUS","child":[{"kodeakun":"641111","text":"Dana Otonomi Khusus Provinsi Aceh","value":{"anggaran":8029791593000,"realisasi":8029791593000,"persentase":"100,00 %"}},{"kodeakun":"641211","text":"Dana Otonomi Khusus Provinsi Papua","value":{"anggaran":5620854115000,"realisasi":5620854115000,"persentase":"100,00 %"}},{"kodeakun":"641212","text":"Dana Tambahan Infrastruktur Provinsi Papua","value":{"anggaran":2400000000000,"realisasi":2400000000000,"persentase":"100,00 %"}},{"kodeakun":"641311","text":"Dana Otonomi Khusus Provinsi Papua Barat","value":{"anggaran":2408937478000,"realisasi":2408937478000,"persentase":"100,00 %"}},{"kodeakun":"641312","text":"Dana Tambahan Infrastruktur Provinsi Papua Barat","value":{"anggaran":1600000000000,"realisasi":1600000000000,"persentase":"100,00 %"}}],"total":{"anggaran":20059583186000,"realisasi":20059583186000,"persentase":"100,00 %"}},"7":{"text":"KEISTIMEWAAN DIY","child":[{"kodeakun":"642111","text":"Dana Keistimewaan Daerah Istimewa Yogyakarta","value":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}}],"total":{"anggaran":1000000000000,"realisasi":650000000000,"persentase":"65,00 %"}},"5":{"text":"DID","child":[{"kodeakun":"643111","text":"Dana Insentif Daerah","value":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}}],"total":{"anggaran":8500000000000,"realisasi":8240125000000,"persentase":"96,94 %"}},"4":{"text":"DAK NON FISIK","child":[{"kodeakun":"654111","text":"Dana Tunjangan Profesi Guru PNSD","value":{"anggaran":56861747531798,"realisasi":55016720212332,"persentase":"96,76 %"}},{"kodeakun":"654112","text":"Dana Tunjangan Khusus Guru PNSD di Daerah Khusus","value":{"anggaran":1922303294676,"realisasi":1727102404861,"persentase":"89,85 %"}},{"kodeakun":"654211","text":"Dana Tambahan Penghasilan Guru Pegawai Negeri Sipil","value":{"anggaran":879468864366,"realisasi":588007130574,"persentase":"66,86 %"}},{"kodeakun":"654311","text":"Dana Bantuan Operasional Sekolah","value":{"anggaran":46311243600000,"realisasi":44367351931006,"persentase":"95,80 %"}},{"kodeakun":"654711","text":"Dana Bantuan Operasional Kesehatan","value":{"anggaran":8551232700000,"realisasi":7454358890915,"persentase":"87,17 %"}},{"kodeakun":"654712","text":"Dana Bantuan Operasional Keluarga Berencana","value":{"anggaran":1808787300000,"realisasi":1753419417979,"persentase":"96,94 %"}},{"kodeakun":"654811","text":"Dana Peningkatan Kapasitas Koperasi","value":{"anggaran":100000000000,"realisasi":92221616897,"persentase":"92,22 %"}},{"kodeakun":"654814","text":"Dana Pelayanan Administrasi Kependudukan","value":{"anggaran":825000000000,"realisasi":745193742984,"persentase":"90,33 %"}},{"kodeakun":"654911","text":"Dana Bantuan Operasional Penyelenggaraan - Pendidikan Anak Usia Dini","value":{"anggaran":3741708600000,"realisasi":3555904601891,"persentase":"95,03 %"}}],"total":{"anggaran":121001491890840,"realisasi":115300279949439,"persentase":"95,29 %"}}},"total":{"anggaran":703712259647840,"realisasi":697657198541618,"persentase":"99,14 %"}},"2":{"text":"Dana Desa","child":{"1":{"text":"DANA DESA","child":[{"kodeakun":"662111","text":"Dana Desa","value":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}],"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"total":{"anggaran":60000000000000,"realisasi":59876140374225,"persentase":"99,79 %"}}},"wilayah":"Nasional","judul":"POSTUR TKDD TA 2018","link_excel":"http:\/\/www.djpk.kemenkeu.go.id\/portal\/data\/csv\/2018\/--\/--\/tkdd","disclaimer":"Data bulan desember","header":["Alokasi","Realisasi"],"special_row":{"text":"Transfer ke daerah dan Dana Desa","total":{"anggaran":763712259647840,"realisasi":757533338915843,"persentase":"99,19 %"}}}

            kodeakun = 0

            return kodeakun

def get_data_tkdd(token, provinsi, pemda, tahun=2018):
    try:
        with session.post('http://www.djpk.kemenkeu.go.id/portal/filter', 
                    data={'_token': token, 'data': 'tkdd', 'tahun': tahun, 'provinsi': provinsi, 'pemda': pemda}) as response:
            if response.status_code==200 and len(response.text) != 0:
                return response.text
    except requests.exceptions.ConnectionError as e:
        return False
        pass
    except Exception as e:
        logger.error(e)
        randomtime = random.randint(1,5)
        logger.warn('ERROR - Retrying again website %s, retrying in %d secs' % (url, randomtime))
        time.sleep(randomtime)
        pass
        return False

def save_csv_tkdd(provinsi, pemda, tahun, fullpathfilename):
    try:
        url = 'http://www.djpk.kemenkeu.go.id/portal/data/csv/'+ str(tahun)+'/'+str(provinsi)+'/'+str(pemda)+'/tkdd'
        with session.get(url) as response:
            response.raise_for_status()
            open(fullpathfilename, 'wb').write(response.content)
    except requests.exceptions.ConnectionError as e:
        return False
        pass
    except Exception as e:
        logger.error(e)
        randomtime = random.randint(1,5)
        logger.warn('ERROR - Retrying again website %s, retrying in %d secs' % (url, randomtime))
        time.sleep(randomtime)
        pass
        return False

def save_to_gdrive():
    path = F"/content/drive/My Drive/Colab Notebooks/Portal TKDD/"
    filename = "data-tkdd-"+ datetime.now(timezone('Asia/Jakarta')).strftime("%Y-%m-%d--%H-%M") + ".csv"
    return True

    
   
if __name__ == "__main__":
    set_global_session()
    allprovinsi = []
    path = F"/content/drive/My Drive/Colab Notebooks/Portal TKDD/"
    # tahun yg ada datanya hanya 2018-2019
    token, allprovinsi = get_alldata()
    # loop tahun
    tahun = 2021
    # loop provinsi
    for provinsi in allprovinsi:
        pemdas = get_pemdas(provinsi)
    
        # loop pemda per provinsi
        for pemda in pemdas.keys():
            
            filename = "data-tkdd-"+ str(tahun) + "-" + str(provinsi) + "-" + str(pemda) + ".xlsx"
            save_csv_tkdd(provinsi, pemda, tahun, path+filename)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ConnectionError: ignored

# Save single file TKDD

In [ ]:
import requests
import urllib3

from google.colab import drive
drive.mount('/content/drive')

def set_global_session():
    global session
    if not session:
        session = requests.Session()
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        session.verify = False

# http://www.djpk.kemenkeu.go.id/portal/data/csv/2021/33/00/tkdd
tahun = 2021
provinsi = '33'
pemda = '00'

session = None
path = F"/content/drive/My Drive/Colab Notebooks/Portal TKDD/"
filename = "data-tkdd-"+ str(tahun) + "-" + str(provinsi) + "-" + str(pemda) + ".xlsx"

set_global_session()
with session.get('http://www.djpk.kemenkeu.go.id/portal/data/csv/'+ str(tahun)+'/'+str(provinsi)+'/'+str(pemda)+'/tkdd') as response:
    response.raise_for_status()
    open(path+filename, 'wb').write(response.content)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
